In [ ]:
# (Paste the simple column-checking snippet here in a python interpreter or jupyter cell)
import pandas as pd
parquet_metadata_file = "integrated_asgard_gv_ortho_interpro.parquet"
try:
    df_meta = pd.read_parquet(parquet_metadata_file, engine='auto')
    print("Columns found in the parquet file:")
    print(df_meta.columns.tolist())
except FileNotFoundError:
    print(f"Error: Parquet file not found at '{parquet_metadata_file}'")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd
from Bio import SeqIO
import sys
import os

def assemble_core_csv_with_fasta(
    parquet_file: str,
    pdb_results_file: str,
    filtered_fasta_file: str, # Path to the FASTA with sequences
    output_csv_file: str
) -> None:
    """
    Assembles the initial core data CSV by combining metadata from a parquet file
    with sequence search hit information and sequences from a FASTA file.

    Args:
        parquet_file: Path to the input parquet file containing integrated metadata.
        pdb_results_file: Path to the MMseqs2 results file from the PDB search.
        filtered_fasta_file: Path to the FASTA file containing sequences for filtering.
        output_csv_file: Path where the assembled CSV file will be written.
    """
    print("Starting CSV assembly...")
    print(f"  Input Parquet: {parquet_file}")
    print(f"  PDB Results: {pdb_results_file}")
    print(f"  Input FASTA: {filtered_fasta_file}")
    print(f"  Output CSV: {output_csv_file}")

    # --- 1. Load main metadata from Parquet ---
    try:
        print(f"\nReading metadata from {parquet_file}...")
        df = pd.read_parquet(parquet_file, engine='auto')
        print(f"  Loaded {len(df):,} records from parquet file.")
        print(f"  Parquet Columns Found: {df.columns.tolist()}")

        # --- Column mapping based on user-provided list ---
        required_cols = {
            'ProteinID': 'ProteinID',
            'GenomeID': 'Source_Genome_Assembly_Accession',
            'OriginalName': 'Source_Protein_Annotation',
            'Dataset': 'Source_Dataset',
            'Phylum': 'Taxonomy_Phylum',
            'Taxonomy': 'Taxonomy_Species',
            'OG_ID': 'Orthogroup',
            'All_IPR_Hits': 'IPR_Signatures',
            'All_GO_Terms': 'IPR_GO_Terms',
            'Num_Domains': 'Num_Domains',
            'Domain_Architecture': 'Domain_Architecture',
            'Type': 'Type',
            'Is_Hypothetical': 'Is_Hypothetical',
            'Has_Known_Structure': 'Has_Known_Structure',
            'ncbi_taxid_placeholder': 'NCBI_TaxID',
            'taxonomy_supergroup_placeholder': 'Taxonomy_Supergroup',
            'taxonomy_class_placeholder': 'Taxonomy_Class',
            'uniprot_ac_placeholder': 'UniProtKB_AC',
            'afdb_status_placeholder': 'AFDB_Status'
        }
        protein_id_col_name = 'ProteinID' # Confirmed column name
        if protein_id_col_name not in df.columns:
             print(f"Error: Crucial column for ProteinID ('{protein_id_col_name}') not found in {parquet_file}.")
             sys.exit(1)
        print(f"  Using '{protein_id_col_name}' as the ProteinID column from parquet.")

    except FileNotFoundError:
        print(f"Error: Parquet file not found at {parquet_file}")
        sys.exit(1)
    except Exception as e:
        print(f"Error reading or processing parquet file {parquet_file}: {e}")
        sys.exit(1)

    # --- 2. Load Sequences from FASTA ---
    sequences = {}
    try:
        print(f"\nReading sequences from {filtered_fasta_file}...")
        count_fasta = 0
        with open(filtered_fasta_file, 'r') as fastafile:
             for record in SeqIO.parse(fastafile, "fasta"):
                  sequences[record.id] = str(record.seq)
                  count_fasta += 1
        print(f"  Loaded {len(sequences):,} sequences from FASTA file.")
        if count_fasta == 0: print(f"  Warning: No sequences found in {filtered_fasta_file}.")
    except FileNotFoundError:
        print(f"Error: Filtered FASTA file not found at {filtered_fasta_file}")
        sys.exit(1)
    except Exception as e:
        print(f"Error reading FASTA file {filtered_fasta_file}: {e}")
        sys.exit(1)

    # --- 3. Load PDB hit IDs ---
    pdb_hit_ids = set()
    try:
        print(f"\nReading PDB hit IDs from {pdb_results_file}...")
        if os.path.exists(pdb_results_file) and os.path.getsize(pdb_results_file) > 0:
             pdb_hits_df = pd.read_csv(
                  pdb_results_file, sep='\t', header=None, usecols=[0],
                  names=['query'], comment='#', low_memory=False
             )
             pdb_hit_ids = set(pdb_hits_df['query'].dropna().unique())
             print(f"  Found {len(pdb_hit_ids):,} unique ProteinIDs with PDB hits.")
        else:
             print(f"  PDB results file '{pdb_results_file}' not found or is empty. Assuming no PDB hits.")
    except Exception as e:
        print(f"Error reading PDB results file {pdb_results_file}: {e}")
        pdb_hit_ids = set()

    # --- 4. Prepare DataFrame for Output ---
    print("\nPreparing final DataFrame...")
    unique_protein_ids_in_parquet = df[protein_id_col_name].unique()
    output_df = pd.DataFrame(index=unique_protein_ids_in_parquet)
    output_df.index.name = 'ProteinID_Index' # Temporary index name

    # --- Map and add columns from the parquet file ---
    print("  Mapping columns from parquet to output DataFrame...")
    for parquet_col_name, output_col_name in required_cols.items():
        if parquet_col_name == protein_id_col_name:
            print(f"  Skipping mapping for '{parquet_col_name}' as it's the primary ID.")
            continue

        if parquet_col_name in df.columns:
            try:
                mapping_series = df.drop_duplicates(subset=[protein_id_col_name]).set_index(protein_id_col_name)[parquet_col_name]
                output_df[output_col_name] = output_df.index.map(mapping_series)
            except KeyError as e:
                print(f"!!! Internal KeyError during mapping for column '{parquet_col_name}': {e}")
                output_df[output_col_name] = pd.NA
            except Exception as e:
                print(f"!!! Error during mapping for column '{parquet_col_name}': {e}")
                output_df[output_col_name] = pd.NA
        else:
            print(f"  Info: Column '{parquet_col_name}' not found in input parquet. Adding empty column '{output_col_name}'.")
            output_df[output_col_name] = pd.NA

    # Add Sequence column from the dictionary
    output_df['Sequence'] = output_df.index.map(sequences)
    missing_seq_count = output_df['Sequence'].isna().sum()
    if missing_seq_count > 0:
         print(f"  Info: {missing_seq_count:,} proteins from parquet file did not have a sequence in the filtered FASTA file (expected).")
    print("  Added 'Sequence' column from FASTA.")

    # <<< CHANGE START >>>
    # Calculate Length robustly, handling potential non-string values
    print("  Calculating 'Length' column...")
    output_df['Length'] = output_df['Sequence'].apply(lambda x: len(x) if isinstance(x, str) else 0).astype(int)
    # <<< CHANGE END >>>
    print("  Calculated 'Length' column.")


    # Add Sequence Search Hit Flags
    output_df['SeqSearch_PDB_Hit'] = output_df.index.isin(pdb_hit_ids)
    output_df['SeqSearch_AFDB_Hit'] = False # Based on previous results
    output_df['SeqSearch_MGnify_Hit'] = False # Search was skipped
    print("  Added 'SeqSearch_*_Hit' flag columns.")

    # Reset index to make ProteinID a regular column again
    output_df.reset_index(inplace=True)
    output_df.rename(columns={'ProteinID_Index': 'ProteinID'}, inplace=True) # Rename index col

    # --- Filter rows: Keep only proteins present in the filtered FASTA ---
    initial_rows = len(output_df)
    output_df = output_df[output_df['ProteinID'].isin(sequences.keys())].copy()
    rows_after_fasta_filter = len(output_df)
    print(f"  Filtered DataFrame to keep only proteins present in '{filtered_fasta_file}'.")
    print(f"  Rows before FASTA filter: {initial_rows:,}. Rows after: {rows_after_fasta_filter:,}")

    # --- Reorder columns ---
    desired_order = [
        'ProteinID', 'Sequence', 'Length', 'Source_Dataset', 'Dataset',
        'Source_Genome_Assembly_Accession', 'GenomeID',
        'Source_Protein_Annotation', 'OriginalName',
        'NCBI_TaxID', 'Taxonomy_Supergroup', 'Taxonomy_Phylum', 'Phylum',
        'Taxonomy_Class', 'Taxonomy_Species', 'Taxonomy',
        'Orthogroup', 'OG_ID',
        'IPR_Signatures', 'All_IPR_Hits',
        'IPR_GO_Terms', 'All_GO_Terms',
        'UniProtKB_AC', 'AFDB_Status',
        'SeqSearch_PDB_Hit', 'SeqSearch_AFDB_Hit', 'SeqSearch_MGnify_Hit',
        'Num_Domains', 'Domain_Architecture', 'Type', 'Is_Hypothetical', 'Has_Known_Structure'
    ]
    present_columns = [col for col in desired_order if col in output_df.columns]
    present_columns.extend([col for col in output_df.columns if col not in present_columns])
    final_columns = pd.Series(present_columns).drop_duplicates().tolist() # Ensure unique cols
    output_df = output_df[final_columns]
    print(f"  Final DataFrame has {len(output_df.columns)} columns and {len(output_df):,} rows.")

    # --- 5. Write to CSV ---
    try:
        print(f"\nWriting final CSV to {output_csv_file}...")
        output_dir = os.path.dirname(output_csv_file)
        if output_dir and not os.path.exists(output_dir):
             os.makedirs(output_dir); print(f"  Created output directory: {output_dir}")
        output_df.to_csv(output_csv_file, index=False, na_rep='NA')
        print(f"  Successfully wrote CSV file to {output_csv_file}")
    except Exception as e:
        print(f"Error writing CSV file {output_csv_file}: {e}")
        sys.exit(1)

    print("\nCSV assembly script finished.")

# --- Main execution block ---
if __name__ == "__main__":
    # --- Define your input and output file paths here ---
    parquet_metadata_file = "integrated_asgard_gv_ortho_interpro.parquet"
    pdb_search_results_file = "results_vs_pdb_v2.txt"
    filtered_fasta_path = "folding_candidates_final_filtered.fasta"
    output_database_csv = "proteome_database_v0.1.csv"

    # --- Basic check if input files exist ---
    if not os.path.exists(parquet_metadata_file):
         print(f"Error: Parquet metadata file '{parquet_metadata_file}' not found.")
         sys.exit(1)
    if not os.path.exists(filtered_fasta_path):
         print(f"Error: Filtered FASTA file '{filtered_fasta_path}' not found.")
         sys.exit(1)

    # --- Run the assembly function ---
    assemble_core_csv_with_fasta(
        parquet_file=parquet_metadata_file,
        pdb_results_file=pdb_search_results_file,
        filtered_fasta_file=filtered_fasta_path,
        output_csv_file=output_database_csv
    )


In [ ]:
import pandas as pd
parquet_metadata_file = "integrated_asgard_gv_ortho_interpro.parquet"
try:
    df_meta = pd.read_parquet(parquet_metadata_file, engine='auto', columns=['ProteinID'])
    print("First 5 ProteinIDs from parquet file:")
    print(df_meta['ProteinID'].head().tolist())
except Exception as e:
    print(f"Error reading ProteinID column: {e}")

In [ ]:
import pandas as pd
from Bio import SeqIO
import sys
import os

def assemble_core_csv_with_fasta(
    parquet_file: str,
    pdb_results_file: str,
    filtered_fasta_file: str, # Path to the FASTA with sequences
    output_csv_file: str
) -> None:
    """
    Assembles the initial core data CSV by combining metadata from a parquet file
    with sequence search hit information and sequences from a FASTA file.
    Handles FASTA headers containing '|' delimiters.

    Args:
        parquet_file: Path to the input parquet file containing integrated metadata.
        pdb_results_file: Path to the MMseqs2 results file from the PDB search.
        filtered_fasta_file: Path to the FASTA file containing sequences for filtering.
        output_csv_file: Path where the assembled CSV file will be written.
    """
    print("Starting CSV assembly...")
    print(f"  Input Parquet: {parquet_file}")
    print(f"  PDB Results: {pdb_results_file}")
    print(f"  Input FASTA: {filtered_fasta_file}")
    print(f"  Output CSV: {output_csv_file}")

    # --- 1. Load main metadata from Parquet ---
    try:
        print(f"\nReading metadata from {parquet_file}...")
        df = pd.read_parquet(parquet_file, engine='auto')
        print(f"  Loaded {len(df):,} records from parquet file.")
        print(f"  Parquet Columns Found: {df.columns.tolist()}")

        # --- Column mapping based on user-provided list ---
        required_cols = {
            'ProteinID': 'ProteinID',
            'GenomeID': 'Source_Genome_Assembly_Accession',
            'OriginalName': 'Source_Protein_Annotation',
            'Dataset': 'Source_Dataset',
            'Phylum': 'Taxonomy_Phylum',
            'Taxonomy': 'Taxonomy_Species',
            'OG_ID': 'Orthogroup',
            'All_IPR_Hits': 'IPR_Signatures',
            'All_GO_Terms': 'IPR_GO_Terms',
            'Num_Domains': 'Num_Domains',
            'Domain_Architecture': 'Domain_Architecture',
            'Type': 'Type',
            'Is_Hypothetical': 'Is_Hypothetical',
            'Has_Known_Structure': 'Has_Known_Structure',
            'ncbi_taxid_placeholder': 'NCBI_TaxID',
            'taxonomy_supergroup_placeholder': 'Taxonomy_Supergroup',
            'taxonomy_class_placeholder': 'Taxonomy_Class',
            'uniprot_ac_placeholder': 'UniProtKB_AC',
            'afdb_status_placeholder': 'AFDB_Status'
        }
        protein_id_col_name = 'ProteinID' # Confirmed column name
        if protein_id_col_name not in df.columns:
             print(f"Error: Crucial column for ProteinID ('{protein_id_col_name}') not found in {parquet_file}.")
             sys.exit(1)
        print(f"  Using '{protein_id_col_name}' as the ProteinID column from parquet.")

    except FileNotFoundError:
        print(f"Error: Parquet file not found at {parquet_file}")
        sys.exit(1)
    except Exception as e:
        print(f"Error reading or processing parquet file {parquet_file}: {e}")
        sys.exit(1)

    # --- 2. Load Sequences from FASTA ---
    sequences = {}
    try:
        print(f"\nReading sequences from {filtered_fasta_file}...")
        count_fasta = 0
        count_parse_errors = 0
        with open(filtered_fasta_file, 'r') as fastafile:
             for record in SeqIO.parse(fastafile, "fasta"):
                  # <<< CHANGE START >>>
                  # Parse the ID: Take the part before the first '|' if present,
                  # otherwise use the whole record.id (which handles headers without '|')
                  fasta_id = record.description.split('|')[0] if '|' in record.description else record.id
                  # Basic check if the extracted ID looks reasonable (optional)
                  if not fasta_id:
                      print(f"  Warning: Could not extract valid ID from FASTA header: {record.description[:100]}...")
                      count_parse_errors += 1
                      continue
                  # <<< CHANGE END >>>

                  sequences[fasta_id] = str(record.seq)
                  count_fasta += 1

        print(f"  Loaded {len(sequences):,} sequences from FASTA file.")
        if count_parse_errors > 0:
            print(f"  Encountered {count_parse_errors} headers where ID parsing failed.")
        if count_fasta == 0: print(f"  Warning: No sequences found in {filtered_fasta_file}.")

    except FileNotFoundError:
        print(f"Error: Filtered FASTA file not found at {filtered_fasta_file}")
        sys.exit(1)
    except Exception as e:
        print(f"Error reading FASTA file {filtered_fasta_file}: {e}")
        sys.exit(1)

    # --- 3. Load PDB hit IDs ---
    pdb_hit_ids = set()
    try:
        print(f"\nReading PDB hit IDs from {pdb_results_file}...")
        if os.path.exists(pdb_results_file) and os.path.getsize(pdb_results_file) > 0:
             pdb_hits_df = pd.read_csv(
                  pdb_results_file, sep='\t', header=None, usecols=[0],
                  names=['query'], comment='#', low_memory=False
             )
             # Also parse the query IDs from the PDB results file just in case they have '|'
             pdb_hit_ids = set(pdb_hits_df['query'].dropna().astype(str).apply(lambda x: x.split('|')[0]))
             print(f"  Found {len(pdb_hit_ids):,} unique ProteinIDs (parsed) with PDB hits.")
        else:
             print(f"  PDB results file '{pdb_results_file}' not found or is empty. Assuming no PDB hits.")
    except Exception as e:
        print(f"Error reading PDB results file {pdb_results_file}: {e}")
        pdb_hit_ids = set()

    # --- 4. Prepare DataFrame for Output ---
    print("\nPreparing final DataFrame...")
    unique_protein_ids_in_parquet = df[protein_id_col_name].unique()
    output_df = pd.DataFrame(index=unique_protein_ids_in_parquet)
    output_df.index.name = 'ProteinID_Index' # Temporary index name

    # --- Map and add columns from the parquet file ---
    print("  Mapping columns from parquet to output DataFrame...")
    for parquet_col_name, output_col_name in required_cols.items():
        if parquet_col_name == protein_id_col_name:
            print(f"  Skipping mapping for '{parquet_col_name}' as it's the primary ID.")
            continue

        if parquet_col_name in df.columns:
            try:
                mapping_series = df.drop_duplicates(subset=[protein_id_col_name]).set_index(protein_id_col_name)[parquet_col_name]
                output_df[output_col_name] = output_df.index.map(mapping_series)
            except KeyError as e:
                print(f"!!! Internal KeyError during mapping for column '{parquet_col_name}': {e}")
                output_df[output_col_name] = pd.NA
            except Exception as e:
                print(f"!!! Error during mapping for column '{parquet_col_name}': {e}")
                output_df[output_col_name] = pd.NA
        else:
            print(f"  Info: Column '{parquet_col_name}' not found in input parquet. Adding empty column '{output_col_name}'.")
            output_df[output_col_name] = pd.NA

    # Add Sequence column from the dictionary
    output_df['Sequence'] = output_df.index.map(sequences)
    missing_seq_count = output_df['Sequence'].isna().sum()
    if missing_seq_count > 0:
         print(f"  Info: {missing_seq_count:,} proteins from parquet file did not have a sequence in the filtered FASTA file (expected).")
    print("  Added 'Sequence' column from FASTA.")

    # Calculate Length robustly, handling potential non-string values
    print("  Calculating 'Length' column...")
    output_df['Length'] = output_df['Sequence'].apply(lambda x: len(x) if isinstance(x, str) else 0).astype(int)
    print("  Calculated 'Length' column.")


    # Add Sequence Search Hit Flags
    output_df['SeqSearch_PDB_Hit'] = output_df.index.isin(pdb_hit_ids)
    output_df['SeqSearch_AFDB_Hit'] = False # Based on previous results
    output_df['SeqSearch_MGnify_Hit'] = False # Search was skipped
    print("  Added 'SeqSearch_*_Hit' flag columns.")

    # Reset index to make ProteinID a regular column again
    output_df.reset_index(inplace=True)
    output_df.rename(columns={'ProteinID_Index': 'ProteinID'}, inplace=True) # Rename index col

    # --- Filter rows: Keep only proteins present in the filtered FASTA ---
    initial_rows = len(output_df)
    # Ensure comparison is robust by checking against the keys from the sequences dict
    output_df = output_df[output_df['ProteinID'].isin(sequences.keys())].copy()
    rows_after_fasta_filter = len(output_df)
    print(f"  Filtered DataFrame to keep only proteins present in '{filtered_fasta_file}'.")
    print(f"  Rows before FASTA filter: {initial_rows:,}. Rows after: {rows_after_fasta_filter:,}")

    # --- Reorder columns ---
    desired_order = [
        'ProteinID', 'Sequence', 'Length', 'Source_Dataset', 'Dataset',
        'Source_Genome_Assembly_Accession', 'GenomeID',
        'Source_Protein_Annotation', 'OriginalName',
        'NCBI_TaxID', 'Taxonomy_Supergroup', 'Taxonomy_Phylum', 'Phylum',
        'Taxonomy_Class', 'Taxonomy_Species', 'Taxonomy',
        'Orthogroup', 'OG_ID',
        'IPR_Signatures', 'All_IPR_Hits',
        'IPR_GO_Terms', 'All_GO_Terms',
        'UniProtKB_AC', 'AFDB_Status',
        'SeqSearch_PDB_Hit', 'SeqSearch_AFDB_Hit', 'SeqSearch_MGnify_Hit',
        'Num_Domains', 'Domain_Architecture', 'Type', 'Is_Hypothetical', 'Has_Known_Structure'
    ]
    present_columns = [col for col in desired_order if col in output_df.columns]
    present_columns.extend([col for col in output_df.columns if col not in present_columns])
    final_columns = pd.Series(present_columns).drop_duplicates().tolist() # Ensure unique cols
    output_df = output_df[final_columns]
    print(f"  Final DataFrame has {len(output_df.columns)} columns and {len(output_df):,} rows.")

    # --- 5. Write to CSV ---
    try:
        print(f"\nWriting final CSV to {output_csv_file}...")
        output_dir = os.path.dirname(output_csv_file)
        if output_dir and not os.path.exists(output_dir):
             os.makedirs(output_dir); print(f"  Created output directory: {output_dir}")

        # Check if DataFrame is empty before writing
        if output_df.empty:
            print("  Warning: Final DataFrame is empty. Writing only headers to CSV.")
            # Write only headers if empty
            with open(output_csv_file, 'w') as f:
                f.write(','.join(output_df.columns) + '\n')
        else:
            output_df.to_csv(output_csv_file, index=False, na_rep='NA') # Use NA for missing values
            print(f"  Successfully wrote CSV file to {output_csv_file}")

    except Exception as e:
        print(f"Error writing CSV file {output_csv_file}: {e}")
        sys.exit(1)

    print("\nCSV assembly script finished.")

# --- Main execution block ---
if __name__ == "__main__":
    # --- Define your input and output file paths here ---
    parquet_metadata_file = "integrated_asgard_gv_ortho_interpro.parquet"
    pdb_search_results_file = "results_vs_pdb_v2.txt"
    filtered_fasta_path = "folding_candidates_final_filtered.fasta"
    output_database_csv = "proteome_database_v0.1.csv"

    # --- Basic check if input files exist ---
    if not os.path.exists(parquet_metadata_file):
         print(f"Error: Parquet metadata file '{parquet_metadata_file}' not found.")
         sys.exit(1)
    if not os.path.exists(filtered_fasta_path):
         print(f"Error: Filtered FASTA file '{filtered_fasta_path}' not found.")
         sys.exit(1)

    # --- Run the assembly function ---
    assemble_core_csv_with_fasta(
        parquet_file=parquet_metadata_file,
        pdb_results_file=pdb_search_results_file,
        filtered_fasta_file=filtered_fasta_path,
        output_csv_file=output_database_csv
    )


In [ ]:
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
import os
import sys # Added for better error handling exit

# --- Configuration: PLEASE UPDATE THESE PATHS ---
unknowns_csv_path = 'your_unknowns_only.csv'  # Path to the CSV file generated by --output-unknowns
interpro_list_path = 'interpro_entry.list' # Path to your InterPro entry list TSV file
output_dir = '.' # Directory to save any output files/plots (optional)

# --- Parameters ---
top_n_ipr = 30  # How many top IPR IDs to display
top_n_keywords = 50 # How many top annotation keywords to display

# --- Ensure output directory exists ---
try:
    os.makedirs(output_dir, exist_ok=True)
    print(f"Output directory: {os.path.abspath(output_dir)}")
except Exception as e:
    print(f"Warning: Could not create output directory '{output_dir}': {e}")
    output_dir = '.' # Default to current directory if creation fails

print(f"Unknowns CSV: {unknowns_csv_path}")
print(f"InterPro List: {interpro_list_path}")

# ==============================================================================
# Section 1: Load Data
# ==============================================================================
print("\n--- Section 1: Loading Data ---")

# --- Load Unknowns CSV ---
try:
    df_unknowns = pd.read_csv(unknowns_csv_path, low_memory=False)
    print(f"Successfully loaded {len(df_unknowns)} rows from {unknowns_csv_path}")
    # Fill NaN values in key columns to avoid errors later
    df_unknowns['IPR_Signatures'] = df_unknowns['IPR_Signatures'].fillna('')
    df_unknowns['Source_Protein_Annotation'] = df_unknowns['Source_Protein_Annotation'].fillna('')
    # Display first few rows and info (optional, uncomment if needed)
    # print("\nUnknowns DataFrame Head:")
    # print(df_unknowns.head())
    # print("\nUnknowns DataFrame Info:")
    # df_unknowns.info()
except FileNotFoundError:
    print(f"ERROR: Unknowns CSV file not found at {unknowns_csv_path}. Please check the path.")
    sys.exit(1) # Stop execution if file not found
except Exception as e:
    print(f"ERROR: Could not load unknowns CSV: {e}")
    sys.exit(1)

# --- Load InterPro List TSV ---
def load_interpro_list_script(filepath):
    """Loads InterPro entry list into a dictionary for script use."""
    print(f"Attempting to load InterPro list from: {filepath}")
    ipr_map = {}
    abs_filepath = os.path.abspath(filepath)

    if not os.path.exists(abs_filepath) or not os.path.isfile(abs_filepath):
        print(f"ERROR: InterPro list file not found or is not a file: {abs_filepath}")
        return None

    try:
        # Use pandas for robust TSV reading
        df_ipr = pd.read_csv(abs_filepath, sep='\t', header=0,
                             names=['ENTRY_AC', 'ENTRY_TYPE', 'ENTRY_NAME'], # Assign names in case header is missing
                             usecols=[0, 1, 2], # Only read first 3 columns
                             on_bad_lines='warn', # Report problematic lines
                             encoding='utf-8', errors='ignore')

        # Check if first row was mistakenly read as data instead of header
        if not df_ipr.empty and df_ipr.iloc[0]['ENTRY_AC'] == 'ENTRY_AC':
             df_ipr = df_ipr.iloc[1:]

        # Filter for valid IPR IDs and convert to dictionary
        df_ipr = df_ipr[df_ipr['ENTRY_AC'].astype(str).str.startswith('IPR')]
        ipr_map = df_ipr.set_index('ENTRY_AC').to_dict('index')

        print(f"Successfully processed {len(ipr_map)} InterPro entries from {abs_filepath}.")
        if len(ipr_map) < 10: # Check if map seems very small
             print("WARN: Very few InterPro entries loaded. Check file format and content.")

    except Exception as e:
        print(f"ERROR: Failed to load or parse InterPro list file {abs_filepath}: {e}")
        return None

    return ipr_map

ipr_details_map = load_interpro_list_script(interpro_list_path)

# Example lookup (if loaded successfully)
if ipr_details_map and len(ipr_details_map) > 0:
    example_id = list(ipr_details_map.keys())[0]
    print(f"Example InterPro entry: {example_id} -> {ipr_details_map.get(example_id)}")
elif not ipr_details_map:
     print("WARN: InterPro map could not be loaded. Analysis requiring IPR details will be skipped.")

# ==============================================================================
# Section 2: Count Proteins Lacking IPR Signatures
# ==============================================================================
print("\n--- Section 2: Counting Proteins Lacking IPR Signatures ---")

total_unknowns = len(df_unknowns)
# Count rows where IPR_Signatures is null, empty string, or whitespace only
unknowns_no_ipr = df_unknowns[df_unknowns['IPR_Signatures'].astype(str).str.strip() == ''].copy()
count_no_ipr = len(unknowns_no_ipr)

print(f"Total 'Unknown/Unclassified' proteins: {total_unknowns}")
if total_unknowns > 0:
    percentage_no_ipr = (count_no_ipr / total_unknowns) * 100
    print(f"Number of unknowns with NO IPR signatures: {count_no_ipr} ({percentage_no_ipr:.2f}%)")
else:
    print("Number of unknowns with NO IPR signatures: 0")

# Display some examples of proteins without IPR signatures (optional)
# if count_no_ipr > 0:
#     print("\nExample rows with no IPR Signatures (Annotation only):")
#     print(unknowns_no_ipr[['Source_Protein_Annotation']].head())

print("\nInsight: If this percentage is high, improving classification might heavily depend on")
print("analyzing and expanding the 'Annotation_Keywords' in the main script's CUSTOM_RULES.")

# ==============================================================================
# Section 3: Find Most Frequent IPR IDs Among Unknowns
# ==============================================================================
print("\n--- Section 3: Finding Most Frequent IPR IDs Among Unknowns ---")

# --- Parse IPR Signatures and Count Frequencies ---
all_ipr_ids_unknowns = []

def extract_iprs(ipr_string):
    # Split by common delimiters, strip whitespace, filter out empty strings
    ids = [ipr_id.strip() for ipr_id in re.split(r'[,;|]', str(ipr_string)) if ipr_id.strip()]
    return ids

# Apply the function to the 'IPR_Signatures' column and flatten the list
# Ensure the column exists before applying
if 'IPR_Signatures' in df_unknowns.columns:
    all_ipr_ids_unknowns = df_unknowns['IPR_Signatures'].apply(extract_iprs).sum()
    print(f"Total IPR signature occurrences found in unknowns: {len(all_ipr_ids_unknowns)}")
else:
    print("WARN: 'IPR_Signatures' column not found in unknowns CSV. Skipping IPR frequency analysis.")
    all_ipr_ids_unknowns = [] # Ensure list exists but is empty

# Count the frequency of each IPR ID
ipr_counts = Counter(all_ipr_ids_unknowns)

# Get the most common IPR IDs
most_common_ipr = ipr_counts.most_common(top_n_ipr)

print(f"\nTop {top_n_ipr} most frequent IPR IDs among unclassified proteins:")
if not most_common_ipr:
    print("No IPR IDs found or processed in the unknowns file.")
else:
    # Create a DataFrame for better display
    df_top_ipr = pd.DataFrame(most_common_ipr, columns=['IPR_ID', 'Frequency'])
    print(df_top_ipr.to_string()) # Print full dataframe

    # --- Basic Plot ---
    try:
        plt.figure(figsize=(10, 8))
        plt.barh(df_top_ipr['IPR_ID'], df_top_ipr['Frequency'])
        plt.xlabel('Frequency')
        plt.ylabel('IPR ID')
        plt.title(f'Top {top_n_ipr} Most Frequent IPR IDs in Unknowns')
        plt.gca().invert_yaxis() # Display top ID at the top
        plt.tight_layout()
        # Save the plot (optional)
        plot_path = os.path.join(output_dir, 'top_ipr_ids_unknowns.png')
        plt.savefig(plot_path)
        print(f"\nPlot saved to {plot_path}")
        # plt.show() # Uncomment to display plot if running interactively
        plt.close() # Close plot to free memory
    except Exception as e:
        print(f"\nWARN: Could not generate IPR frequency plot: {e}")


print("\nInsight: These IPR IDs are the most common signatures NOT being caught by your current rules.")
print("They are the primary candidates to investigate further.")

# ==============================================================================
# Section 4: Analyze Details of Frequent Unknown IPR IDs
# ==============================================================================
print("\n--- Section 4: Analyzing Details of Frequent Unknown IPR IDs ---")

if not ipr_details_map:
    print("Skipping IPR detail analysis because the InterPro map was not loaded.")
elif not most_common_ipr:
    print("Skipping IPR detail analysis because no frequent IPR IDs were found.")
else:
    print(f"Looking up details for the top {top_n_ipr} IPR IDs...")
    top_ipr_data = []
    for ipr_id, frequency in most_common_ipr:
        details = ipr_details_map.get(ipr_id, {'type': 'Not Found', 'name': 'Not Found'})
        top_ipr_data.append({
            'IPR_ID': ipr_id,
            'Frequency': frequency,
            'IPR_Type': details.get('type', 'N/A'),
            'IPR_Name': details.get('name', 'N/A')
        })

    df_top_ipr_details = pd.DataFrame(top_ipr_data)

    print("\nDetails of Top Frequent IPR IDs in Unknowns:")
    # Display the full table
    with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
        print(df_top_ipr_details.to_string())

print("\nInsight:")
print("- Look at the IPR_Name: Does it suggest a function? Does it contain keywords you could add")
print("  to an existing category's 'IPR_Keywords' list (remember to use lowercase)?")
print("- Does the function represent a new category you need to create in CUSTOM_RULES?")
print("- Look at the IPR_Type: Is it 'Domain', 'Family', etc.? This helps understand the nature")
print("  of the unclassified signatures.")

# ==============================================================================
# Section 5: Analyze Annotation Keywords
# ==============================================================================
print("\n--- Section 5: Analyzing Annotation Keywords ---")

if 'Source_Protein_Annotation' not in df_unknowns.columns:
     print("WARN: 'Source_Protein_Annotation' column not found. Skipping annotation keyword analysis.")
else:
    print("Analyzing keywords in 'Source_Protein_Annotation'...")

    # Define common words to ignore (customize this list as needed!)
    stop_words = {
        'protein', 'hypothetical', 'uncharacterized', 'predicted', 'putative',
        'domain', 'family', 'containing', 'like', 'of', 'the', 'a', 'an', 'in', 'to', 'and',
        'is', 'it', 'with', 'by', 'on', 'at', 'from', 'as', 'for', 'or', 'et', 'al',
        'type', 'subunit', 'chain', 'region', 'motif', 'repeat', 'protein,', 'unknown', 'function'
        # Add more domain-specific or common words if they obscure results
    }

    all_words = []
    # Ensure the column is treated as string and lowercase
    annotation_series = df_unknowns['Source_Protein_Annotation'].astype(str).str.lower()

    # Simple word tokenization and filtering
    for annotation in annotation_series:
        # Remove punctuation (basic), split into words
        words = re.findall(r'\b\w+\b', annotation)
        # Filter out stop words and very short words (e.g., <= 2 letters)
        filtered_words = [word for word in words if word not in stop_words and len(word) > 2]
        all_words.extend(filtered_words)

    print(f"Total potentially relevant words found in annotations: {len(all_words)}")

    # Count word frequencies
    keyword_counts = Counter(all_words)
    most_common_keywords = keyword_counts.most_common(top_n_keywords)

    print(f"\nTop {top_n_keywords} most frequent keywords in annotations (excluding common words):")
    if not most_common_keywords:
        print("No significant keywords found after filtering.")
    else:
        df_top_keywords = pd.DataFrame(most_common_keywords, columns=['Keyword', 'Frequency'])
        print(df_top_keywords.to_string())

        # --- Basic Plot ---
        try:
            plt.figure(figsize=(10, 10))
            plt.barh(df_top_keywords['Keyword'], df_top_keywords['Frequency'])
            plt.xlabel('Frequency')
            plt.ylabel('Keyword')
            plt.title(f'Top {top_n_keywords} Annotation Keywords in Unknowns')
            plt.gca().invert_yaxis()
            plt.tight_layout()
            # Save the plot (optional)
            plot_path = os.path.join(output_dir, 'top_annotation_keywords_unknowns.png')
            plt.savefig(plot_path)
            print(f"\nPlot saved to {plot_path}")
            # plt.show() # Uncomment to display plot if running interactively
            plt.close() # Close plot to free memory
        except Exception as e:
             print(f"\nWARN: Could not generate annotation keyword plot: {e}")


    print("\nInsight:")
    print("- Are there recurring functional terms here that are missing from your")
    print("  'Annotation_Keywords' lists in CUSTOM_RULES (remember lowercase)?")
    print("- Do these keywords suggest functions that belong to existing categories or point")
    print("  towards new categories needed?")

# ==============================================================================
# Section 6: Conclusion
# ==============================================================================
print("\n--- Section 6: Conclusion & Next Steps ---")
print("\nThis analysis provides several starting points for refining the `CUSTOM_RULES`")
print("in your `add_specific_category_IPR_v10.py` script:")
print("\n1. Proteins without IPR: If many unknowns lack IPR IDs, focus on improving")
print("   the `Annotation_Keywords` rules.")
print("2. Frequent Unknown IPR IDs: Investigate the functions of the top IPR IDs")
print("   identified. Add relevant IDs or derived keywords (`IPR_Keywords`) to your")
print("   existing or new categories in `CUSTOM_RULES`.")
print("3. Frequent Annotation Keywords: Add relevant keywords found in the annotation")
print("   analysis to the `Annotation_Keywords` lists in `CUSTOM_RULES`.")
print("\nRecommendation:")
print("* Iteratively update the `CUSTOM_RULES` dictionary in `add_specific_category_IPR_v10.py`")
print("  based on these findings.")
print("* Re-run the main script.")
print("* Re-run this analysis script on the *new* unknowns file to see if the")
print("  classification has improved and identify the next set of common unknowns.")
print("* Repeat this process until the number of 'Unknown/Unclassified' proteins")
print("  is acceptably low.")

print("\nAnalysis complete.")


In [ ]:
import pandas as pd
from Bio import SeqIO
import sys
import os

# --- Configuration ---
unknowns_csv_path = 'your_unknowns_only.csv' # Input CSV with unknown protein IDs
original_fasta_files = [
    'Fastas_filtered/Asgard_all_globular_proteins.fasta', # Path to first original FASTA
    'Fastas_filtered/GV_all_globular_proteins.fasta'     # Path to second original FASTA
    # Add more original FASTA files if needed
]
output_fasta_path = 'unknown_protein_sequences.fasta' # Output FASTA for InterProScan input

# Column in the CSV containing the protein IDs (adjust if different)
protein_id_column = 'ProteinID'

# --- Script Logic ---

print(f"Loading unknown protein IDs from: {unknowns_csv_path}")

try:
    df_unknowns = pd.read_csv(unknowns_csv_path)
    if protein_id_column not in df_unknowns.columns:
        print(f"ERROR: Protein ID column '{protein_id_column}' not found in {unknowns_csv_path}")
        print(f"Available columns: {df_unknowns.columns.tolist()}")
        sys.exit(1)
    # Ensure IDs are strings and handle potential NaN values
    unknown_ids = set(df_unknowns[protein_id_column].dropna().astype(str))
    print(f"Found {len(unknown_ids)} unique unknown protein IDs.")
    if not unknown_ids:
        print("ERROR: No protein IDs found in the unknowns file. Exiting.")
        sys.exit(1)

except FileNotFoundError:
    print(f"ERROR: Unknowns CSV file not found at {unknowns_csv_path}")
    sys.exit(1)
except Exception as e:
    print(f"ERROR: Failed to read or process {unknowns_csv_path}: {e}")
    sys.exit(1)


print(f"\nExtracting sequences from:")
for f in original_fasta_files:
    print(f"- {f}")

sequences_found = 0
sequences_written = 0
ids_found = set()

try:
    with open(output_fasta_path, 'w') as outfile:
        for fasta_file in original_fasta_files:
            if not os.path.exists(fasta_file):
                print(f"WARNING: Original FASTA file not found, skipping: {fasta_file}")
                continue

            print(f"Processing {fasta_file}...")
            try:
                for record in SeqIO.parse(fasta_file, "fasta"):
                    # --- MODIFIED LINE ---
                    # Extract the ID part before the first pipe '|'
                    # This assumes your CSV ID matches this part, e.g., 'RLI68853.1'
                    current_id = record.id.split('|')[0]
                    # --- END MODIFIED LINE ---

                    if current_id in unknown_ids:
                        sequences_found += 1
                        ids_found.add(current_id)
                        SeqIO.write(record, outfile, "fasta")
                        sequences_written += 1
            except Exception as e:
                print(f"ERROR: Failed to parse {fasta_file}: {e}")
                # Decide if you want to continue or exit on parse error
                # continue

except IOError as e:
     print(f"ERROR: Could not write to output file {output_fasta_path}: {e}")
     sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred during sequence extraction: {e}")
    sys.exit(1)


print(f"\nExtraction complete.")
print(f"Total sequences processed where ID matched an unknown ID: {sequences_found}")
print(f"Total unique unknown IDs found in FASTA files: {len(ids_found)}")
print(f"Total sequences written to {output_fasta_path}: {sequences_written}")

# --- Report missing IDs ---
missing_ids = unknown_ids - ids_found
if missing_ids:
    print(f"\nWARNING: {len(missing_ids)} unknown IDs were not found in the provided FASTA files.")
    # Optionally print the first few missing IDs
    # print("Example missing IDs:", list(missing_ids)[:10])
    # Consider writing missing IDs to a file
    missing_ids_file = "missing_unknown_ids.txt"
    try:
        with open(missing_ids_file, "w") as f_missing:
            for mid in sorted(list(missing_ids)):
                f_missing.write(mid + "\n")
        print(f"Full list of missing IDs written to {missing_ids_file}")
    except Exception as e:
        print(f"ERROR: Could not write missing IDs file: {e}")

else:
    print("\nAll unknown IDs were found in the provided FASTA files.")



In [ ]:
import pandas as pd
import sys
import os
import numpy as np # For checking NaN

# --- Configuration: PLEASE UPDATE THESE PATHS ---

# Path to the main database CSV file (output from the last classification run)
main_database_csv = 'proteome_database_v0.3.csv'

# Path to the NEW InterProScan TSV output file (from the run on unknowns)
# Adjust the filename if InterProScan generated a different one.
new_ipr_tsv = 'InterProScan_Results/unknown_protein_sequences.fasta.tsv'

# Path for the updated output CSV file
updated_database_csv = 'proteome_database_v0.4.csv'

# --- Column Names ---
# Adjust these if your main CSV uses different names
protein_id_col_main = 'ProteinID'
ipr_col_main = 'IPR_Signatures'

# --- Script Logic ---

print("--- Starting Integration of New InterProScan Results ---")

# --- Step 1: Parse the new InterProScan TSV output ---
print(f"Reading new InterProScan results from: {new_ipr_tsv}")

if not os.path.exists(new_ipr_tsv):
    print(f"ERROR: New InterProScan TSV file not found: {new_ipr_tsv}")
    sys.exit(1)

try:
    # Define column names based on standard InterProScan TSV format
    # We only need Protein ID (col 0) and IPR ID (col 11)
    col_names = [
        'Protein_ID_raw', 'MD5', 'Length', 'Analysis', 'Sig_Acc', 'Sig_Desc',
        'Start', 'Stop', 'Score', 'Status', 'Date', 'IPR_ID', 'IPR_Desc',
        'GO', 'Pathway'
    ]
    # Read only necessary columns, specify separator and no header
    df_new_ipr = pd.read_csv(
        new_ipr_tsv,
        sep='\t',
        header=None,
        names=col_names,
        usecols=['Protein_ID_raw', 'IPR_ID'], # Read raw ID first
        dtype={'Protein_ID_raw': str, 'IPR_ID': str} # Read as string initially
    )
    print(f"Read {len(df_new_ipr)} lines from TSV.")

    # --- *** ID PARSING FIX *** ---
    # Extract the part before the first pipe '|' from the raw protein ID column
    print("Parsing Protein IDs from TSV (removing extra info)...")
    df_new_ipr['Protein_ID'] = df_new_ipr['Protein_ID_raw'].str.split('|').str[0]
    # --- *** END ID PARSING FIX *** ---


    # Filter out rows without an IPR ID (often represented as '-')
    df_new_ipr.dropna(subset=['IPR_ID'], inplace=True)
    df_new_ipr = df_new_ipr[df_new_ipr['IPR_ID'] != '-']
    print(f"Found {len(df_new_ipr)} annotations with IPR IDs.")

    # --- Step 2: Aggregate IPR IDs per Protein ---
    # Group by the *parsed* Protein ID and join unique IPR IDs with a semicolon
    print("Aggregating new IPR IDs per protein...")
    # Ensure IDs are unique before joining
    new_ipr_map_series = df_new_ipr.groupby('Protein_ID')['IPR_ID'].apply(lambda x: ';'.join(sorted(x.unique())))

    if new_ipr_map_series.empty:
         print("WARNING: No valid IPR signatures found in the new TSV file after processing.")
         # Decide whether to exit or continue (continuing will just rewrite the main CSV)
         # sys.exit(1) # Uncomment to exit if no new IPRs found
    else:
        print(f"Aggregated new IPR signatures for {len(new_ipr_map_series)} unique proteins.")
        # Display some examples (optional)
        # print("\nExample aggregated signatures:")
        # print(new_ipr_map_series.head())

except FileNotFoundError:
    print(f"ERROR: New InterProScan TSV file not found at {new_ipr_tsv}")
    sys.exit(1)
except pd.errors.EmptyDataError:
    print(f"ERROR: New InterProScan TSV file is empty: {new_ipr_tsv}")
    sys.exit(1)
except Exception as e:
    print(f"ERROR: Failed to read or process {new_ipr_tsv}: {e}")
    sys.exit(1)


# --- Step 3: Load the main database CSV ---
print(f"\nReading main database CSV: {main_database_csv}")

if not os.path.exists(main_database_csv):
    print(f"ERROR: Main database CSV file not found: {main_database_csv}")
    sys.exit(1)

try:
    df_main = pd.read_csv(main_database_csv, low_memory=False)
    print(f"Read {len(df_main)} rows from main database.")

    # --- Validate required columns exist ---
    if protein_id_col_main not in df_main.columns:
        print(f"ERROR: Protein ID column '{protein_id_col_main}' not found in {main_database_csv}")
        sys.exit(1)
    if ipr_col_main not in df_main.columns:
        print(f"ERROR: IPR Signatures column '{ipr_col_main}' not found in {main_database_csv}")
        # If it doesn't exist, create it
        print(f"Creating column '{ipr_col_main}'...")
        df_main[ipr_col_main] = np.nan # Initialize with NaN

except FileNotFoundError:
    print(f"ERROR: Main database CSV file not found at {main_database_csv}")
    sys.exit(1)
except Exception as e:
    print(f"ERROR: Failed to read main database CSV {main_database_csv}: {e}")
    sys.exit(1)


# --- Step 4: Merge/Update IPR Signatures ---
print("\nUpdating IPR signatures in the main database...")

# Ensure the IPR column is treated as string for checking emptiness
# Use .fillna('') before checking to handle actual NaN values correctly
df_main[ipr_col_main] = df_main[ipr_col_main].fillna('').astype(str)

# Create a boolean mask for rows where IPR_Signatures is currently empty or effectively 'nan'
is_empty_mask = (df_main[ipr_col_main].str.strip() == '') | (df_main[ipr_col_main].str.strip().str.lower() == 'nan')

# Get the Protein IDs for the rows that need updating (using the mask)
ids_to_update = df_main.loc[is_empty_mask, protein_id_col_main]

# Map the new signatures onto these IDs
# Use .get() on the series map to handle IDs that might be in the main DB but not have new IPRs
# signatures_to_apply = ids_to_update.map(lambda pid: new_ipr_map_series.get(pid)) # Not directly needed for update

# Update the main DataFrame only where the original was empty AND new data exists
# Create a combined mask: original was empty AND new signature was found
update_mask = is_empty_mask & df_main[protein_id_col_main].isin(new_ipr_map_series.index)

# Get the number of rows that will actually be updated
rows_to_be_updated_count = update_mask.sum()
print(f"Found {rows_to_be_updated_count} rows with previously empty IPR signatures that have new results.")

# Apply the update using .loc
# We map directly from the series using the index alignment provided by Protein_ID
# Ensure that only rows matching the update_mask are targeted
if rows_to_be_updated_count > 0:
    df_main.loc[update_mask, ipr_col_main] = df_main.loc[update_mask, protein_id_col_main].map(new_ipr_map_series)
    print("IPR signature update applied.")
else:
    print("No rows needed updating.")


# --- Step 5: Save the updated database ---
print(f"\nSaving updated database to: {updated_database_csv}")

try:
    df_main.to_csv(updated_database_csv, index=False)
    print(f"Successfully wrote {len(df_main)} rows to {updated_database_csv}.")
except Exception as e:
    print(f"ERROR: Failed to write updated database CSV: {e}")
    sys.exit(1)

print("\n--- Integration Finished ---")
print(f"\nNext Step: Re-run the classification script (add_specific_category_IPR_v10.py) using '{updated_database_csv}' as the input CSV.")

In [ ]:
# %% [markdown]
# # Integrate USPNet Signal Peptide Predictions
#
# This cell reads the USPNet prediction results, aligns them with the main protein database using the original input FASTA order, and merges the predictions into the main dataframe.

# %% [code]
import pandas as pd
from Bio import SeqIO
import sys
import os
import numpy as np

# --- Configuration: File Paths ---
# Path to the main database CSV file (output from the last step, e.g., taxonomy refinement)
main_csv_path = 'proteome_database_v0.6.csv'

# Path to the USPNet output CSV file (generated by predict_fast.py)
uspnet_results_path = 'USPNet_Intermediate/results.csv' # Default location based on run_uspnet.sh

# Path to the FASTA file originally used as input for USPNet
input_fasta_path = 'all_proteins_for_dtm.fasta'

# Path for the updated output CSV file
output_csv_path = 'proteome_database_v0.7.csv'

# --- Configuration: Column Names ---
# In main input CSV
PROTEIN_ID_COL_MAIN = 'ProteinID'
# In USPNet output CSV
USP_SEQUENCE_COL = 'sequence' # We won't use this directly for merging
USP_PRED_TYPE_COL = 'predicted_type'
USP_PRED_CLEAVAGE_COL = 'predicted_cleavage'
# New columns to create in the output CSV
NEW_SP_TYPE_COL = 'Signal_Peptide_USPNet'
NEW_SP_CLEAVAGE_COL = 'SP_Cleavage_Site_USPNet'

# --- Script Logic ---

print("--- Starting Integration of USPNet Results ---")
print(f"Main database CSV: {main_csv_path}")
print(f"USPNet results CSV: {uspnet_results_path}")
print(f"Original FASTA input for USPNet: {input_fasta_path}")
print(f"Output database CSV: {output_csv_path}")

# --- Validate input files ---
if not os.path.exists(main_csv_path):
    print(f"ERROR: Main database CSV not found: {main_csv_path}")
    # sys.exit(1) # Use raise instead in a notebook or handle differently
    raise FileNotFoundError(f"Main database CSV not found: {main_csv_path}")
if not os.path.exists(uspnet_results_path):
    print(f"ERROR: USPNet results CSV not found: {uspnet_results_path}")
    raise FileNotFoundError(f"USPNet results CSV not found: {uspnet_results_path}")
if not os.path.exists(input_fasta_path):
    print(f"ERROR: Input FASTA file not found: {input_fasta_path}")
    raise FileNotFoundError(f"Input FASTA file not found: {input_fasta_path}")

# --- Step 1: Read Protein IDs from FASTA in order ---
print(f"\nReading Protein IDs from FASTA: {input_fasta_path}...")
protein_ids_in_order = []
try:
    for record in SeqIO.parse(input_fasta_path, "fasta"):
        # Assuming the ID used by USPNet corresponds to record.id
        protein_ids_in_order.append(record.id)
    print(f"Read {len(protein_ids_in_order)} IDs from FASTA.")
    if not protein_ids_in_order:
        print("ERROR: No sequences found in the FASTA file.")
        raise ValueError("No sequences found in FASTA file.")
except Exception as e:
    print(f"ERROR: Failed to read or parse FASTA file {input_fasta_path}: {e}")
    raise # Re-raise the exception

# --- Step 2: Read USPNet results ---
print(f"Reading USPNet results: {uspnet_results_path}...")
try:
    df_uspnet = pd.read_csv(uspnet_results_path)
    print(f"Read {len(df_uspnet)} rows from USPNet results.")
except Exception as e:
    print(f"ERROR: Failed to read USPNet results CSV {uspnet_results_path}: {e}")
    raise # Re-raise the exception

# --- Step 3: Check length consistency and add ProteinID ---
if len(protein_ids_in_order) != len(df_uspnet):
    print(f"ERROR: Mismatch in number of sequences!")
    print(f"  FASTA file '{input_fasta_path}' has {len(protein_ids_in_order)} sequences.")
    print(f"  USPNet results '{uspnet_results_path}' has {len(df_uspnet)} rows.")
    print("Cannot reliably merge results. Please check the inputs.")
    raise ValueError("Mismatch between FASTA sequence count and USPNet result count.")

print("Adding ProteinID column to USPNet results based on FASTA order...")
df_uspnet[PROTEIN_ID_COL_MAIN] = protein_ids_in_order
# Select and rename columns for merging
df_uspnet_to_merge = df_uspnet[[PROTEIN_ID_COL_MAIN, USP_PRED_TYPE_COL, USP_PRED_CLEAVAGE_COL]].copy()
df_uspnet_to_merge.rename(columns={
    USP_PRED_TYPE_COL: NEW_SP_TYPE_COL,
    USP_PRED_CLEAVAGE_COL: NEW_SP_CLEAVAGE_COL
}, inplace=True)

# --- Step 4: Load main database ---
print(f"\nLoading main database: {main_csv_path}...")
try:
    df_main = pd.read_csv(main_csv_path, low_memory=False)
    print(f"Loaded {len(df_main)} rows.")
except Exception as e:
    print(f"ERROR: Failed to load main database CSV {main_csv_path}: {e}")
    raise # Re-raise the exception

# --- Step 5: Merge USPNet results ---
print(f"Merging USPNet results into main database using '{PROTEIN_ID_COL_MAIN}'...")

# Check if merge columns already exist and remove them first if necessary
if NEW_SP_TYPE_COL in df_main.columns:
    print(f"Warning: Column '{NEW_SP_TYPE_COL}' already exists. It will be overwritten.")
    df_main.drop(columns=[NEW_SP_TYPE_COL], inplace=True)
if NEW_SP_CLEAVAGE_COL in df_main.columns:
    print(f"Warning: Column '{NEW_SP_CLEAVAGE_COL}' already exists. It will be overwritten.")
    df_main.drop(columns=[NEW_SP_CLEAVAGE_COL], inplace=True)

# Perform left merge to keep all rows from the main database
df_merged = pd.merge(
    df_main,
    df_uspnet_to_merge,
    on=PROTEIN_ID_COL_MAIN,
    how='left' # Keep all rows from df_main
)

# Check if merge resulted in the same number of rows
if len(df_merged) != len(df_main):
    print("ERROR: Merge resulted in an unexpected number of rows!")
    print(f" Original rows: {len(df_main)}, Merged rows: {len(df_merged)}")
    raise ValueError("Merge changed the number of rows in the dataframe.")

# Report how many rows got USPNet data
populated_uspnet = df_merged[NEW_SP_TYPE_COL].notna().sum()
print(f"Successfully merged USPNet data for {populated_uspnet} proteins.")
if populated_uspnet < len(df_uspnet_to_merge):
     print(f"Warning: Some proteins with USPNet results ({len(df_uspnet_to_merge) - populated_uspnet}) were not found in the main database '{main_csv_path}'.")

# --- Step 6: Save Updated CSV ---
print(f"\nSaving updated database to: {output_csv_path}")
try:
    # Ensure new columns are strings, fill potential merge NaNs with empty string
    df_merged[NEW_SP_TYPE_COL] = df_merged[NEW_SP_TYPE_COL].fillna('').astype(str)
    df_merged[NEW_SP_CLEAVAGE_COL] = df_merged[NEW_SP_CLEAVAGE_COL].fillna('').astype(str)

    df_merged.to_csv(output_csv_path, index=False)
    print(f"Successfully wrote {len(df_merged)} rows to {output_csv_path}.")
except Exception as e:
    print(f"ERROR: Failed to write updated CSV: {e}")
    raise # Re-raise the exception

print("\n--- USPNet Integration Finished ---")
print(f"\nNext Step: Integrate DeepTMHMM results into '{output_csv_path}'.")

# Display first few rows of the updated dataframe (optional, common in notebooks)
print("\nPreview of updated data:")
display(df_merged.head())


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import warnings

# --- Configuration ---
# Set the path to your CSV file
csv_file_path = 'proteome_database_v0.7.csv'

# Define columns used
# Adjust these if your column names are different
protein_id_col = 'ProteinID' # Assuming this is your protein identifier column
sequence_col = 'Sequence'
virus_family_col = 'Virus_Name'
archaea_phylum_col = 'Asgard_Phylum' # Or use 'Source_Dataset' if more reliable
uspnet_col = 'Signal_Peptide_USPNet'
cleavage_site_col = 'SP_Cleavage_Site_USPNet' # Column with the sequence UP TO the cleavage site

# Define output columns
localization_output_col = 'Predicted_Subcellular_Localization'
mature_seq_output_col = 'Mature_Protein_Sequence'

# --- Load Data ---
try:
    df = pd.read_csv(csv_file_path)
    print(f"Successfully loaded '{csv_file_path}'. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: File not found at '{csv_file_path}'. Please ensure the path is correct.")
    # Stop execution if file not found
    raise
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")
    raise

# --- Data Validation ---
# Check for essential columns for the entire process
required_cols = [
    protein_id_col, sequence_col, virus_family_col, archaea_phylum_col,
    uspnet_col, cleavage_site_col
]
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"Error: The following required columns are missing from the CSV: {missing_cols}")
    raise KeyError(f"Missing columns: {missing_cols}")
else:
    print("All required columns found.")

# --- Define Organism Type ---
is_virus = df[virus_family_col].notna()
is_archaea = df[archaea_phylum_col].notna() & ~is_virus
is_other = ~is_virus & ~is_archaea
if is_other.sum() > 0:
    warnings.warn(f"{is_other.sum()} proteins did not clearly map to 'Virus' or 'Archaea'. Localization/Mature sequence might be less specific.")

# --- Define Localization Logic ---
conditions = [
    is_archaea & (df[uspnet_col] == 'NO_SP'),
    is_archaea & (df[uspnet_col].isin(['SP', 'TAT'])),
    is_archaea & (df[uspnet_col].isin(['LIPO', 'TATLIPO', 'PILIN'])),
    is_virus & (df[uspnet_col] == 'NO_SP'),
    is_virus & (df[uspnet_col].isin(['SP', 'TAT'])),
    is_virus & (df[uspnet_col].isin(['LIPO', 'TATLIPO', 'PILIN'])),
]
outputs = [
    'Archaea: Cytoplasmic/Membrane (non-SP)',
    'Archaea: Secreted/Membrane (Sec/Tat pathway)',
    'Archaea: Membrane-associated (Lipoprotein/Pilin)',
    'Host: Cytoplasm/Nucleus/Virus Factory',
    'Host: Secretory Pathway (Secreted/Membrane/Organelle)',
    'Host: Membrane-associated (Lipoprotein/Pilin-like)',
]
default_output = 'Unknown/Other'

# --- Apply Localization Logic ---
df[localization_output_col] = np.select(conditions, outputs, default=default_output)
print(f"\nAdded '{localization_output_col}' column.")

# --- Define Mature Sequence Logic ---
def get_mature_sequence(row):
    """
    Calculates the mature protein sequence based on USPNet prediction.
    Removes the signal peptide sequence if predicted.
    """
    full_sequence = row[sequence_col]
    sp_type = row[uspnet_col]
    cleavage_seq = row[cleavage_site_col] # Sequence up to and including cleavage site

    # Ensure sequence is a string (handle potential NaNs)
    if not isinstance(full_sequence, str):
        return None # Or return '', or np.nan depending on desired handling

    # Default to the full sequence
    mature_sequence = full_sequence

    # If a signal peptide is predicted (not NO_SP)
    if sp_type != 'NO_SP':
        # Check if cleavage site sequence is valid (string, not empty)
        if isinstance(cleavage_seq, str) and cleavage_seq:
            # Check if the full sequence starts with the cleavage site sequence
            if full_sequence.startswith(cleavage_seq):
                # Calculate the length of the cleavage site sequence
                cleavage_len = len(cleavage_seq)
                # Extract the sequence *after* the cleavage site
                mature_sequence = full_sequence[cleavage_len:]
            else:
                # Warning: Predicted SP but cleavage site doesn't match start of sequence
                warnings.warn(f"ProteinID {row[protein_id_col]}: SP predicted ({sp_type}), but cleavage site '{cleavage_seq[:20]}...' not found at start of sequence. Using full sequence.")
                mature_sequence = full_sequence # Fallback to full sequence
        else:
            # Warning: Predicted SP but cleavage site data is missing/invalid
            warnings.warn(f"ProteinID {row[protein_id_col]}: SP predicted ({sp_type}), but cleavage site data is missing or invalid. Using full sequence.")
            mature_sequence = full_sequence # Fallback to full sequence

    return mature_sequence

# --- Apply Mature Sequence Logic ---
print("\nCalculating Mature Protein Sequences...")
df[mature_seq_output_col] = df.apply(get_mature_sequence, axis=1)
print(f"Added '{mature_seq_output_col}' column.")

# --- Display Results ---
print("\nValue Counts for Predicted Subcellular Localization:")
print(df[localization_output_col].value_counts())

# Add sequence length columns for comparison
df['Original_Seq_Length'] = df[sequence_col].str.len()
df['Mature_Seq_Length'] = df[mature_seq_output_col].str.len()

# Handle potential None values in Mature_Seq_Length if sequence was None
df['Mature_Seq_Length'] = df['Mature_Seq_Length'].fillna(0).astype(int)

print(f"\nFirst 5 rows showing sequence length changes:")
display_cols = [
    protein_id_col,
    uspnet_col,
    localization_output_col,
    'Original_Seq_Length',
    'Mature_Seq_Length' # Show lengths instead of full sequences for brevity
]
# Ensure columns exist before trying to display them
display_cols = [col for col in display_cols if col in df.columns]
print(df[display_cols].head())

# --- Optional: Save the updated DataFrame ---
output_csv_path = 'proteome_database_v0.8.csv' # Consider incrementing version
try:
    # Select columns to save (optional, can save all)
    # cols_to_save = [...]
    # df.to_csv(output_csv_path, columns=cols_to_save, index=False)
    df.to_csv(output_csv_path, index=False)
    print(f"\nSuccessfully saved updated data to '{output_csv_path}'")
except Exception as e:
    print(f"\nError saving updated data: {e}")

# Clean up temporary length columns if you don't want them saved
# df = df.drop(columns=['Original_Seq_Length', 'Mature_Seq_Length'])


In [ ]:
!pip install tabulate

In [ ]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Path to your main data file (output from the previous step)
main_csv_path = 'proteome_database_v0.8.csv'
# Path to the mapping file provided
mapping_file_path = 'mapping_parquet_proteinid_to_uniprotkb_or_upi.tsv'
# Column names used for merging and updating
protein_id_col = 'ProteinID'
uniprot_col = 'UniProtKB_AC'

# --- Load Data ---
try:
    # Load the main dataframe which should include the empty UniProtKB_AC column
    df_main = pd.read_csv(main_csv_path)
    print(f"Successfully loaded main data '{main_csv_path}'. Shape: {df_main.shape}")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{main_csv_path}'. Make sure it was saved correctly in the previous step.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main CSV: {e}")
    raise

try:
    # Load the mapping file - assuming it's tab-separated (TSV)
    df_mapping = pd.read_csv(mapping_file_path, sep='\t')
    print(f"Successfully loaded mapping file '{mapping_file_path}'. Shape: {df_mapping.shape}")
except FileNotFoundError:
    print(f"Error: Mapping file not found at '{mapping_file_path}'. Please ensure the path is correct.")
    raise
except Exception as e:
    print(f"An error occurred while loading the mapping TSV: {e}")
    raise

# --- Data Validation ---
# Ensure necessary columns exist in both dataframes
if protein_id_col not in df_main.columns:
    print(f"Error: Column '{protein_id_col}' not found in main dataframe '{main_csv_path}'.")
    raise KeyError(f"Missing column: {protein_id_col} in main dataframe")
if uniprot_col not in df_main.columns:
    print(f"Warning: Column '{uniprot_col}' not found in main dataframe '{main_csv_path}'. It will be created.")
    # Add the column if it's missing (though it should exist based on your header)
    df_main[uniprot_col] = np.nan

if protein_id_col not in df_mapping.columns:
    print(f"Error: Column '{protein_id_col}' not found in mapping dataframe '{mapping_file_path}'.")
    raise KeyError(f"Missing column: {protein_id_col} in mapping dataframe")
if uniprot_col not in df_mapping.columns:
    print(f"Error: Column '{uniprot_col}' not found in mapping dataframe '{mapping_file_path}'.")
    raise KeyError(f"Missing column: {uniprot_col} in mapping dataframe")

# --- Prepare for Merge ---
# Check how many rows currently have a UniProt AC in the main dataframe
# Convert potential empty strings or placeholders to NaN for accurate counting
df_main[uniprot_col] = df_main[uniprot_col].replace('', np.nan)
initial_filled_count = df_main[uniprot_col].notna().sum()
print(f"\nInitial count of non-empty '{uniprot_col}' entries in main dataframe: {initial_filled_count}")

# Select only the necessary columns from the mapping file for efficiency
df_mapping_subset = df_mapping[[protein_id_col, uniprot_col]].copy()

# Handle potential duplicate ProteinIDs in the mapping file.
# If a ProteinID maps to multiple UniProt ACs, this keeps the first one found.
duplicates_in_mapping = df_mapping_subset[protein_id_col].duplicated().sum()
if duplicates_in_mapping > 0:
    print(f"Warning: Found {duplicates_in_mapping} duplicate '{protein_id_col}' entries in the mapping file. Keeping the first occurrence for each.")
    df_mapping_subset = df_mapping_subset.drop_duplicates(subset=[protein_id_col], keep='first')

# --- Perform Merge ---
# Use a left merge: keep all rows from df_main, add UniProt ACs from df_mapping_subset.
# 'suffixes' handles the case where the uniprot_col already exists in df_main.
# The original column remains unchanged, the merged data goes into 'UniProtKB_AC_mapped'.
df_merged = pd.merge(
    df_main,
    df_mapping_subset,
    on=protein_id_col,
    how='left',
    suffixes=('', '_mapped') # Suffix for the column coming from the mapping file
)

# --- Update the UniProtKB_AC Column ---
# Check if the merge created the new column with the suffix
if uniprot_col + '_mapped' in df_merged.columns:
    print(f"Updating '{uniprot_col}' column with mapped values...")
    # Fill NaN values in the original UniProtKB_AC column using the mapped values.
    # This preserves any existing values and only fills where it was originally NaN.
    df_merged[uniprot_col] = df_merged[uniprot_col].fillna(df_merged[uniprot_col + '_mapped'])

    # Clean up: Drop the temporary mapped column
    df_merged = df_merged.drop(columns=[uniprot_col + '_mapped'])
else:
    # This case is unlikely with the suffixes parameter used correctly, but included as a fallback.
    print(f"Warning: Merge did not create a separate '{uniprot_col}_mapped' column. Check merge logic.")

# Ensure the updated column is treated as string (or object) to handle NaNs gracefully
df_merged[uniprot_col] = df_merged[uniprot_col].astype(object)

# --- Display Results ---
final_filled_count = df_merged[uniprot_col].notna().sum()
print(f"\nMerge complete.")
print(f"Final count of non-empty '{uniprot_col}' entries: {final_filled_count}")
print(f"Number of entries newly filled: {final_filled_count - initial_filled_count}")

# Show some rows where the UniProt AC was potentially filled
print(f"\nExample rows with '{uniprot_col}' populated (showing first 5):")
print(df_merged[df_merged[uniprot_col].notna()][[protein_id_col, uniprot_col]].head().to_markdown(index=False, numalign="left", stralign="left"))

# Show some rows where the UniProt AC might still be empty (no match in mapping file)
print(f"\nExample rows where '{uniprot_col}' might still be empty (showing first 5):")
print(df_merged[df_merged[uniprot_col].isna()][[protein_id_col, uniprot_col]].head().to_markdown(index=False, numalign="left", stralign="left"))

# --- Save the updated DataFrame ---
# Overwrite the original file with the updated data
output_csv_path = 'proteome_database_v0.9.csv' 
try:
    df_merged.to_csv(output_csv_path, index=False)
    print(f"\nSuccessfully saved updated data back to '{output_csv_path}'")
except Exception as e:
    print(f"\nError saving updated data: {e}")



In [ ]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Path to your main data file (output from the previous step)
main_csv_path = 'proteome_database_v0.8.csv' # Start with v0.8 again
# Path to the mapping file provided (ensure this filename is correct)
mapping_file_path = 'mapping_parquet_proteinid_to_uniprotkb_or_upi.tsv'
# Column names used for merging and updating
protein_id_col = 'ProteinID'
uniprot_col = 'UniProtKB_AC'
# Output file path
output_csv_path = 'proteome_database_v0.9.csv'

# --- Load Data ---
try:
    # Load the main dataframe
    df_main = pd.read_csv(main_csv_path)
    print(f"Successfully loaded main data '{main_csv_path}'. Shape: {df_main.shape}")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{main_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main CSV: {e}")
    raise

try:
    # Load the mapping file - assuming it's tab-separated (TSV)
    df_mapping = pd.read_csv(mapping_file_path, sep='\t')
    print(f"Successfully loaded mapping file '{mapping_file_path}'. Shape: {df_mapping.shape}")
except FileNotFoundError:
    print(f"Error: Mapping file not found at '{mapping_file_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the mapping TSV: {e}")
    raise

# --- Data Validation ---
# Ensure necessary columns exist in both dataframes
required_cols_main = [protein_id_col, uniprot_col]
required_cols_map = [protein_id_col, uniprot_col]

if not all(col in df_main.columns for col in required_cols_main):
    missing = [col for col in required_cols_main if col not in df_main.columns]
    print(f"Error: Columns {missing} not found in main dataframe '{main_csv_path}'.")
    raise KeyError(f"Missing columns in main dataframe")

if not all(col in df_mapping.columns for col in required_cols_map):
    missing = [col for col in required_cols_map if col not in df_mapping.columns]
    print(f"Error: Columns {missing} not found in mapping dataframe '{mapping_file_path}'.")
    raise KeyError(f"Missing columns in mapping dataframe")

# --- Debugging: Inspect IDs ---
print("\n--- Debugging Info ---")
print(f"First 5 '{protein_id_col}' values from main data:")
print(df_main[protein_id_col].head().to_list())
print(f"\nFirst 5 '{protein_id_col}' values from mapping data:")
print(df_mapping[protein_id_col].head().to_list())

# --- Debugging: Clean IDs (Remove Whitespace) ---
# Convert to string and strip whitespace just in case
df_main[protein_id_col] = df_main[protein_id_col].astype(str).str.strip()
df_mapping[protein_id_col] = df_mapping[protein_id_col].astype(str).str.strip()
print("\nApplied .str.strip() to ProteinID columns.")

# --- Debugging: Check Overlap ---
main_ids = set(df_main[protein_id_col])
mapping_ids = set(df_mapping[protein_id_col])
overlapping_ids = main_ids.intersection(mapping_ids)
print(f"\nNumber of unique ProteinIDs in main data: {len(main_ids)}")
print(f"Number of unique ProteinIDs in mapping data: {len(mapping_ids)}")
print(f"Number of ProteinIDs overlapping between the two files: {len(overlapping_ids)}")

if len(overlapping_ids) == 0:
    print(">>> Critical Issue: No ProteinIDs match between the two files! Merge will result in no changes.")
    # Optional: Add logic to strip version numbers if that's suspected
    # Example (use with caution):
    # df_main[protein_id_col + '_base'] = df_main[protein_id_col].str.split('.').str[0]
    # df_mapping[protein_id_col + '_base'] = df_mapping[protein_id_col].str.split('.').str[0]
    # Then merge on protein_id_col + '_base'
else:
     print(">>> Some overlapping IDs found. Proceeding with merge.")
print("--- End Debugging Info ---")


# --- Prepare for Merge ---
# Check how many rows currently have a UniProt AC in the main dataframe
df_main[uniprot_col] = df_main[uniprot_col].replace('', np.nan) # Ensure empty strings are NaN
initial_filled_count = df_main[uniprot_col].notna().sum()
print(f"\nInitial count of non-empty '{uniprot_col}' entries in main dataframe: {initial_filled_count}")

# Select only the necessary columns from the mapping file
df_mapping_subset = df_mapping[[protein_id_col, uniprot_col]].copy()

# Handle potential duplicate ProteinIDs in the mapping file.
duplicates_in_mapping = df_mapping_subset[protein_id_col].duplicated().sum()
if duplicates_in_mapping > 0:
    print(f"Warning: Found {duplicates_in_mapping} duplicate '{protein_id_col}' entries in the mapping file. Keeping the first occurrence for each.")
    df_mapping_subset = df_mapping_subset.drop_duplicates(subset=[protein_id_col], keep='first')

# --- Perform Merge ---
# Use a left merge
df_merged = pd.merge(
    df_main,
    df_mapping_subset,
    on=protein_id_col,
    how='left',
    suffixes=('', '_mapped') # Suffix for the column coming from the mapping file
)

# --- Update the UniProtKB_AC Column ---
if uniprot_col + '_mapped' in df_merged.columns:
    print(f"Updating '{uniprot_col}' column with mapped values...")
    # Use .fillna() to update only where the original column is NaN
    df_merged[uniprot_col] = df_merged[uniprot_col].fillna(df_merged[uniprot_col + '_mapped'])
    # Clean up: Drop the temporary mapped column
    df_merged = df_merged.drop(columns=[uniprot_col + '_mapped'])
else:
    print(f"Warning: Merge did not create a separate '{uniprot_col}_mapped' column.")

# Ensure the updated column is treated as string (or object)
df_merged[uniprot_col] = df_merged[uniprot_col].astype(object)

# --- Display Results ---
final_filled_count = df_merged[uniprot_col].notna().sum()
print(f"\nMerge complete.")
print(f"Final count of non-empty '{uniprot_col}' entries: {final_filled_count}")
print(f"Number of entries newly filled: {final_filled_count - initial_filled_count}")

# Show some rows where the UniProt AC was potentially filled
print(f"\nExample rows with '{uniprot_col}' populated (showing first 5):")
print(df_merged[df_merged[uniprot_col].notna()][[protein_id_col, uniprot_col]].head().to_markdown(index=False, numalign="left", stralign="left"))

# Show some rows where the UniProt AC might still be empty
print(f"\nExample rows where '{uniprot_col}' might still be empty (showing first 5):")
print(df_merged[df_merged[uniprot_col].isna()][[protein_id_col, uniprot_col]].head().to_markdown(index=False, numalign="left", stralign="left"))

# --- Save the updated DataFrame ---
try:
    df_merged.to_csv(output_csv_path, index=False)
    print(f"\nSuccessfully saved updated data back to '{output_csv_path}'")
except Exception as e:
    print(f"\nError saving updated data: {e}")



In [ ]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Path to your main data file (output from the previous step)
main_csv_path = 'proteome_database_v0.8.csv' # Start with v0.8 again
# Path to the mapping file provided (ensure this filename is correct)
mapping_file_path = 'mapping_parquet_proteinid_to_uniprotkb_or_upi.tsv'
# Column names used for merging and updating
protein_id_col = 'ProteinID'
uniprot_col = 'UniProtKB_AC'
# Temporary column for merging based on base ID (without version suffix)
base_id_col = protein_id_col + '_base'
# Output file path
output_csv_path = 'proteome_database_v0.9.csv'

# --- Load Data ---
try:
    # Load the main dataframe
    df_main = pd.read_csv(main_csv_path)
    print(f"Successfully loaded main data '{main_csv_path}'. Shape: {df_main.shape}")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{main_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main CSV: {e}")
    raise

try:
    # Load the mapping file - assuming it's tab-separated (TSV)
    df_mapping = pd.read_csv(mapping_file_path, sep='\t')
    print(f"Successfully loaded mapping file '{mapping_file_path}'. Shape: {df_mapping.shape}")
except FileNotFoundError:
    print(f"Error: Mapping file not found at '{mapping_file_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the mapping TSV: {e}")
    raise

# --- Data Validation ---
# Ensure necessary columns exist in both dataframes
required_cols_main = [protein_id_col, uniprot_col]
required_cols_map = [protein_id_col, uniprot_col]

if not all(col in df_main.columns for col in required_cols_main):
    missing = [col for col in required_cols_main if col not in df_main.columns]
    print(f"Error: Columns {missing} not found in main dataframe '{main_csv_path}'.")
    raise KeyError(f"Missing columns in main dataframe")

if not all(col in df_mapping.columns for col in required_cols_map):
    missing = [col for col in required_cols_map if col not in df_mapping.columns]
    print(f"Error: Columns {missing} not found in mapping dataframe '{mapping_file_path}'.")
    raise KeyError(f"Missing columns in mapping dataframe")

# --- Create Base ID for Merging ---
print("\nCreating base ID columns (without version suffix) for merging...")
# Main data: Split ProteinID at '.' and take the first part
df_main[base_id_col] = df_main[protein_id_col].astype(str).str.split('.', n=1).str[0]
# Mapping data: Assume it already lacks suffix, just copy
df_mapping[base_id_col] = df_mapping[protein_id_col].astype(str).str.strip() # Ensure clean

# --- Debugging: Inspect IDs ---
print("\n--- Debugging Info ---")
print(f"First 5 '{protein_id_col}' values from main data:")
print(df_main[protein_id_col].head().to_list())
print(f"First 5 '{base_id_col}' values from main data:")
print(df_main[base_id_col].head().to_list())

print(f"\nFirst 5 '{protein_id_col}' values from mapping data:")
print(df_mapping[protein_id_col].head().to_list())
print(f"First 5 '{base_id_col}' values from mapping data:")
print(df_mapping[base_id_col].head().to_list())


# --- Debugging: Check Overlap using Base ID ---
main_base_ids = set(df_main[base_id_col])
mapping_base_ids = set(df_mapping[base_id_col])
overlapping_base_ids = main_base_ids.intersection(mapping_base_ids)
print(f"\nNumber of unique Base IDs in main data: {len(main_base_ids)}")
print(f"Number of unique Base IDs in mapping data: {len(mapping_base_ids)}")
print(f"Number of Base IDs overlapping between the two files: {len(overlapping_base_ids)}")

if len(overlapping_base_ids) == 0:
    print(">>> Critical Issue: No Base IDs match between the two files! Merge will result in no changes.")
else:
     print(">>> Some overlapping Base IDs found. Proceeding with merge.")
print("--- End Debugging Info ---")


# --- Prepare for Merge ---
# Check how many rows currently have a UniProt AC in the main dataframe
df_main[uniprot_col] = df_main[uniprot_col].replace('', np.nan) # Ensure empty strings are NaN
initial_filled_count = df_main[uniprot_col].notna().sum()
print(f"\nInitial count of non-empty '{uniprot_col}' entries in main dataframe: {initial_filled_count}")

# Select only the necessary columns from the mapping file for the merge
# Include the base_id_col and the uniprot_col
df_mapping_subset = df_mapping[[base_id_col, uniprot_col]].copy()

# Handle potential duplicate Base IDs in the mapping file.
# If a Base ID maps to multiple UniProt ACs, this keeps the first one found.
duplicates_in_mapping = df_mapping_subset[base_id_col].duplicated().sum()
if duplicates_in_mapping > 0:
    print(f"Warning: Found {duplicates_in_mapping} duplicate '{base_id_col}' entries in the mapping file. Keeping the first occurrence for each.")
    df_mapping_subset = df_mapping_subset.drop_duplicates(subset=[base_id_col], keep='first')

# --- Perform Merge on Base ID ---
# Use a left merge: keep all rows from df_main, add UniProt ACs based on base_id_col match.
print(f"\nPerforming merge on '{base_id_col}'...")
df_merged = pd.merge(
    df_main,
    df_mapping_subset,
    on=base_id_col, # Merge using the base ID column
    how='left',
    suffixes=('', '_mapped') # Suffix for the UniProtKB_AC column coming from mapping
)

# --- Update the UniProtKB_AC Column ---
if uniprot_col + '_mapped' in df_merged.columns:
    print(f"Updating '{uniprot_col}' column with mapped values...")
    # Use .fillna() to update only where the original column is NaN
    # This prevents overwriting any existing values if df_main already had some
    df_merged[uniprot_col] = df_merged[uniprot_col].fillna(df_merged[uniprot_col + '_mapped'])
    # Clean up: Drop the temporary mapped column
    df_merged = df_merged.drop(columns=[uniprot_col + '_mapped'])
else:
    print(f"Warning: Merge did not create a separate '{uniprot_col}_mapped' column.")

# Clean up the temporary base ID column from the final dataframe
if base_id_col in df_merged.columns:
    df_merged = df_merged.drop(columns=[base_id_col])
    print(f"Removed temporary column '{base_id_col}'.")


# Ensure the updated column is treated as string (or object)
df_merged[uniprot_col] = df_merged[uniprot_col].astype(object)

# --- Display Results ---
final_filled_count = df_merged[uniprot_col].notna().sum()
print(f"\nMerge complete.")
print(f"Final count of non-empty '{uniprot_col}' entries: {final_filled_count}")
print(f"Number of entries newly filled: {final_filled_count - initial_filled_count}")

# Show some rows where the UniProt AC was potentially filled
print(f"\nExample rows with '{uniprot_col}' populated (showing first 5):")
print(df_merged[df_merged[uniprot_col].notna()][[protein_id_col, uniprot_col]].head().to_markdown(index=False, numalign="left", stralign="left"))

# Show some rows where the UniProt AC might still be empty
print(f"\nExample rows where '{uniprot_col}' might still be empty (showing first 5):")
print(df_merged[df_merged[uniprot_col].isna()][[protein_id_col, uniprot_col]].head().to_markdown(index=False, numalign="left", stralign="left"))

# --- Save the updated DataFrame ---
try:
    df_merged.to_csv(output_csv_path, index=False)
    print(f"\nSuccessfully saved updated data back to '{output_csv_path}'")
except Exception as e:
    print(f"\nError saving updated data: {e}")



In [ ]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Path to your updated main data file
main_csv_path = 'proteome_database_v0.9.csv'
# Path to the file containing UniProt ACs found in AFDB
afdb_uniprot_list_path = 'PDB_AFDB_screening/afdb_processing_output/afdb_found_uniprot_acs_or_upi.csv'
# Column names
uniprot_col = 'UniProtKB_AC'
protein_id_col = 'ProteinID' # Used for displaying examples

# --- Load Data ---
try:
    # Load the main dataframe (which now has UniProtKB_AC populated)
    df_main = pd.read_csv(main_csv_path)
    print(f"Successfully loaded main data '{main_csv_path}'. Shape: {df_main.shape}")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{main_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main CSV: {e}")
    raise

try:
    # Load the AFDB UniProt list file
    # Assuming it's a CSV and the UniProt IDs are in the first column (index 0)
    # Adjust 'header=None' and 'usecols=[0]' if the file has headers or a different structure
    df_afdb_uniprot = pd.read_csv(afdb_uniprot_list_path, header=None, usecols=[0], names=[uniprot_col])
    print(f"Successfully loaded AFDB UniProt list '{afdb_uniprot_list_path}'. Shape: {df_afdb_uniprot.shape}")
except FileNotFoundError:
    print(f"Error: AFDB UniProt list file not found at '{afdb_uniprot_list_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the AFDB UniProt list CSV: {e}")
    raise

# --- Process and Count ---
# Get the set of unique UniProt ACs from the AFDB list
# Drop any potential NaN values and ensure they are strings
afdb_uniprot_set = set(df_afdb_uniprot[uniprot_col].dropna().astype(str))
print(f"\nFound {len(afdb_uniprot_set)} unique UniProt ACs in the AFDB list file.")

# Filter the main dataframe:
# 1. Keep rows where UniProtKB_AC is not NaN/null
# 2. Keep rows where the UniProtKB_AC is present in the afdb_uniprot_set
df_main_filtered = df_main[
    df_main[uniprot_col].notna() &
    df_main[uniprot_col].astype(str).isin(afdb_uniprot_set)
].copy() # Use .copy() to avoid SettingWithCopyWarning

# Count the number of rows in the filtered dataframe
overlap_count = len(df_main_filtered)
total_with_uniprot = df_main[uniprot_col].notna().sum()

print(f"\nTotal entries in main database with a UniProtKB_AC: {total_with_uniprot}")
print(f"Number of entries whose UniProtKB_AC was found in the AFDB list: {overlap_count}")

if total_with_uniprot > 0:
    percentage = (overlap_count / total_with_uniprot) * 100
    print(f"Percentage of mapped entries found in AFDB list: {percentage:.2f}%")

# Display a few examples of overlapping proteins
print("\nExample ProteinIDs whose UniProtKB_AC matched the AFDB list (first 5):")
print(df_main_filtered[[protein_id_col, uniprot_col]].head().to_markdown(index=False, numalign="left", stralign="left"))



In [ ]:
import pandas as pd
import numpy as np
import re # Import regular expressions for UniProt ID check

# --- Configuration ---
# Path to the AFESM cluster metadata file (unzipped)
cluster_meta_path = '2-repID_isOnlyESM_nMem_nAllMem_repPlddt_avgPlddt_avgAllPlddt_repLen_avgLen_avgAllLen_LCAtaxID_nBiome_LCBID.tsv'
# Path to the AFESM cluster membership file (unzipped)
cluster_members_path = '1-AFESMClusters-repId_memId_cluFlag_taxId_biomeId.tsv' # File 1 from description, assuming entryId is memId based on docs
# Output file for the filtered UniProt IDs
output_ids_path = 'afesm_esm_only_uniprot_ids.txt'

# Define expected column names for AFESM files based on documentation
# File 2: Cluster Metadata
meta_cols = [
    'repID', 'isOnlyESM', 'nMem', 'nAllMem', 'repPlddt', 'avgPlddt',
    'avgAllPlddt', 'repLen', 'avgLen', 'avgAllLen', 'LCAtaxID',
    'nBiome', 'LCBID'
]
# File 1: Cluster Membership - Assuming entryId is the member ID based on file name
members_cols = ['repId', 'memId', 'cluFlag', 'taxId', 'biomeID'] # Using File 1 description, mapping entryId to memId

# Variables to hold specific column names used in the script
afesm_rep_id_col = 'repID' # Use 'repID' from meta_cols for consistency
afesm_member_id_col = 'memId' # From members_cols
afesm_is_esm_only_col = 'isOnlyESM' # From meta_cols

# --- Load Cluster Metadata ---
try:
    # Load cluster metadata, specifying NO header and assigning names.
    df_meta = pd.read_csv(cluster_meta_path, sep='\t', header=None, names=meta_cols)
    print(f"Successfully loaded cluster metadata '{cluster_meta_path}' (no header, unzipped). Shape: {df_meta.shape}")
except FileNotFoundError:
    print(f"Error: Cluster metadata file not found at '{cluster_meta_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading cluster metadata: {e}")
    raise

# --- Identify ESM-only Cluster Representatives ---
# Ensure the isOnlyESM column is numeric
df_meta[afesm_is_esm_only_col] = pd.to_numeric(df_meta[afesm_is_esm_only_col], errors='coerce')
esm_only_reps = set(df_meta[df_meta[afesm_is_esm_only_col] == 1][afesm_rep_id_col].unique())
print(f"\nIdentified {len(esm_only_reps)} representative IDs for ESM-only clusters.")

# Free up memory from df_meta as it's no longer needed
del df_meta
import gc
gc.collect()
print("Freed memory from metadata dataframe.")

if not esm_only_reps:
    print("Warning: No ESM-only cluster representatives found. Stopping.")
    # Exit if no reps found, otherwise the next step processes everything
    exit()

# --- Process Cluster Members in Chunks (to save memory) ---
print(f"\nProcessing cluster members file '{cluster_members_path}' in chunks...")
chunk_size = 10_000_000 # Process 10 million rows at a time, adjust if needed
esm_only_uniprot_ids_set = set()
processed_rows = 0

try:
    # Use iterator=True and get_chunk to read the large file piece by piece
    chunk_iterator = pd.read_csv(
        cluster_members_path,
        sep='\t',
        header=None,
        names=members_cols,
        low_memory=False,
        iterator=True,
        chunksize=chunk_size
    )

    for i, chunk in enumerate(chunk_iterator):
        processed_rows += len(chunk)
        print(f"  Processing chunk {i+1} (Rows {processed_rows - chunk_size + 1} - {processed_rows})...")

        # Ensure correct data types for filtering
        chunk['repId'] = chunk['repId'].astype(str)
        chunk[afesm_member_id_col] = chunk[afesm_member_id_col].astype(str)

        # Filter the chunk for members belonging to ESM-only clusters
        esm_only_members_chunk = chunk[chunk['repId'].isin(esm_only_reps)]

        # Extract member IDs from this chunk
        member_ids_chunk = set(esm_only_members_chunk[afesm_member_id_col].dropna())

        # Filter for potential UniProt IDs and add to the main set
        for mem_id in member_ids_chunk:
             if not mem_id.startswith('MGYP') and not mem_id.startswith('pdb|'):
                 esm_only_uniprot_ids_set.add(mem_id)

        print(f"    Found {len(member_ids_chunk)} members in ESM-only clusters in this chunk.")
        print(f"    Current total unique potential UniProt IDs found: {len(esm_only_uniprot_ids_set)}")

    print(f"\nFinished processing {processed_rows} rows from cluster members file.")
    print(f"Total unique potential UniProt IDs found in ESM-only clusters: {len(esm_only_uniprot_ids_set)}")

    # --- Save the Filtered IDs to a File ---
    print(f"\nSaving filtered UniProt IDs to '{output_ids_path}'...")
    with open(output_ids_path, 'w') as f_out:
        for uniprot_id in sorted(list(esm_only_uniprot_ids_set)): # Sort for consistency
            f_out.write(f"{uniprot_id}\n")
    print("Save complete.")

except FileNotFoundError:
    print(f"Error: Cluster members file not found at '{cluster_members_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while processing cluster members: {e}")
    raise

In [ ]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Path to your main data file
main_csv_path = 'proteome_database_v0.9.csv'
# Path to the filtered list of ESM-only UniProt IDs created in the previous step
afesm_ids_path = 'afesm_esm_only_uniprot_ids.txt'
# Column names in your main file
protein_id_col = 'ProteinID'
uniprot_col = 'UniProtKB_AC'
# Optional: Output file for your proteins that matched
output_match_path = 'proteins_hitting_afesm_esm_only_uniprot.csv'

# --- Load Data ---
try:
    df_main = pd.read_csv(main_csv_path)
    print(f"Successfully loaded main data '{main_csv_path}'. Shape: {df_main.shape}")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{main_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main CSV: {e}")
    raise

try:
    # Load the filtered AFESM UniProt IDs into a set for efficient lookup
    print(f"Loading filtered AFESM UniProt IDs from '{afesm_ids_path}'...")
    with open(afesm_ids_path, 'r') as f_in:
        # Read lines, strip whitespace/newlines, and add to set
        afesm_esm_only_uniprot_ids_set = {line.strip() for line in f_in if line.strip()}
    print(f"Successfully loaded {len(afesm_esm_only_uniprot_ids_set)} unique IDs into a set.")
except FileNotFoundError:
    print(f"Error: AFESM ID list file not found at '{afesm_ids_path}'. Make sure the previous step ran correctly.")
    raise
except Exception as e:
    print(f"An error occurred while loading the AFESM ID list: {e}")
    raise

# --- Compare Your Database UniProt IDs to the AFESM Set ---
# Get the set of non-null UniProt ACs from your main database
main_uniprot_ids = set(df_main[uniprot_col].dropna().astype(str))
print(f"\nFound {len(main_uniprot_ids)} unique UniProt ACs in your main database.")

# Find the intersection
overlapping_uniprot_ids = main_uniprot_ids.intersection(afesm_esm_only_uniprot_ids_set)
overlap_count = len(overlapping_uniprot_ids)

print(f"\nNumber of your UniProtKB_ACs found within the AFESM ESM-only UniProt ID set: {overlap_count}")

if len(main_uniprot_ids) > 0:
    percentage = (overlap_count / len(main_uniprot_ids)) * 100
    print(f"Percentage of your mapped UniProt IDs found in the AFESM ESM-only set: {percentage:.2f}%")
else:
    print("No UniProt IDs found in your main database to compare.")

# --- Display Results & Save ---
# Filter the main dataframe to show the proteins that matched
df_main_matched = df_main[df_main[uniprot_col].astype(str).isin(overlapping_uniprot_ids)].copy()

print("\nExample ProteinIDs whose UniProtKB_AC matched the AFESM ESM-only set (first 10):")
print(df_main_matched[[protein_id_col, uniprot_col]].head(10).to_markdown(index=False, numalign="left", stralign="left"))

# Save the list of your ProteinIDs that hit the ESM-only set
if not df_main_matched.empty:
    try:
        df_main_matched[[protein_id_col, uniprot_col]].to_csv(output_match_path, index=False)
        print(f"\nSaved list of matching ProteinIDs ({overlap_count} entries) to '{output_match_path}'")
    except Exception as e:
        print(f"\nError saving matching ProteinIDs: {e}")
else:
    print("\nNo matching proteins found to save.")



In [ ]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Path to your main data file
main_csv_path = 'proteome_database_v0.9.csv'
# Path to the filtered list of ESM-only UniProt IDs created in the previous step
afesm_ids_path = 'afesm_esm_only_uniprot_ids.txt'
# Column names in your main file
protein_id_col = 'ProteinID'
uniprot_col = 'UniProtKB_AC'
# Optional: Output file for your proteins that matched
output_match_path = 'proteins_hitting_afesm_esm_only_uniprot.csv'

# --- Load Data ---
try:
    df_main = pd.read_csv(main_csv_path)
    print(f"Successfully loaded main data '{main_csv_path}'. Shape: {df_main.shape}")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{main_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main CSV: {e}")
    raise

try:
    # Load the filtered AFESM UniProt IDs into a set for efficient lookup
    print(f"Loading filtered AFESM UniProt IDs from '{afesm_ids_path}'...")
    afesm_esm_only_uniprot_ids_list = [] # Load into list first for inspection
    with open(afesm_ids_path, 'r') as f_in:
        for line in f_in:
            stripped_line = line.strip()
            if stripped_line: # Ensure non-empty lines
                afesm_esm_only_uniprot_ids_list.append(stripped_line)

    afesm_esm_only_uniprot_ids_set = set(afesm_esm_only_uniprot_ids_list)
    print(f"Successfully loaded {len(afesm_esm_only_uniprot_ids_set)} unique IDs into a set.")

    # --- Debugging: Print first few loaded IDs ---
    print("\n--- Debugging Info ---")
    print("First 10 UniProt IDs loaded from AFESM list file:")
    print(afesm_esm_only_uniprot_ids_list[:10])
    print("--- End Debugging Info ---")

except FileNotFoundError:
    print(f"Error: AFESM ID list file not found at '{afesm_ids_path}'. Make sure the previous step ran correctly.")
    raise
except Exception as e:
    print(f"An error occurred while loading the AFESM ID list: {e}")
    raise

# --- Compare Your Database UniProt IDs to the AFESM Set ---
# Get the set of non-null UniProt ACs from your main database
main_uniprot_list = df_main[uniprot_col].dropna().astype(str).tolist() # Get as list first
main_uniprot_ids = set(main_uniprot_list)
print(f"\nFound {len(main_uniprot_ids)} unique UniProt ACs in your main database.")

# --- Debugging: Print first few loaded IDs from main DB ---
print("\n--- Debugging Info ---")
print("First 10 non-null UniProt IDs loaded from main database:")
# Ensure list isn't empty before slicing
print(main_uniprot_list[:10] if main_uniprot_list else "No UniProt IDs found in main DB")
print("--- End Debugging Info ---")

# Find the intersection
print("\nPerforming intersection...")
overlapping_uniprot_ids = main_uniprot_ids.intersection(afesm_esm_only_uniprot_ids_set)
overlap_count = len(overlapping_uniprot_ids)

print(f"\nNumber of your UniProtKB_ACs found within the AFESM ESM-only UniProt ID set: {overlap_count}")

if len(main_uniprot_ids) > 0:
    percentage = (overlap_count / len(main_uniprot_ids)) * 100
    print(f"Percentage of your mapped UniProt IDs found in the AFESM ESM-only set: {percentage:.2f}%")
else:
    print("No UniProt IDs found in your main database to compare.")

# --- Display Results & Save ---
# Filter the main dataframe to show the proteins that matched
df_main_matched = df_main[df_main[uniprot_col].astype(str).isin(overlapping_uniprot_ids)].copy()

print("\nExample ProteinIDs whose UniProtKB_AC matched the AFESM ESM-only set (first 10):")
print(df_main_matched[[protein_id_col, uniprot_col]].head(10).to_markdown(index=False, numalign="left", stralign="left"))

# Save the list of your ProteinIDs that hit the ESM-only set
if not df_main_matched.empty:
    try:
        df_main_matched[[protein_id_col, uniprot_col]].to_csv(output_match_path, index=False)
        print(f"\nSaved list of matching ProteinIDs ({overlap_count} entries) to '{output_match_path}'")
    except Exception as e:
        print(f"\nError saving matching ProteinIDs: {e}")
else:
    print("\nNo matching proteins found to save.")



In [ ]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Path to your main data file
main_csv_path = 'proteome_database_v0.9.csv'
# Path to the filtered list of ESM-only UniProt IDs created in the previous step
afesm_ids_path = 'afesm_esm_only_uniprot_ids.txt'
# Column names in your main file
protein_id_col = 'ProteinID'
uniprot_col = 'UniProtKB_AC'
# Optional: Output file for your proteins that matched
output_match_path = 'proteins_hitting_afesm_esm_only_uniprot.csv'

# --- Load Data ---
try:
    df_main = pd.read_csv(main_csv_path)
    print(f"Successfully loaded main data '{main_csv_path}'. Shape: {df_main.shape}")
except FileNotFoundError:
    print(f"Error: Main data file not found at '{main_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main CSV: {e}")
    raise

try:
    # Load the filtered AFESM UniProt IDs into a set for efficient lookup
    print(f"Loading filtered AFESM UniProt IDs from '{afesm_ids_path}'...")
    afesm_esm_only_uniprot_ids_list = [] # Load into list first for inspection
    with open(afesm_ids_path, 'r') as f_in:
        for line in f_in:
            stripped_line = line.strip()
            if stripped_line: # Ensure non-empty lines
                afesm_esm_only_uniprot_ids_list.append(stripped_line)

    # Convert to uppercase when creating the set
    afesm_esm_only_uniprot_ids_set = {item.upper() for item in afesm_esm_only_uniprot_ids_list}
    print(f"Successfully loaded {len(afesm_esm_only_uniprot_ids_set)} unique UPPERCASE IDs into a set.")

    # --- Debugging: Print first few loaded IDs ---
    print("\n--- Debugging Info ---")
    print("First 10 UniProt IDs loaded from AFESM list file (as read):")
    print(afesm_esm_only_uniprot_ids_list[:10])
    print("First 10 UniProt IDs from AFESM list file (UPPERCASE set sample):")
    # Convert set back to list temporarily for slicing sample
    print(list(afesm_esm_only_uniprot_ids_set)[:10])
    print("--- End Debugging Info ---")


except FileNotFoundError:
    print(f"Error: AFESM ID list file not found at '{afesm_ids_path}'. Make sure the previous step ran correctly.")
    raise
except Exception as e:
    print(f"An error occurred while loading the AFESM ID list: {e}")
    raise

# --- Compare Your Database UniProt IDs to the AFESM Set ---
# Get the set of non-null UniProt ACs from your main database, convert to uppercase
main_uniprot_list = df_main[uniprot_col].dropna().astype(str).tolist() # Get as list first
main_uniprot_ids = {item.upper() for item in main_uniprot_list} # Convert to uppercase set
print(f"\nFound {len(main_uniprot_ids)} unique UPPERCASE UniProt ACs in your main database.")

# --- Debugging: Print first few loaded IDs from main DB ---
print("\n--- Debugging Info ---")
print("First 10 non-null UniProt IDs loaded from main database (as read):")
print(main_uniprot_list[:10] if main_uniprot_list else "No UniProt IDs found in main DB")
print("First 10 non-null UniProt IDs from main database (UPPERCASE set sample):")
print(list(main_uniprot_ids)[:10] if main_uniprot_ids else "No UniProt IDs found in main DB")
print("--- End Debugging Info ---")


# Find the intersection
print("\nPerforming intersection...")
overlapping_uniprot_ids = main_uniprot_ids.intersection(afesm_esm_only_uniprot_ids_set)
overlap_count = len(overlapping_uniprot_ids)

print(f"\nNumber of your UniProtKB_ACs found within the AFESM ESM-only UniProt ID set: {overlap_count}")

if len(main_uniprot_ids) > 0:
    percentage = (overlap_count / len(main_uniprot_ids)) * 100
    print(f"Percentage of your mapped UniProt IDs found in the AFESM ESM-only set: {percentage:.2f}%")
else:
    print("No UniProt IDs found in your main database to compare.")

# --- Display Results & Save ---
# Filter the main dataframe to show the proteins that matched
# Need to compare the uppercase version of the column to the overlapping set
df_main_matched = df_main[df_main[uniprot_col].astype(str).str.upper().isin(overlapping_uniprot_ids)].copy()


print("\nExample ProteinIDs whose UniProtKB_AC matched the AFESM ESM-only set (first 10):")
print(df_main_matched[[protein_id_col, uniprot_col]].head(10).to_markdown(index=False, numalign="left", stralign="left"))

# Save the list of your ProteinIDs that hit the ESM-only set
if not df_main_matched.empty:
    try:
        df_main_matched[[protein_id_col, uniprot_col]].to_csv(output_match_path, index=False)
        print(f"\nSaved list of matching ProteinIDs ({overlap_count} entries) to '{output_match_path}'")
    except Exception as e:
        print(f"\nError saving matching ProteinIDs: {e}")
else:
    print("\nNo matching proteins found to save.")



In [ ]:
import pandas as pd

# --- Configuration ---
# Input file path
input_csv_path = 'proteome_database_v0.9.csv'
# Output file path
output_csv_path = 'proteome_database_v10.csv'
# Columns to drop
columns_to_drop = [
    'AFDB_Status',
    'SeqSearch_PDB_Hit',
    'SeqSearch_AFDB_Hit',
    'SeqSearch_MGnify_Hit'
]

# --- Load Data ---
try:
    df = pd.read_csv(input_csv_path)
    print(f"Successfully loaded '{input_csv_path}'. Shape: {df.shape}")
    print(f"Original columns ({len(df.columns)}): {df.columns.tolist()}")
except FileNotFoundError:
    print(f"Error: Input file not found at '{input_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")
    raise

# --- Drop Columns ---
# Check which of the specified columns actually exist in the DataFrame
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
missing_columns = [col for col in columns_to_drop if col not in df.columns]

if missing_columns:
    print(f"\nWarning: The following columns specified for dropping were not found: {missing_columns}")

if not existing_columns_to_drop:
    print("\nNo columns to drop were found in the DataFrame. Saving original data.")
    df_dropped = df.copy() # Create a copy to proceed with saving
else:
    print(f"\nDropping columns: {existing_columns_to_drop}")
    try:
        df_dropped = df.drop(columns=existing_columns_to_drop)
        print(f"Columns dropped successfully.")
        print(f"New columns ({len(df_dropped.columns)}): {df_dropped.columns.tolist()}")
    except Exception as e:
        print(f"An error occurred while dropping columns: {e}")
        raise

# --- Save Updated Data ---
try:
    df_dropped.to_csv(output_csv_path, index=False)
    print(f"\nSuccessfully saved updated data to '{output_csv_path}'. Shape: {df_dropped.shape}")
except Exception as e:
    print(f"\nError saving updated data: {e}")

In [ ]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Input file path (output from the previous step)
input_csv_path = 'proteome_database_v10.csv'
# Output file path for the unique virus names
output_txt_path = 'unique_virus_names.txt'
# Column containing the virus names
virus_name_col = 'Virus_Name'

# --- Load Data ---
try:
    df = pd.read_csv(input_csv_path)
    print(f"Successfully loaded '{input_csv_path}'. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: Input file not found at '{input_csv_path}'. Make sure the previous step ran correctly.")
    raise
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")
    raise

# --- Extract Unique Virus Names ---
# Check if the column exists
if virus_name_col not in df.columns:
    print(f"Error: Column '{virus_name_col}' not found in the DataFrame.")
    raise KeyError(f"Column '{virus_name_col}' not found.")

# Select the column, drop missing values (NaN, None, etc.), and get unique values
unique_names = df[virus_name_col].dropna().unique()

# Convert to a list and sort alphabetically
unique_names_list = sorted(list(unique_names))

print(f"\nFound {len(unique_names_list)} unique virus names.")

# --- Save to Text File ---
try:
    print(f"Saving unique virus names to '{output_txt_path}'...")
    with open(output_txt_path, 'w') as f_out:
        for name in unique_names_list:
            f_out.write(f"{name}\n")
    print("Save complete.")
    print(f"\nFirst 10 unique names saved:")
    print(unique_names_list[:10])

except Exception as e:
    print(f"\nAn error occurred while saving the text file: {e}")

In [ ]:
import pandas as pd
import numpy as np
import re

# --- Configuration ---
# Input file path (output from the previous step)
input_csv_path = 'proteome_database_v10.csv'
# Output file path
output_csv_path = 'proteome_database_v11.csv'
# Column containing the virus names
virus_name_col = 'Virus_Name'
# Column to create/update with the family assignment
virus_family_col = 'Virus_Family'

# --- Keyword to Family Mapping ---
# Dictionary mapping lowercase keywords to family names.
# Order matters: More specific keywords should come before less specific ones.
# (e.g., 'moumouvirus' before 'virus')
# Based on unique_virus_names.txt and known NCLDV families/groups.
FAMILY_MAP = {
    # Mimiviridae and related groups (Imitervirales)
    'mimivirus': 'Mimiviridae',
    'mamavirus': 'Mimiviridae',
    'moumouvirus': 'Mimiviridae',
    'megavirus': 'Mimiviridae', # Proposed Megaviridae, often grouped
    'tupanvirus': 'Mimiviridae', # Proposed Kheliviricetes
    'cedratvirus': 'Mimiviridae', # Proposed Pithoviricetes
    'faustovirus': 'Mimiviridae', # Proposed Duplodnaviria
    'pacmanvirus': 'Mimiviridae', # Proposed Duplodnaviria
    'orpheovirus': 'Mimiviridae', # Proposed Pithoviricetes
    'klosneuvirus': 'Mimiviridae', # Proposed Klosneuvirinae
    'catovirus': 'Mimiviridae', # Proposed Klosneuvirinae
    'hokovirus': 'Mimiviridae', # Proposed Klosneuvirinae
    'indivirus': 'Mimiviridae', # Proposed Klosneuvirinae
    'samba_virus': 'Mimiviridae',
    'bandra_megavirus': 'Mimiviridae', # Likely related
    'niemeyer_virus': 'Mimiviridae',
    'terrestrivirus': 'Mimiviridae',
    'harvfovirus': 'Mimiviridae',
    'barrevirus': 'Mimiviridae',
    'dasosvirus': 'Mimiviridae',
    'gaeavirus': 'Mimiviridae',
    'satyrvirus': 'Mimiviridae',
    'hirudovirus': 'Mimiviridae',
    'edafosvirus': 'Mimiviridae',
    'homavirus': 'Mimiviridae',
    'acanthamoeba_polyphaga_lentillevirus': 'Mimiviridae', # Added back
    'cotonvirus': 'Mimiviridae',
    'hyperionvirus': 'Mimiviridae',
    'powai_lake_megavirus': 'Mimiviridae', # Likely related
    # Marseilleviridae
    'marseillevirus': 'Marseilleviridae',
    'lausannevirus': 'Marseilleviridae',
    'tokyovirus': 'Marseilleviridae',
    'noumeavirus': 'Marseilleviridae',
    'kurlavirus': 'Marseilleviridae',
    'port-miou_virus': 'Marseilleviridae',
    'golden marseillevirus': 'Marseilleviridae', # Handle space
    # Phycodnaviridae
    'phycodnavirus': 'Phycodnaviridae',
    'chlorella_virus': 'Phycodnaviridae',
    'ostreococcus_virus': 'Phycodnaviridae',
    'micromonas_pusilla_virus': 'Phycodnaviridae',
    'bathycoccus_virus': 'Phycodnaviridae',
    'phaeocystis_globosa_virus': 'Phycodnaviridae',
    'emiliania_huxleyi_virus': 'Phycodnaviridae',
    'chrysochromulina_virus': 'Phycodnaviridae',
    'feldmannia_virus': 'Phycodnaviridae',
    'ectocarpus_siliculosus_virus': 'Phycodnaviridae',
    'prasinovirus': 'Phycodnaviridae', # Often grouped here
    # Iridoviridae
    'ranavirus': 'Iridoviridae', # Specific genus first
    'lymphocystis_disease_virus': 'Iridoviridae', # Specific genus first
    'chloriridovirus': 'Iridoviridae', # Specific genus first
    'iridovirus': 'Iridoviridae', # General term last for this family
    # Ascoviridae
    'ascovirus': 'Ascoviridae',
    # Pithoviridae
    'pithovirus': 'Pithoviridae',
    # Pandoraviridae
    'pandoravirus': 'Pandoraviridae',
    # Yaraviridae
    'yaravirus': 'Yaraviridae',
    # Specific Archaeal Virus Families
    'bicaudavirus': 'Bicaudaviridae',
    'acidianus_two-tailed_virus': 'Bicaudaviridae',
    'sulfolobus_virus_stsv': 'Rudiviridae',
    'acidianus_tailed_spindle_virus': 'Fuselloviridae',
    'sulfolobus_monocaudavirus': 'Fuselloviridae', # Or proposed Thaspiviridae
    # Grouping for other proposed NCLDV families/genera
    'faunusvirus': 'Chaseviridae',
    'solivirus': 'Pithoviridae-like',
    'solumvirus': 'Pithoviridae-like',
    # Catch-all for 'virus' if no other keyword matched
    'virus': 'Unclassified Virus' # General catch-all if name contains 'virus'
}

# --- Helper Function ---
def assign_family(name):
    """Assigns a virus family based on keywords in the name."""
    if pd.isna(name):
        return np.nan # Return NaN if input is NaN

    name_lower = str(name).lower() # Convert to string and lower case

    for keyword, family in FAMILY_MAP.items():
        # Use regex to find keyword as a whole word or part of compound name
        # \b matches word boundaries, allowing matches like 'ranavirus' but not 'miranavirus'
        # We also allow matches if the keyword is followed by '_' or '-' or ends the string
        # Or if it starts the string and is followed by '_' or '-'
        # This handles cases like 'Megavirus_lba' or 'Frog_virus_3'
        pattern = r'(?:^|\b|_|-)' + re.escape(keyword) + r'(?:$|\b|_|-)'
        if re.search(pattern, name_lower):
            return family

    # If no keyword matches, return Unknown
    return 'Unknown'

# --- Load Data ---
try:
    df = pd.read_csv(input_csv_path)
    print(f"Successfully loaded '{input_csv_path}'. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: Input file not found at '{input_csv_path}'. Make sure the previous step ran correctly.")
    raise
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")
    raise

# --- Apply Mapping ---
print(f"\nAssigning families based on '{virus_name_col}'...")
# Apply the function to the Virus_Name column
# Ensure the target column exists, create if not
if virus_family_col not in df.columns:
    df[virus_family_col] = np.nan

# Apply the function only where Virus_Name is not null
df[virus_family_col] = df[virus_name_col].apply(assign_family)
print("Family assignment complete.")

# --- Display Results ---
print("\nValue counts for assigned Virus Families:")
print(df[virus_family_col].value_counts(dropna=False)) # Include NaNs in count

print(f"\nExample rows with assigned '{virus_family_col}' (showing first 15 where Virus_Name is not NaN):")
print(df[df[virus_name_col].notna()][[virus_name_col, virus_family_col]].head(15).to_markdown(index=False, numalign="left", stralign="left"))

# --- Save Updated Data ---
try:
    df.to_csv(output_csv_path, index=False)
    print(f"\nSuccessfully saved updated data with '{virus_family_col}' to '{output_csv_path}'.")
except Exception as e:
    print(f"\nError saving updated data: {e}")



In [ ]:
import pandas as pd
import numpy as np
import os

# --- Configuration ---
# Input file path (the scrambled combined file)
input_csv_path = 'proteome_database_combined_final_v1.3.csv'
# Output file path (overwrite the input or save as new version)
output_csv_path = 'proteome_database_combined_final_v1.3.csv' # Overwriting

# Define the DESIRED column order
# Based on the schema of proteome_database_v1.1.csv plus added columns
desired_column_order = [
    'ProteinID', 'Sequence', 'Length', 'Source_Dataset',
    'Source_Genome_Assembly_Accession', 'Source_Protein_Annotation',
    'NCBI_TaxID', 'Asgard_Phylum', 'Virus_Family', 'Virus_Name',
    'Orthogroup', # Suffixed version
    'IPR_Signatures', 'IPR_GO_Terms', 'UniProtKB_AC',
    'Num_Domains', 'Domain_Architecture', 'Type', 'Is_Hypothetical',
    'Has_Known_Structure', # Original flag
    'Percent_Disorder',
    'Specific_Functional_Category', 'Category_Trigger',
    'Signal_Peptide_USPNet', 'SP_Cleavage_Site_USPNet',
    'Predicted_Subcellular_Localization', 'Mature_Protein_Sequence',
    'Original_Seq_Length', 'Mature_Seq_Length',
    'Group', # Asgard/GV/Other
    'SeqSearch_PDB_Hit', # Added back
    'SeqSearch_AFDB_Hit', # Added back
    'Has_Reference_Structure' # Derived flag
    # Add any other columns that should be present
]

# --- Load Data ---
print(f"--- Loading Scrambled Database ---")
try:
    df = pd.read_csv(input_csv_path, low_memory=False)
    print(f"Loaded '{input_csv_path}'. Shape: {df.shape}")
    print(f"Original (scrambled) columns: {df.columns.tolist()}")
except FileNotFoundError:
    print(f"Error: Input file not found at '{input_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")
    raise

# --- Reorder Columns ---
print("\n--- Reordering Columns ---")
# Check if all desired columns are actually present in the loaded dataframe
current_cols = df.columns.tolist()
final_cols = []
missing_cols = []
extra_cols = [col for col in current_cols if col not in desired_column_order]

for col in desired_column_order:
    if col in current_cols:
        final_cols.append(col)
    else:
        missing_cols.append(col)

# Add any extra columns found in the df but not in the desired list to the end
final_cols.extend(extra_cols)

if missing_cols:
    print(f"Warning: The following columns defined in 'desired_column_order' were NOT found in the loaded CSV: {missing_cols}")
if extra_cols:
     print(f"Warning: The following columns were found in the CSV but not in 'desired_column_order' (will be appended to the end): {extra_cols}")

try:
    df_reordered = df[final_cols]
    print("Columns successfully reordered.")
    print(f"New column order: {df_reordered.columns.tolist()}")
except KeyError as e:
     print(f"Error during reordering - a column in final_cols was unexpectedly missing: {e}")
     # Fallback: Don't reorder if something went wrong
     df_reordered = df
except Exception as e:
     print(f"An unexpected error occurred during reordering: {e}")
     df_reordered = df


# --- Save Reordered Data ---
try:
    df_reordered.to_csv(output_csv_path, index=False)
    print(f"\nSuccessfully saved reordered data back to '{output_csv_path}'.")
except Exception as e:
    print(f"\nError saving reordered data: {e}")



In [ ]:
import pandas as pd
import numpy as np
import os

# --- Configuration ---
# Input file path (the reordered combined file from Cell 7)
input_csv_path = 'proteome_database_combined_final_v1.3.csv'
# Input path for MAIN DB *after* USPNet integration (contains correct derived cols)
# !! CHECK THIS FILENAME !! Example: v0.8 if USPNet was added to v0.7
main_db_with_uspnet_path = 'proteome_database_v1.1.csv'
# Input path for FILTERED-OUT DB *after* USPNet integration (contains correct derived cols)
# !! CHECK THIS FILENAME !! Example: v0.7 if USPNet was added to v0.6
filtered_db_with_uspnet_path = 'all_filtered_out_proteins_v0.8.csv'
# Output file path
output_csv_path = 'proteome_database_combined_final_v1.4_complete.csv' # Final analysis-ready version

# Columns to add/populate
protein_id_col = 'ProteinID'
cols_to_add = [
    'Predicted_Subcellular_Localization',
    'Mature_Protein_Sequence',
    'Mature_Seq_Length'
]
# Also include ProteinID for merging
cols_to_extract = [protein_id_col] + cols_to_add

# --- Load Target Combined Data ---
print(f"--- Loading Reordered Combined Database ---")
try:
    df_combined = pd.read_csv(input_csv_path, low_memory=False)
    print(f"Loaded '{input_csv_path}'. Shape: {df_combined.shape}")
    # Drop existing (likely empty or incomplete) versions of the columns if they exist
    existing_cols_to_drop = [col for col in cols_to_add if col in df_combined.columns]
    if existing_cols_to_drop:
        print(f"Dropping existing columns before merge: {existing_cols_to_drop}")
        df_combined = df_combined.drop(columns=existing_cols_to_drop)

except FileNotFoundError:
    print(f"Error: Input file not found at '{input_csv_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the combined CSV: {e}")
    raise

# --- Load Source Dataframes (with correct derived columns) ---
print("\n--- Loading Source Databases with USPNet Data ---")
df_source_data_list = []
loaded_source = False

# Load Main DB with USPNet
try:
    df_main_source = pd.read_csv(main_db_with_uspnet_path, usecols=cols_to_extract, low_memory=False)
    print(f"Loaded main source DB '{main_db_with_uspnet_path}'. Shape: {df_main_source.shape}")
    # Check if all needed columns were loaded
    if all(col in df_main_source.columns for col in cols_to_extract):
        df_source_data_list.append(df_main_source)
        loaded_source = True
    else:
        print(f"Warning: Main source DB missing one or more required columns: {cols_to_extract}")
except FileNotFoundError:
    print(f"Warning: Main source DB file not found at '{main_db_with_uspnet_path}'.")
except Exception as e:
    print(f"Warning: Error loading main source DB: {e}")

# Load Filtered-Out DB with USPNet
try:
    df_filtered_source = pd.read_csv(filtered_db_with_uspnet_path, usecols=cols_to_extract, low_memory=False)
    print(f"Loaded filtered-out source DB '{filtered_db_with_uspnet_path}'. Shape: {df_filtered_source.shape}")
     # Check if all needed columns were loaded
    if all(col in df_filtered_source.columns for col in cols_to_extract):
        df_source_data_list.append(df_filtered_source)
        loaded_source = True
    else:
        print(f"Warning: Filtered-out source DB missing one or more required columns: {cols_to_extract}")
except FileNotFoundError:
    print(f"Warning: Filtered-out source DB file not found at '{filtered_db_with_uspnet_path}'.")
except Exception as e:
    print(f"Warning: Error loading filtered-out source DB: {e}")


# --- Combine and Merge Source Data ---
if not loaded_source:
    print("\nError: Could not load necessary source data with USPNet columns. Cannot proceed.")
    # Handle error - maybe raise exception
    raise RuntimeError("Failed to load source data with USPNet columns.")
else:
    print("\nCombining source data for merge...")
    df_source_combined = pd.concat(df_source_data_list, ignore_index=True)
    # Drop duplicates based on ProteinID, keeping the first instance found
    df_source_combined = df_source_combined.drop_duplicates(subset=[protein_id_col], keep='first')
    print(f"Combined source data shape (unique ProteinIDs): {df_source_combined.shape}")

    print("\nMerging missing columns into the main combined dataframe...")
    # Ensure ProteinID types match for merge
    df_combined[protein_id_col] = df_combined[protein_id_col].astype(str)
    df_source_combined[protein_id_col] = df_source_combined[protein_id_col].astype(str)

    # Perform left merge
    df_final = pd.merge(
        df_combined,
        df_source_combined,
        on=protein_id_col,
        how='left' # Keep all rows from df_combined
    )
    print("Merge complete.")

    # --- Final Checks and Type Conversions ---
    print("\nPerforming final checks and type conversions...")
    # Check if merge changed row count (shouldn't happen with left merge on unique IDs)
    if len(df_final) != len(df_combined):
        print(f"Warning: Merge changed row count from {len(df_combined)} to {len(df_final)}!")

    # Fill NaNs and set types for the newly merged columns
    df_final['Predicted_Subcellular_Localization'] = df_final['Predicted_Subcellular_Localization'].fillna('Unknown').astype(str)
    df_final['Mature_Protein_Sequence'] = df_final['Mature_Protein_Sequence'].fillna('').astype(str) # Fill with empty string? Or keep NaN?
    df_final['Mature_Seq_Length'] = pd.to_numeric(df_final['Mature_Seq_Length'], errors='coerce').fillna(0).astype(int)

    print("Final checks complete.")
    print(f"Columns in final dataframe: {df_final.columns.tolist()}")
    print(f"Non-null counts for added columns:")
    print(f"  Predicted_Subcellular_Localization: {df_final['Predicted_Subcellular_Localization'].notna().sum()}")
    print(f"  Mature_Protein_Sequence: {df_final['Mature_Protein_Sequence'].notna().sum()}")
    print(f"  Mature_Seq_Length: {df_final['Mature_Seq_Length'].notna().sum()}")


    # --- Save Final Data ---
    try:
        # Optional: Reorder columns one last time if needed
        # desired_final_order = [...]
        # df_final = df_final[desired_final_order]
        df_final.to_csv(output_csv_path, index=False)
        print(f"\nSuccessfully saved final combined and corrected data to '{output_csv_path}'. Shape: {df_final.shape}")
    except Exception as e:
        print(f"\nError saving final data: {e}")



In [ ]:
# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time

# --- Configuration ---
# Input file paths (adjust if necessary)
main_db_path = 'proteome_database_combined_v1.4.csv'
mgnify_results_path = 'results_vs_mgnify.m8' # Your MMseqs2 output file

# Output file path
output_db_path = 'proteome_database_combined_v1.5_mgnify.csv'

# Column names
protein_id_col = 'ProteinID' # Column in your main database
mgnify_query_col = 'query' # Column in the MMseqs2 .m8 file (first column)
new_hit_col = 'SeqSearch_MGnify_Hit' # Name of the new column to add

# --- Load Main Database ---
print(f"--- Loading Main Database ---")
start_time = time.time()
try:
    # Use low_memory=False for large files, consistent with your analysis notebook
    df_main = pd.read_csv(main_db_path, low_memory=False)
    print(f"Loaded '{main_db_path}'. Shape: {df_main.shape}")
    # Ensure ProteinID column is string type for reliable matching
    if protein_id_col in df_main.columns:
        df_main[protein_id_col] = df_main[protein_id_col].astype(str)
    else:
        raise KeyError(f"Required column '{protein_id_col}' not found in main database.")
except FileNotFoundError:
    print(f"Error: Main database file not found at '{main_db_path}'.")
    raise
except KeyError as e:
     print(f"Error loading main database: {e}")
     raise
except Exception as e:
    print(f"An error occurred while loading the main database CSV: {e}")
    raise
print(f"Main database loaded in {time.time() - start_time:.2f} seconds.")

# --- Load MGnify MMseqs2 Results ---
print(f"\n--- Loading MGnify Search Results ---")
start_time = time.time()
try:
    # Define column names for the .m8 file based on the format used in the search plan
    # We only really need the 'query' column
    m8_cols = ['query', 'target', 'pident', 'qcov', 'tcov', 'evalue']
    # Load only the query column to save memory, use tab separator
    df_mgnify_hits = pd.read_csv(
        mgnify_results_path,
        sep='\t',
        header=None, # .m8 files typically don't have headers
        names=m8_cols,
        usecols=[mgnify_query_col] # Only load the first column ('query')
    )
    print(f"Loaded '{mgnify_results_path}'. Found {len(df_mgnify_hits)} total hits.")
    # Ensure query column is string type
    df_mgnify_hits[mgnify_query_col] = df_mgnify_hits[mgnify_query_col].astype(str)

except FileNotFoundError:
    print(f"Error: MGnify results file not found at '{mgnify_results_path}'.")
    raise
except Exception as e:
    print(f"An error occurred while loading the MGnify results CSV: {e}")
    raise
print(f"MGnify results loaded in {time.time() - start_time:.2f} seconds.")

# --- Identify Proteins with Hits ---
print(f"\n--- Identifying Proteins with MGnify Hits ---")
# Get the set of unique ProteinIDs that had at least one hit in the MGnify search
proteins_with_mgnify_hits = set(df_mgnify_hits[mgnify_query_col].unique())
print(f"Found {len(proteins_with_mgnify_hits)} unique proteins with hits in MGnify.")

# --- Add New Column to Main Database ---
print(f"\n--- Adding '{new_hit_col}' column ---")
# Initialize the new column to False
df_main[new_hit_col] = False

# Find the rows in the main dataframe where the ProteinID is in our set of hits
hit_mask = df_main[protein_id_col].isin(proteins_with_mgnify_hits)

# Set the new column to True for those rows
df_main.loc[hit_mask, new_hit_col] = True

# --- Verify ---
print(f"\nValue counts for the new '{new_hit_col}' column:")
print(df_main[new_hit_col].value_counts())
num_true = df_main[new_hit_col].sum()
if num_true == len(proteins_with_mgnify_hits):
    print("Verification successful: Number of True values matches number of unique hitting proteins.")
else:
    print(f"Warning: Mismatch detected! Number of True values ({num_true}) does not match unique hitting proteins ({len(proteins_with_mgnify_hits)}). Check ProteinID matching.")

# --- Save Updated Database ---
print(f"\n--- Saving Updated Database ---")
start_time = time.time()
try:
    df_main.to_csv(output_db_path, index=False)
    print(f"Successfully saved updated database with MGnify hits to '{output_db_path}'.")
    print(f"Final shape: {df_main.shape}")
except Exception as e:
    print(f"Error saving updated database: {e}")
    raise
print(f"Database saved in {time.time() - start_time:.2f} seconds.")

print("\n--- Integration Complete ---")


In [ ]:
# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time

# --- Configuration ---
# Input file paths (adjust if necessary)
# Use the output from the MGnify integration step or v1.4 if starting fresh
main_db_path = 'proteome_database_combined_v1.5_mgnify.csv' # Or v1.4
mgnify_results_path = 'results_vs_mgnify.m8' # Your MMseqs2 output file

# *** USER ACTION NEEDED: Path to your downloaded AFESM metadata file ***
afesm_metadata_path = '2-repID_isOnlyESM_nMem_nAllMem_repPlddt_avgPlddt_avgAllPlddt_repLen_avgLen_avgAllLen_LCAtaxID_nBiome_LCBID.tsv' # <-- UPDATE THIS PATH

# Output file path
output_db_path = 'proteome_database_combined_v1.6_esm_dark.csv'

# Column names
protein_id_col = 'ProteinID' # Column in your main database
mgnify_query_col = 'query' # Column in the MMseqs2 .m8 file (first column)
mgnify_target_col = 'target' # Column in the MMseqs2 .m8 file (second column)
afesm_repid_col = 'repID' # Column name in the AFESM metadata file

# Hit columns (ensure these exist in your input main_db_path)
pdb_hit_col = 'SeqSearch_PDB_Hit'
afdb_hit_col = 'SeqSearch_AFDB_Hit'
mgnify_hit_col = 'SeqSearch_MGnify_Hit' # Added previously
esma_hit_col = 'SeqSearch_ESMA_Hit' # New column for ESMA hits via MGnify/AFESM
dark_col = 'Is_Structurally_Dark' # New triple-negative flag

# --- Load Main Database ---
print(f"--- Loading Main Database ---")
start_time = time.time()
try:
    df_main = pd.read_csv(main_db_path, low_memory=False)
    print(f"Loaded '{main_db_path}'. Shape: {df_main.shape}")
    # Ensure ProteinID column is string type for reliable matching
    if protein_id_col in df_main.columns:
        df_main[protein_id_col] = df_main[protein_id_col].astype(str)
    else:
        raise KeyError(f"Required column '{protein_id_col}' not found in main database.")
    # Ensure previous hit columns exist and are boolean
    for col in [pdb_hit_col, afdb_hit_col, mgnify_hit_col]:
        if col not in df_main.columns:
             print(f"Warning: Column '{col}' not found. Adding as False.")
             df_main[col] = False
        else:
             # Fill potential NAs from previous steps and ensure boolean
             df_main[col] = df_main[col].fillna(False).astype(bool)

except FileNotFoundError:
    print(f"Error: Main database file not found at '{main_db_path}'.")
    raise
except KeyError as e:
     print(f"Error loading main database: {e}")
     raise
except Exception as e:
    print(f"An error occurred while loading the main database CSV: {e}")
    raise
print(f"Main database loaded in {time.time() - start_time:.2f} seconds.")

# --- Load AFESM Metadata (Representative IDs with Structures) ---
print(f"\n--- Loading AFESM Metadata ---")
start_time = time.time()
afesm_repids_with_structure = set()
try:
    # Read the gzipped TSV file, using only the first column ('repID')
    # Assuming the first column is indeed 'repID' based on the file name and description
    df_afesm_meta = pd.read_csv(
        afesm_metadata_path,
        sep='\t',
        usecols=[0], # Load only the first column
        header=0, # Use the first row as header to get the column name
    )
    # Check the actual column name loaded (should be 'repID')
    loaded_col_name = df_afesm_meta.columns[0]
    if loaded_col_name != afesm_repid_col:
        print(f"Warning: Expected column name '{afesm_repid_col}' but found '{loaded_col_name}'. Using '{loaded_col_name}'.")
        afesm_repid_col = loaded_col_name # Use the actual loaded name

    # Extract unique IDs into the set, ensuring they are strings
    afesm_repids_with_structure = set(df_afesm_meta[afesm_repid_col].dropna().astype(str))

    if not afesm_repids_with_structure:
         print(f"Warning: No IDs loaded from '{afesm_metadata_path}'. Check file content and path.")
    else:
         print(f"Loaded {len(afesm_repids_with_structure)} AFESM representative IDs with structures from '{afesm_metadata_path}'.")

except FileNotFoundError:
    print(f"Error: AFESM metadata file not found at '{afesm_metadata_path}'. Cannot determine ESMA hits.")
    print("Proceeding without ESMA hit information. The 'SeqSearch_ESMA_Hit' and 'Is_Structurally_Dark' columns will reflect this.")
    afesm_repids_with_structure = set() # Ensure empty set
except Exception as e:
    print(f"An error occurred while loading the AFESM metadata: {e}")
    afesm_repids_with_structure = set() # Ensure empty set
print(f"AFESM metadata loaded in {time.time() - start_time:.2f} seconds.")


# --- Load MGnify MMseqs2 Results (Query and Target) ---
# Only proceed if we have AFESM IDs to check against
proteins_hitting_esma_target = set()
if afesm_repids_with_structure:
    print(f"\n--- Loading MGnify Search Results (Query & Target) ---")
    start_time = time.time()
    try:
        # Define column names for the .m8 file
        m8_cols = ['query', 'target', 'pident', 'qcov', 'tcov', 'evalue']
        # Load only the query and target columns
        df_mgnify_hits = pd.read_csv(
            mgnify_results_path,
            sep='\t',
            header=None,
            names=m8_cols,
            usecols=[mgnify_query_col, mgnify_target_col] # Load query and target
        )
        print(f"Loaded '{mgnify_results_path}'. Found {len(df_mgnify_hits)} total hits.")
        # Ensure columns are string type
        df_mgnify_hits[mgnify_query_col] = df_mgnify_hits[mgnify_query_col].astype(str)
        df_mgnify_hits[mgnify_target_col] = df_mgnify_hits[mgnify_target_col].astype(str)

        # --- Identify Your Proteins Hitting MGnify Targets that ARE AFESM Representatives ---
        print(f"\n--- Filtering MGnify hits for targets present in AFESM metadata ---")
        # Filter the hits dataframe where the target is in our set of AFESM repIDs
        df_esma_relevant_hits = df_mgnify_hits[df_mgnify_hits[mgnify_target_col].isin(afesm_repids_with_structure)]
        print(f"Found {len(df_esma_relevant_hits)} hits where the MGnify target is an AFESM representative with a structure.")

        # Get the set of unique ProteinIDs (query) from these relevant hits
        proteins_hitting_esma_target = set(df_esma_relevant_hits[mgnify_query_col].unique())
        print(f"Found {len(proteins_hitting_esma_target)} unique query proteins hitting an AFESM representative target.")

    except FileNotFoundError:
        print(f"Error: MGnify results file not found at '{mgnify_results_path}'. Cannot determine ESMA hits.")
        proteins_hitting_esma_target = set() # Ensure empty set
    except Exception as e:
        print(f"An error occurred while loading or processing the MGnify results CSV: {e}")
        proteins_hitting_esma_target = set() # Ensure empty set
    print(f"MGnify results loaded and processed in {time.time() - start_time:.2f} seconds.")
else:
     print("\nSkipping MGnify results loading/processing as no AFESM metadata was loaded.")


# --- Add New ESMA Hit Column to Main Database ---
print(f"\n--- Adding '{esma_hit_col}' column ---")
# Initialize the new column to False
df_main[esma_hit_col] = False
# Find rows where the ProteinID is in our set of proteins hitting an AFESM target
if proteins_hitting_esma_target: # Only update if we have hits
    esma_hit_mask = df_main[protein_id_col].isin(proteins_hitting_esma_target)
    # Set the new column to True for those rows
    df_main.loc[esma_hit_mask, esma_hit_col] = True

print(f"\nValue counts for the new '{esma_hit_col}' column:")
print(df_main[esma_hit_col].value_counts())
num_esma_true = df_main[esma_hit_col].sum()
if not afesm_repids_with_structure:
     print("(Note: ESMA hits could not be determined due to missing AFESM metadata.)")
elif num_esma_true == len(proteins_hitting_esma_target):
    print("Verification successful: Number of True values matches number of unique hitting proteins.")
else:
    print(f"Warning: Mismatch detected! Number of True values ({num_esma_true}) does not match unique hitting proteins ({len(proteins_hitting_esma_target)}). Check ProteinID matching.")


# --- Add 'Is_Structurally_Dark' Column ---
print(f"\n--- Adding '{dark_col}' column ---")
# True if PDB hit is False AND AFDB hit is False AND ESMA hit is False
df_main[dark_col] = (
    (~df_main[pdb_hit_col]) &
    (~df_main[afdb_hit_col]) &
    (~df_main[esma_hit_col])
)

print(f"\nValue counts for the new '{dark_col}' column:")
print(df_main[dark_col].value_counts())
print(f"Identified {df_main[dark_col].sum()} 'structurally dark' proteins (Triple Negative).")


# --- Save Updated Database ---
print(f"\n--- Saving Updated Database ---")
start_time = time.time()
try:
    # Reorder columns slightly? Optional. Put new columns near other hit columns.
    cols = df_main.columns.tolist()
    # Find insertion point (e.g., after mgnify_hit_col)
    try:
        insert_idx = cols.index(mgnify_hit_col) + 1
        # Ensure new columns actually exist before trying to reorder
        if esma_hit_col in cols:
             cols.insert(insert_idx, cols.pop(cols.index(esma_hit_col)))
             insert_idx += 1 # Increment index for the next column
        if dark_col in cols:
             cols.insert(insert_idx, cols.pop(cols.index(dark_col)))
        df_main = df_main[cols]
    except ValueError:
         print("Could not reorder columns, saving with new columns at the end.")

    df_main.to_csv(output_db_path, index=False)
    print(f"Successfully saved updated database to '{output_db_path}'.")
    print(f"Final shape: {df_main.shape}")
except Exception as e:
    print(f"Error saving updated database: {e}")
    raise
print(f"Database saved in {time.time() - start_time:.2f} seconds.")

print("\n--- Integration Complete ---")


In [ ]:
# --- Standard Library Imports ---
import pandas as pd
import numpy as np
import os
import time
from collections import Counter
import re # For keyword searching

# --- Configuration ---
# Assumes setup cell (jupyter_arcadia_analysis_setup) ran for plot settings
# Required variables from setup: summary_data_dir

# *** ADDED: Define the summary data directory if not defined earlier ***
if 'summary_data_dir' not in locals():
    summary_data_dir = 'output_summary_data'
    print(f"Defined summary_data_dir = '{summary_data_dir}'")
    # Ensure the directory exists
    if not os.path.exists(summary_data_dir):
        try:
            os.makedirs(summary_data_dir)
            print(f"Created directory: {summary_data_dir}")
        except OSError as e:
            print(f"Warning: Could not create summary data directory '{summary_data_dir}'. Error: {e}")
            summary_data_dir = None # Disable saving if creation fails

# Input file path for the LATEST combined database
# This will be BOTH input and the base for the output
combined_db_path = 'proteome_database_combined_v1.6_esm_dark.csv' # <-- UPDATE if needed
# Path to InterPro entry list file (ensure this is correct)
interpro_entry_path = 'interpro_entry.txt' # Adjust path if needed

# *** MODIFIED: Output file path is now for the UPDATED main database ***
if summary_data_dir:
     # Save updated DB in the main directory, not summary_data_dir? Or keep consistent?
     # Let's save it in the main directory for now.
     output_db_with_cat_path = 'proteome_database_combined_v1.7_broad_cat.csv'
else:
    output_db_with_cat_path = 'proteome_database_combined_v1.7_broad_cat.csv' # Save locally if dir failed
    print(f"Warning: Saving updated database locally as '{output_db_with_cat_path}'")


# Columns from main db
protein_id_col = 'ProteinID'
og_col = 'Orthogroup'
source_annot_col = 'Source_Protein_Annotation'
ipr_col = 'IPR_Signatures'
# Optional: Original specific category column for fallback
func_cat_col = 'Specific_Functional_Category'
# Name for the new column
broad_cat_col = 'Broad_Functional_Category'

# --- Define Broad Categories and Associated Keywords/Patterns ---
# (Using the same definitions as v5)
CATEGORY_DEFINITIONS = [
    ('ESCRT/Endosomal Sorting', [
        'escrt', 'vps\d*', 'snf7', 'vps2[\\b_]', 'vps4[\\b_]', 'vps20', 'vps22', 'vps24', 'vps25', 'vps28', 'vps32', 'vps36', 'vps60',
        'retromer', 'sortilin', 'vps5', 'vps10', 'vps17', 'vps26', 'vps29', 'vps35', # Retromer
        'corvet', 'hops', 'vps11', 'vps16', 'vps18', 'vps33', 'vps39', 'vps41', # HOPS/CORVET
        'garp', 'vps51', 'vps52', 'vps53', 'vps54', # GARP
        'steadinessbox', 'glue domain', 'ist1', 'brot1', 'mvb12', # Other ESCRT/Sorting related
    ]),
    ('Cytoskeleton', [
        'actin', 'arp2/3', 'profilin', 'cofilin', 'formin', 'gelsolin', # Actin
        'tubulin', 'kinesin', 'dynein', # Tubulin
        'septin', # Other cytoskeleton
        'lokiactin', # Specific Asgard term
    ]),
     ('Membrane Trafficking/Vesicles', [
        'dynamin', # Membrane remodeling
        'rab', 'rho', 'rac', 'ran', 'cdc42', 'arl', 'arf', 'yipt', # Small GTPases
        'sec\d*', 'sec59', 'sec61', 'sec65', 'signal recognition particle', 'srp', # Secretion/Translocation (Sec)
        'coatomer', 'copi', 'copii', 'clathrin', # Vesicular transport coats
        'adaptin', 'ap complex', 'adaptor protein complex', # Adaptor complexes
        'snare', # SNAREs
        'longin', 'mon1', 'roadblock', 'lc7', 'hook domain', 'arftrapp', # Other trafficking
        'tralphin', # Specific Asgard term? (Check spelling/context)
    ]),
    ('Ubiquitin System', [
        'ubiquitin', 'sumo', 'nedd8', 'ubl', 'ubiquitin-like', # Ubiquitin and UBLs
        'proteasome', 'proteasomal', # Proteasome
        'ubiquitinating', 'ubiquitin conjugating', 'ubiquitin ligase', 'ubiquitin activating', # Enzymes E1/E2/E3
        'ubiquitin domain', 'ubx domain', # Domains
    ]),
    ('N-glycosylation', [
        'glycosylation', 'ost\d*', 'stt3', 'wbp1', 'ddost', 'ribophorin', # OST complex
        'trap alpha', 'trap beta', 'trap gamma', 'trap delta', # TRAP complex (often associated)
    ]),
    ('Nuclear Transport/Pore', [
        'nuclear pore', 'nucleoporin', 'nup\d*', 'importin', 'exportin', 'karyopherin', 'ranbp', # Nuclear transport
    ]),
    ('DNA Info Processing', [
        'dna polymerase', 'dna replication', 'dna repair', 'recombination', # DNA processing
        'topoisomerase', 'helicase', 'primase', 'ligase', # Core DNA enzymes
        'histone', 'nucleosome', # Chromatin
        'e2f', 'dp transcription factor', # Specific factors if known ESPs
        'mcm\d*', 'orc\d*', 'cdc\d*', # Cell cycle related DNA factors
    ]),
    ('RNA Info Processing', [
        'rna polymerase', 'tbp', 'tata-binding', 'tfii[a-z]', 'transcription factor', # Transcription machinery
        'spliceosome', 'snrna', 'snrnp', 'splicing factor', # Splicing
        'rna processing', 'rna modification', 'rrp\d*', 'exosome', # RNA processing/turnover
    ]),
    ('Translation', [
        'ribosome', 'ribosomal protein', 'rrna', # Ribosome components
        'eif[0-9]', 'eef[0-9]', 'translation initiation', 'translation elongation', 'translation termination', # Translation factors
        'trna synthetase', 'aminoacyl-trna', # tRNA related
        'pelota', # Translation related
    ]),
     ('Signal Transduction', [
         'kinase', 'phosphatase', 'receptor', 'gtpase', 'signal transduction',
     ]),
     ('Metabolism', [
         'metabolism', 'synthase', 'transferase', 'oxidoreductase', 'hydrolase',
     ]),
]

# Compile regex patterns for faster matching
CATEGORY_PATTERNS = []
for category, keywords in CATEGORY_DEFINITIONS:
    patterns = [re.compile(r'\b' + kw + r'\b', re.IGNORECASE) for kw in keywords]
    CATEGORY_PATTERNS.append({'category': category, 'patterns': patterns})

# Define generic patterns/terms to exclude from source/IPR names during fallback
generic_source_patterns = [
    re.compile(r'hypothetical', re.IGNORECASE), re.compile(r'uncharacterized', re.IGNORECASE),
    re.compile(r'unknown function', re.IGNORECASE), re.compile(r'predicted protein', re.IGNORECASE),
    re.compile(r'^na$', re.IGNORECASE), re.compile(r'^protein$', re.IGNORECASE),
    re.compile(r'domain-containing protein', re.IGNORECASE), re.compile(r'conserved protein', re.IGNORECASE),
]
generic_ipr_types = {'Domain', 'Family', 'Repeat', 'Conserved_site', 'PTM'}
generic_ipr_names = {'Domain of unknown function', 'Uncharacterised protein family'}


# --- Load InterPro Entry Data ---
print(f"\n--- Loading InterPro Entry Data ---")
ipr_lookup = {}
start_time = time.time()
try:
    ipr_info = pd.read_csv(
        interpro_entry_path, sep='\t', usecols=[0, 1, 2],
        names=['IPR_ID', 'Type', 'Name'], header=None, comment='#', on_bad_lines='warn'
    )
    ipr_info['IPR_ID'] = ipr_info['IPR_ID'].astype(str).str.strip()
    ipr_lookup = ipr_info.set_index('IPR_ID')[['Type', 'Name']].to_dict('index')
    print(f"Loaded InterPro entry data for {len(ipr_lookup)} entries in {time.time() - start_time:.2f} seconds.")
except FileNotFoundError:
    print(f"Warning: InterPro entry file not found at '{interpro_entry_path}'. Cannot use IPR names for categorization.")
except Exception as e:
    print(f"Warning: An error occurred loading or processing '{interpro_entry_path}': {e}")
if 'ipr_lookup' not in globals(): ipr_lookup = {}


# --- Load Main Database ---
print(f"\n--- Loading Main Database ---")
start_time = time.time()
try:
    # Load ALL columns this time, as we will save the whole thing
    df_main = pd.read_csv(combined_db_path, low_memory=False)
    print(f"Loaded '{combined_db_path}'. Shape: {df_main.shape}")

    # --- Data Cleaning and Type Checks ---
    # Ensure OG column is string for matching
    if og_col not in df_main.columns: raise KeyError(f"'{og_col}' missing from main DB.")
    df_main[og_col] = df_main[og_col].astype(str)

    # Ensure annotation columns exist and fill NaNs
    if source_annot_col not in df_main.columns:
        print(f"Warning: '{source_annot_col}' not found, adding empty column.")
        df_main[source_annot_col] = ''
    else:
        df_main[source_annot_col] = df_main[source_annot_col].fillna('')

    if ipr_col not in df_main.columns:
        print(f"Warning: '{ipr_col}' not found, adding empty column.")
        df_main[ipr_col] = ''
    else:
        df_main[ipr_col] = df_main[ipr_col].fillna('')

    if func_cat_col not in df_main.columns:
        print(f"Warning: '{func_cat_col}' not found, adding 'Unknown/Unclassified'.")
        df_main[func_cat_col] = 'Unknown/Unclassified'
    else:
        df_main[func_cat_col] = df_main[func_cat_col].fillna('Unknown/Unclassified')

except FileNotFoundError:
    print(f"Error: Combined database file not found at '{combined_db_path}'. Cannot map functions.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main database CSV: {e}")
    raise
print(f"Main DB loaded in {time.time() - start_time:.2f} seconds.")


# --- Function to Assign Broad Functional Category ---
# (Using the same hierarchical function 'assign_broad_category' from v5)
def assign_broad_category(series_group):
    """Assigns a broad functional category based on hierarchical rules."""
    # --- 1. Check Source Annotations against Keyword Patterns (Priority Order) ---
    source_annots = series_group[source_annot_col].dropna().astype(str)
    if not source_annots.empty:
        for category_info in CATEGORY_PATTERNS:
            category_name = category_info['category']
            patterns = category_info['patterns']
            for annot in source_annots:
                for pattern in patterns:
                    if pattern.search(annot):
                        return category_name # Return first category match based on priority

    # --- 2. Check Informative IPR Signatures ---
    if ipr_col in series_group.columns and ipr_lookup:
        ipr_lists = series_group.dropna(subset=[ipr_col])[ipr_col].astype(str).str.split('|')
        all_ipr_ids = [ipr.strip() for sublist in ipr_lists for ipr in sublist if ipr.strip()]
        if all_ipr_ids:
            ipr_counts = Counter(all_ipr_ids)
            # Check most common IPRs against category keywords/patterns
            for ipr_id, count in ipr_counts.most_common():
                ipr_info = ipr_lookup.get(ipr_id)
                if ipr_info:
                    ipr_name = ipr_info.get('Name', '')
                    # Check IPR name against category patterns
                    for category_info in CATEGORY_PATTERNS:
                         category_name = category_info['category']
                         patterns = category_info['patterns']
                         for pattern in patterns:
                              if pattern.search(ipr_name):
                                   return category_name # Return category if IPR name matches
                    # If no keyword match, check if IPR itself is informative (but not keyworded)
                    ipr_type = ipr_info.get('Type', '')
                    if ipr_name and ipr_type not in generic_ipr_types and ipr_name not in generic_ipr_names:
                         pass # Fall through

    # --- 3. Check Other Specific Source Annotations (Fallback) ---
    if not source_annots.empty:
        specific_annots = []
        processed_annots = set()
        for annot in source_annots:
             annot_clean = annot.strip()
             if annot_clean and annot_clean not in processed_annots and not any(p.search(annot_clean) for p in generic_source_patterns):
                  specific_annots.append(annot_clean)
                  processed_annots.add(annot_clean)
        if specific_annots:
             annot_counts = Counter(specific_annots)
             most_common_annots = annot_counts.most_common()
             return "Other Specific Annotation" # Placeholder category

    # --- 4. Fallback Original Specific_Functional_Category ---
    if func_cat_col in series_group.columns:
        func_cats = series_group[func_cat_col].dropna()
        filtered_cats = func_cats[~func_cats.isin(['Unknown/Unclassified', 'Domain', 'Family', 'Repeat', 'Conserved_site', 'PTM'])]
        if not filtered_cats.empty:
            mode_val = filtered_cats.mode()
            if not mode_val.empty:
                 return sorted(mode_val.tolist())[0]

    # --- 5. Lowest Priority: Unknown ---
    return 'Unknown/Unclassified'


# --- Apply Categorization Function ---
print("\nApplying hierarchical categorization function to all OGs...")
start_time = time.time()
# Create the mapping DataFrame (OG -> Broad_Functional_Category)
og_broad_categories_map = df_main.groupby(og_col).apply(assign_broad_category).reset_index()
og_broad_categories_map.columns = [og_col, broad_cat_col] # Rename columns
print(f"Categorization mapping complete in {time.time() - start_time:.2f} seconds.")

# --- Merge Category Mapping into Main DataFrame ---
print(f"\nMerging '{broad_cat_col}' into the main database...")
start_merge_time = time.time()
# Ensure OG columns have same type before merge
df_main[og_col] = df_main[og_col].astype(str)
og_broad_categories_map[og_col] = og_broad_categories_map[og_col].astype(str)

# Remove the new column if it already exists from a previous run
if broad_cat_col in df_main.columns:
    print(f"Column '{broad_cat_col}' already exists. Dropping before merge.")
    df_main = df_main.drop(columns=[broad_cat_col])

df_main_updated = pd.merge(
    df_main,
    og_broad_categories_map,
    on=og_col,
    how='left' # Keep all proteins, even if OG somehow missing from map
)
# Fill any NaNs created by the merge (shouldn't happen if map is complete)
df_main_updated[broad_cat_col] = df_main_updated[broad_cat_col].fillna('Unknown/Unclassified')
print(f"Merge complete in {time.time() - start_merge_time:.2f} seconds.")
print(f"Shape after merge: {df_main_updated.shape}")


# --- Display Results ---
print(f"\nValue Counts for new '{broad_cat_col}' column:")
print(df_main_updated[broad_cat_col].value_counts().to_markdown())

# --- Save the Updated Main Database ---
print(f"\n--- Saving Updated Main Database ---")
start_save_time = time.time()
# Check if output path is defined (depends on summary_data_dir)
if output_db_with_cat_path:
    try:
        # Optional: Reorder columns to place new category near other functional columns
        cols = df_main_updated.columns.tolist()
        try:
            # Find insertion point (e.g., after Specific_Functional_Category)
            insert_idx = cols.index(func_cat_col) + 1
            cols.insert(insert_idx, cols.pop(cols.index(broad_cat_col)))
            df_main_updated = df_main_updated[cols]
        except ValueError:
             print("Could not reorder columns, saving with new column at the end.")

        df_main_updated.to_csv(output_db_with_cat_path, index=False)
        print(f"\nSuccessfully saved updated database with '{broad_cat_col}' to '{output_db_with_cat_path}'")
    except Exception as e: print(f"Error saving updated database: {e}")
else:
    print("\nSkipping save of updated database (output path not defined).")
print(f"Database saved in {time.time() - start_save_time:.2f} seconds.")


print("\n--- Broad Functional Categorization and DB Update Complete (v5.1) ---")


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import glob # For finding files in a folder
import time
import re # Added for parsing organism names

# --- Configuration ---
# Main database file
db_path = 'proteome_database_combined_v1.7_broad_cat.csv'
# Folder containing DIAMOND search results files (tab-separated)
diamond_results_folder = 'euk_diamond_search_results' # User provided this folder name
# ESP Orthogroup list file
esp_og_list_path = 'output_summary_data/all_esp_orthogroup_list_v4.txt'

# Output file for the database with DIAMOND hits and ESP column integrated
output_db_updated_path = 'proteome_database_combined_v1.8_euk_hits_esps.csv' # Example new version

# DIAMOND parsing parameters
e_value_threshold = 1e-10 # Adjust as needed, 1e-5 is also common
# DIAMOND output columns (standard 12 fields for `outfmt 6`):
# qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore
# User example also had qlen and slen at the end, so we'll use 14 columns
diamond_col_names = [
    'qseqid', 'sseqid_full', 'pident', 'length', 'mismatch', 'gapopen',
    'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'qlen', 'slen'
]

# Columns to be added to the main DataFrame
hit_flag_col = 'Has_Euk_DIAMOND_Hit'
best_hit_sseqid_col = 'Euk_Hit_SSEQID'
best_hit_organism_col = 'Euk_Hit_Organism'
best_hit_pident_col = 'Euk_Hit_PIDENT'
best_hit_evalue_col = 'Euk_Hit_EVALUE'
esp_col = 'Is_ESP' # Name of the new ESP column we will create

# Define relevant column names from your main database
protein_id_col = 'ProteinID'
group_col = 'Group' # 'Asgard' or 'Virus'
orthogroup_col = 'Orthogroup' # Needed for ESP identification
structurally_dark_col = 'Is_Structurally_Dark'
num_domains_col = 'Num_Domains'


print("--- Starting DIAMOND Hit Integration, ESP Definition, and Analysis ---")

# --- 1. Load Main Database ---
print(f"\nLoading main database from: {db_path}")
try:
    df_main = pd.read_csv(db_path, low_memory=False)
    print(f"Successfully loaded main database. Shape: {df_main.shape}")
    # Diagnostic: Print some ProteinIDs from Asgard group in df_main
    if protein_id_col in df_main.columns and group_col in df_main.columns:
        print("  Example Asgard ProteinIDs from main DB (first 5):")
        print(df_main[df_main[group_col] == 'Asgard'][protein_id_col].head().tolist())
except FileNotFoundError:
    print(f"ERROR: Main database file not found at '{db_path}'. Please check the path.")
    raise
except Exception as e:
    print(f"An error occurred while loading the main database: {e}")
    raise

# --- 2. Define ESPs based on Orthogroup List ---
print(f"\nLoading ESP Orthogroup list from: {esp_og_list_path}")
esp_og_set = set()
try:
    esp_og_df = pd.read_csv(esp_og_list_path, header=None, names=[orthogroup_col])
    esp_og_set = set(esp_og_df[orthogroup_col])
    print(f"Successfully loaded {len(esp_og_set)} ESP Orthogroups.")
except FileNotFoundError:
    print(f"ERROR: ESP Orthogroup list file not found at '{esp_og_list_path}'. '{esp_col}' will be False for all.")
except Exception as e:
    print(f"An error occurred while loading the ESP Orthogroup list: {e}. '{esp_col}' will be False for all.")

# Add 'Is_ESP' column to the main DataFrame
if orthogroup_col in df_main.columns and group_col in df_main.columns and esp_og_set:
    df_main[esp_col] = df_main.apply(
        lambda row: row[orthogroup_col] in esp_og_set if row[group_col] == 'Asgard' else False,
        axis=1
    )
    print(f"Added '{esp_col}' column. Found {df_main[esp_col].sum()} ESPs in Asgard.")
else:
    df_main[esp_col] = False # Default to False if OGs not loaded or columns missing
    if not esp_og_set:
        print(f"ESP Orthogroup list was not loaded or is empty. '{esp_col}' column set to False for all proteins.")
    else:
        print(f"Could not define ESPs due to missing '{orthogroup_col}' or '{group_col}' columns. '{esp_col}' column set to False.")


# --- 3. Parse DIAMOND Results from Folder ---
print(f"\nParsing DIAMOND results from folder: {diamond_results_folder}")
best_hits_data = {} # To store best hit info for each query protein
raw_diamond_files = glob.glob(os.path.join(diamond_results_folder, '*_diamond_hits.tsv'))

if not raw_diamond_files:
    print(f"WARNING: No '*_diamond_hits.tsv' files found in '{diamond_results_folder}'.")
    df_diamond_raw_list = []
else:
    print(f"Found {len(raw_diamond_files)} DIAMOND result files to process.")
    df_diamond_raw_list = []
    for f_path in raw_diamond_files:
        try:
            df_temp = pd.read_csv(f_path, sep='\t', header=None, names=diamond_col_names)
            # CRITICAL CHANGE: Do NOT add .ASG suffix here.
            # The qseqid from DIAMOND output should match ProteinID in df_main directly.
            # df_temp['qseqid'] = df_temp['qseqid'].astype(str) + ".ASG" # REMOVED THIS LINE
            df_temp['qseqid'] = df_temp['qseqid'].astype(str) # Ensure it's string
            df_diamond_raw_list.append(df_temp)
        except Exception as e:
            print(f"  Error reading or processing file {f_path}: {e}")
            continue # Skip problematic files

if df_diamond_raw_list:
    df_diamond_raw = pd.concat(df_diamond_raw_list, ignore_index=True)
    print(f"  Read {len(df_diamond_raw)} total DIAMOND alignments from all files.")

    # Filter by e-value
    df_diamond_filtered = df_diamond_raw[df_diamond_raw['evalue'] <= e_value_threshold].copy()
    print(f"  Found {len(df_diamond_filtered)} alignments passing e-value threshold <= {e_value_threshold}.")

    if not df_diamond_filtered.empty:
        # Sort by qseqid and then by e-value (ascending), then bitscore (descending) to easily get the best hit
        df_diamond_filtered.sort_values(by=['qseqid', 'evalue', 'bitscore'], ascending=[True, True, False], inplace=True)
        
        # Keep only the best hit per qseqid
        df_best_hits = df_diamond_filtered.drop_duplicates(subset=['qseqid'], keep='first').copy()
        print(f"  Identified {len(df_best_hits)} unique query proteins with significant eukaryotic hits.")
        
        # Diagnostic: Print some qseqids from DIAMOND best hits
        print("  Example qseqids from DIAMOND best hits (first 5):")
        print(df_best_hits['qseqid'].head().tolist())


        # Prepare data for merging
        for _, row in df_best_hits.iterrows():
            query_id_for_db = row['qseqid'] # This is now the original ID from DIAMOND file
            subject_full = str(row['sseqid_full'])
            sseqid = subject_full
            organism = "Unknown" # Default
            
            if '|' in subject_full: # Standard parsing for NCBI-like headers
                parts = subject_full.split('|', 1)
                sseqid = parts[0]
                if len(parts) > 1 and parts[1]: # Check if there's anything after the first pipe
                    desc_text = parts[1]
                    # Try to extract organism from [Organism Name]
                    match = re.search(r"\[(.*?)\]", desc_text)
                    if match:
                        organism = match.group(1)
                    else:
                        # If no brackets, try to get it from common patterns like XP_id|Genus species
                        # This part might need refinement based on the exact format of sseqid_full
                        sub_parts = desc_text.split(' ')
                        if len(sub_parts) > 0:
                             # Check if the first part after pipe is just an accession (e.g. XP_12345)
                            if not (sub_parts[0].startswith("XP_") or sub_parts[0].startswith("NP_") or sub_parts[0].startswith("WP_")):
                                organism = sub_parts[0] # Often the first word is genus
                                if len(sub_parts) > 1 and not (sub_parts[1].startswith("XP_") or sub_parts[1].startswith("NP_") or sub_parts[1].startswith("WP_")):
                                     organism = f"{sub_parts[0]} {sub_parts[1]}" # Genus species
                            elif len(sub_parts) > 1: # If first part is an ID, try the next
                                organism = sub_parts[1]


            best_hits_data[query_id_for_db] = {
                hit_flag_col: True,
                best_hit_sseqid_col: sseqid,
                best_hit_organism_col: organism,
                best_hit_pident_col: row['pident'],
                best_hit_evalue_col: row['evalue']
            }
    else:
        print("  No DIAMOND hits passed the e-value threshold after processing all files.")
else:
    print("  No valid DIAMOND alignments loaded. Skipping hit processing.")


# --- 4. Merge DIAMOND Hit Information with Main Database ---
print(f"\nMerging DIAMOND hit information into the main database...")
df_main_updated = df_main.copy() # Start with the df_main that now includes Is_ESP

if best_hits_data:
    df_hits_to_merge = pd.DataFrame.from_dict(best_hits_data, orient='index')
    df_hits_to_merge.index.name = protein_id_col 
    
    # Diagnostic: Check dtypes before merge
    print(f"  dtype of df_main_updated['{protein_id_col}']: {df_main_updated[protein_id_col].dtype}")
    print(f"  dtype of df_hits_to_merge.index: {df_hits_to_merge.index.dtype}")
    
    # Ensure ProteinID is string in both for robust merging
    df_main_updated[protein_id_col] = df_main_updated[protein_id_col].astype(str)
    df_hits_to_merge.index = df_hits_to_merge.index.astype(str)

    df_main_updated = df_main_updated.merge(df_hits_to_merge, on=protein_id_col, how='left')
    
    df_main_updated[hit_flag_col] = df_main_updated[hit_flag_col].fillna(False)
else:
    print("  No best hits data to merge. Adding empty hit flag column.")
    df_main_updated[hit_flag_col] = False
    for col in [best_hit_sseqid_col, best_hit_organism_col, best_hit_pident_col, best_hit_evalue_col]:
        if col not in df_main_updated.columns: 
            df_main_updated[col] = np.nan


print(f"  Merge complete. Updated database shape: {df_main_updated.shape}")
print(f"  Total proteins flagged with eukaryotic hits: {df_main_updated[hit_flag_col].sum()}")

# --- 5. Save Updated Database ---
print(f"\nSaving updated database (with ESPs and DIAMOND hits) to: {output_db_updated_path}")
try:
    df_main_updated.to_csv(output_db_updated_path, index=False)
    print(f"Successfully saved updated database.")
except Exception as e:
    print(f"ERROR: Could not save the updated database. Error: {e}")

# --- 6. Initial Summary Analyses ---
print("\n\n--- Initial Summary of Eukaryotic DIAMOND Hits ---")

if group_col not in df_main_updated.columns:
    print(f"ERROR: Group column '{group_col}' not found. Cannot perform group-specific summary.")
else:
    # Overall counts
    total_proteins = len(df_main_updated)
    total_hits = df_main_updated[hit_flag_col].sum()
    print(f"\nOverall:")
    print(f"  Total proteins in database: {total_proteins}")
    print(f"  Total proteins with eukaryotic DIAMOND hits: {total_hits} ({(total_hits/total_proteins*100 if total_proteins > 0 else 0.0):.1f}%)")

    # Asgard proteins
    df_asgard = df_main_updated[df_main_updated[group_col] == 'Asgard']
    if not df_asgard.empty:
        total_asgard = len(df_asgard)
        asgard_hits = df_asgard[hit_flag_col].sum()
        print(f"\nAsgard Archaea:")
        print(f"  Total Asgard proteins: {total_asgard}")
        print(f"  Asgard proteins with eukaryotic hits: {asgard_hits} ({(asgard_hits/total_asgard*100 if total_asgard > 0 else 0.0):.1f}%)")

        # Asgard ESPs
        if esp_col in df_asgard.columns: 
            df_asgard_esps = df_asgard[df_asgard[esp_col] == True]
            if not df_asgard_esps.empty:
                total_asgard_esps = len(df_asgard_esps)
                asgard_esps_hits = df_asgard_esps[hit_flag_col].sum()
                print(f"  Asgard ESPs with eukaryotic hits: {asgard_esps_hits} ({(asgard_esps_hits/total_asgard_esps*100 if total_asgard_esps > 0 else 0.0):.1f}%)")
            else:
                print(f"  No Asgard ESPs identified in the Asgard subset (column '{esp_col}' has no True values).")
        else:
             print(f"  ESP column ('{esp_col}') was not successfully added. Skipping ESP hit analysis.")


        # Structurally Dark Asgard proteins
        if structurally_dark_col in df_asgard.columns:
            df_asgard_dark = df_asgard[df_asgard[structurally_dark_col] == True]
            if not df_asgard_dark.empty:
                total_asgard_dark = len(df_asgard_dark)
                asgard_dark_hits = df_asgard_dark[hit_flag_col].sum()
                print(f"  Structurally Dark Asgard proteins with eukaryotic hits: {asgard_dark_hits} ({(asgard_dark_hits/total_asgard_dark*100 if total_asgard_dark > 0 else 0.0):.1f}%)")
            else:
                print(f"  No Structurally Dark Asgard proteins identified.")
        else:
            print(f"  Structurally Dark column ('{structurally_dark_col}') not found. Skipping dark protein hit analysis.")

        # Domain-less Asgard proteins
        if num_domains_col in df_asgard.columns:
            df_asgard_domainless = df_asgard[df_asgard[num_domains_col].isna()]
            if not df_asgard_domainless.empty:
                total_asgard_domainless = len(df_asgard_domainless)
                asgard_domainless_hits = df_asgard_domainless[hit_flag_col].sum()
                print(f"  Domain-less Asgard proteins with eukaryotic hits: {asgard_domainless_hits} ({(asgard_domainless_hits/total_asgard_domainless*100 if total_asgard_domainless > 0 else 0.0):.1f}%)")
            else:
                print(f"  No Domain-less Asgard proteins identified.")
        else:
            print(f"  Num_Domains column ('{num_domains_col}') not found. Skipping domain-less protein hit analysis.")
    else:
        print("\nNo Asgard proteins found in the database.")

    # Giant Virus (GV) proteins
    df_gv_subset = df_main_updated[df_main_updated[group_col] == 'Virus'] 
    if not df_gv_subset.empty:
        total_gv = len(df_gv_subset)
        gv_hits = df_gv_subset[hit_flag_col].sum()
        print(f"\nGiant Viruses (GV):")
        print(f"  Total GV proteins: {total_gv}")
        print(f"  GV proteins with eukaryotic hits: {gv_hits} ({(gv_hits/total_gv*100 if total_gv > 0 else 0.0):.1f}%)")
    else:
        print("\nNo Giant Virus proteins found in the database.")

print("\n\n--- DIAMOND Hit Integration, ESP Definition, and Initial Analysis Complete ---")
print(f"Next steps could include:")
print(f"  - Detailed analysis of the '{best_hit_organism_col}' distribution for hits.")
print(f"  - Comparing domain architectures between Asgard ESPs and their eukaryotic hits (as per plans).")
print(f"  - Focusing on hits for specific proteins of interest (e.g., Ig-like domain proteins).")



In [ ]:
#!/usr/bin/env python3

"""
Updates the Euk_Hit_Organism column and adds Euk_Hit_Protein_Name information 
to the main protein database by parsing headers from a eukaryotic proteome FASTA file.
Version 1.2: Removed per-line debug logging to prevent performance issues.
"""

import pandas as pd
import numpy as np
from pathlib import Path
import time
import sys
import logging
import re # Added for stripping version in fallback lookup
# Ensure Biopython is installed: pip install biopython
try:
    from Bio import SeqIO 
except ImportError:
    print("ERROR: Biopython is required for this script. Please install it: pip install biopython")
    sys.exit(1)


# --- Setup Logging ---
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()
# Prevent adding handlers multiple times if script is re-run in interactive session
if not logger.hasHandlers():
    # Set level back to INFO to avoid excessive output
    logger.setLevel(logging.INFO) 
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(log_formatter)
    logger.addHandler(console_handler)
else:
    logger.setLevel(logging.INFO) # Ensure level is INFO


# --- Configuration ---
# INPUT: Main database CSV (should contain Euk_Hit_SSEQID and Euk_Hit_Organism columns)
db_with_hits_path = Path('proteome_database_combined_v1.8_euk_hits_esps.csv')

# INPUT: Eukaryotic proteome FASTA file with informative headers
euk_fasta_path = Path('euk63_proteomes_final.fasta') 

# OUTPUT: Updated database CSV file
output_db_updated_path = Path('proteome_database_combined_v1.9_euk_fasta_annots_updated_org.csv')

# --- Define Key Column Names ---
protein_id_col = 'ProteinID'
group_col = 'Group'
hit_flag_col = 'Has_Euk_DIAMOND_Hit'
euk_hit_sseqid_col = 'Euk_Hit_SSEQID' # The NCBI ID (e.g., XP_...) from DIAMOND
euk_hit_organism_col = 'Euk_Hit_Organism' # Existing column to UPDATE

# New column to add
euk_hit_protein_name_col = 'Euk_Hit_Protein_Name'

# --- Helper Function to Parse FASTA Headers (Improved ID Extraction) ---
def parse_euk_fasta_headers(fasta_file: Path) -> dict:
    """
    Parses a FASTA file to extract information from headers.
    Extracts ID before the first pipe '|' or space ' '.
    Assumes header format like: >ID|Genus species|Protein Name
    
    Returns:
        A dictionary mapping the extracted sequence ID (e.g., 'XP_020145405.1') to 
        {'Genus_Species': 'Genus species', 'Protein_Name': 'Protein Name'}
    """
    logging.info(f"Parsing FASTA headers from: {fasta_file}...")
    header_info = {}
    count = 0
    skipped_count = 0
    
    if not fasta_file.is_file():
        logging.error(f"FASTA file not found: {fasta_file}")
        return header_info

    try:
        for record in SeqIO.parse(fasta_file, "fasta"):
            count += 1
            # Use record.description (full header line after '>') for parsing
            description = record.description 
            
            # --- Improved ID Extraction ---
            # Find the first pipe or space
            first_pipe = description.find('|')
            first_space = description.find(' ')
            
            end_of_id = -1
            if first_pipe != -1 and first_space != -1:
                end_of_id = min(first_pipe, first_space)
            elif first_pipe != -1:
                end_of_id = first_pipe
            elif first_space != -1:
                end_of_id = first_space
                
            if end_of_id != -1:
                fasta_id_key = description[:end_of_id].strip()
            else:
                # If no pipe or space, the whole description might be the ID (unlikely for NCBI)
                fasta_id_key = description.strip() 
            # --- End Improved ID Extraction ---

            if not fasta_id_key:
                 skipped_count += 1
                 logging.warning(f"Could not extract valid ID Key from header: {description[:100]}...")
                 continue

            # --- Removed per-line debug logging ---
            # logging.debug(f"Record {count}: Extracted FASTA Key = '{fasta_id_key}' from header = '{description[:60]}...'")


            # Default values
            genus_species = "Unknown Species"
            protein_name = "Unknown Protein"

            # Attempt to parse based on pipe '|' delimiter using the full description
            parts = description.split('|')
            
            # We already extracted the ID, now parse the rest based on pipe count
            if len(parts) >= 3: 
                # Assumes ID|Genus species|Protein Name
                genus_species = parts[1].strip()
                protein_name = " | ".join(p.strip() for p in parts[2:]) # Join remaining parts
                    
            elif len(parts) == 2:
                # Might be ID|Protein Name or ID|Genus species
                second_part = parts[1].strip()
                # Heuristic: If it contains space and isn't all caps, assume species
                if ' ' in second_part and not second_part.isupper(): 
                    genus_species = second_part
                    protein_name = "Unknown Protein (Genus/Species only in header)"
                else:
                    genus_species = "Unknown Species (Name only in header)"
                    protein_name = second_part

            elif len(parts) == 1:
                 # No pipes. Try splitting by space after the extracted ID key
                 id_len = len(fasta_id_key)
                 # Find the first space *after* the ID key in the original description
                 first_space_after_id = description.find(' ', id_len)
                 if first_space_after_id != -1:
                     protein_name = description[first_space_after_id:].strip()
                     genus_species = "Unknown Species (Space delimited header)"
                 else:
                     protein_name = "Unknown Protein (ID only header)"
                     genus_species = "Unknown Species (ID only header)"

            # Store using the explicitly extracted ID key
            # Basic cleanup: replace underscores with spaces in genus_species if needed
            if genus_species != "Unknown Species":
                 genus_species = genus_species.replace('_', ' ')
                 
            header_info[fasta_id_key] = {
                'Genus_Species': genus_species,
                'Protein_Name': protein_name
            }

            if count % 50000 == 0:
                logging.info(f"  Processed {count:,} FASTA records...")

    except Exception as e:
        logging.error(f"Error parsing FASTA file {fasta_file}: {e}", exc_info=True) # Log traceback
        
    logging.info(f"Finished parsing. Extracted info for {len(header_info):,} sequences.")
    if skipped_count > 0:
        logging.warning(f"Skipped {skipped_count} records due to missing/unparseable IDs.")
        
    # Diagnostic: Print first 5 keys from the map
    if header_info:
        logging.info("First 5 keys extracted from FASTA headers (for verification):")
        keys_sample = list(header_info.keys())[:5]
        logging.info(keys_sample)
        
    return header_info

# --- Main Script Logic ---
def main():
    logging.info("--- Starting Eukaryotic Annotation Addition Script (v1.2) ---")
    start_time = time.time()

    # 1. Load Main Database
    logging.info(f"Loading main database: {db_with_hits_path}")
    if not db_with_hits_path.is_file():
        logging.error(f"Database file not found: {db_with_hits_path}")
        sys.exit(1)
    try:
        df_main = pd.read_csv(db_with_hits_path, low_memory=False)
        logging.info(f"Loaded database shape: {df_main.shape}")
    except Exception as e:
        logging.error(f"Failed to load database: {e}")
        sys.exit(1)

    # Ensure key columns exist
    required_cols = [protein_id_col, group_col, hit_flag_col, euk_hit_sseqid_col, euk_hit_organism_col]
    if not all(col in df_main.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df_main.columns]
        logging.error(f"Database is missing required columns: {missing}")
        sys.exit(1)

    # 2. Parse Eukaryotic FASTA Headers
    euk_header_map = parse_euk_fasta_headers(euk_fasta_path)
    if not euk_header_map:
        logging.warning("FASTA header map is empty. Cannot add annotations.")
        # Decide whether to exit or continue and save just the original data
        # For now, let's save the original data if no map is generated
        # sys.exit(1) 

    # 3. Add New Columns and Populate
    logging.info(f"Preparing to update '{euk_hit_organism_col}' and add '{euk_hit_protein_name_col}'")
    
    # Add the new protein name column if it doesn't exist
    if euk_hit_protein_name_col not in df_main.columns:
        df_main[euk_hit_protein_name_col] = np.nan
    else:
        logging.info(f"Column '{euk_hit_protein_name_col}' already exists. Values may be overwritten.")
        df_main[euk_hit_protein_name_col] = np.nan # Clear existing values before populating

    # Get indices of Asgard proteins with hits
    asgard_hit_indices = df_main[
        (df_main[group_col] == 'Asgard') & (df_main[hit_flag_col] == True)
    ].index

    logging.info(f"Processing {len(asgard_hit_indices):,} Asgard proteins with eukaryotic hits...")
    
    update_count = 0
    map_miss_count = 0
    overwritten_org_count = 0
    
    # Diagnostic: Print first 5 Euk_Hit_SSEQID values to be looked up
    if not asgard_hit_indices.empty:
         logging.info("First 5 Euk_Hit_SSEQIDs from database to be looked up (for verification):")
         logging.info(df_main.loc[asgard_hit_indices[:5], euk_hit_sseqid_col].tolist())
    
    # Iterate only through relevant rows
    for idx in asgard_hit_indices:
        # Get the ID from the database; ensure it's a string and clean it
        euk_sseqid_db = df_main.loc[idx, euk_hit_sseqid_col]
        if pd.isna(euk_sseqid_db) or not isinstance(euk_sseqid_db, str):
            continue 
        euk_sseqid_db = euk_sseqid_db.strip() # Clean whitespace
        if not euk_sseqid_db:
             continue
             
        # --- Removed per-line debug logging ---
        # logging.debug(f"Looking up DB ID: '{euk_sseqid_db}'")

        original_org_name = df_main.loc[idx, euk_hit_organism_col] # Get original name for comparison
        
        # Look up directly using the cleaned ID from the database
        hit_info = euk_header_map.get(euk_sseqid_db) 
        
        if hit_info:
            new_genus_species = hit_info.get('Genus_Species', 'Not Found in FASTA')
            new_protein_name = hit_info.get('Protein_Name', 'Not Found in FASTA')

            # Update columns
            # Check if we are overwriting a potentially useful existing organism name
            if not pd.isna(original_org_name) and original_org_name not in ['Unknown', 'Unknown Organism'] and original_org_name != new_genus_species:
                 # Log only if overwriting a non-generic name
                 if original_org_name != 'Not Found in FASTA':
                    logging.debug(f"Overwriting existing organism '{original_org_name}' with '{new_genus_species}' for hit {euk_sseqid_db}")
                    overwritten_org_count += 1
                 
            df_main.loc[idx, euk_hit_organism_col] = new_genus_species
            df_main.loc[idx, euk_hit_protein_name_col] = new_protein_name
            update_count += 1
        else:
             map_miss_count += 1
             # --- Removed per-line debug logging ---
             # logging.debug(f"Could not find header info for Euk_Hit_SSEQID: {euk_sseqid_db}")
             # Decide whether to clear existing info or leave it
             df_main.loc[idx, euk_hit_protein_name_col] = 'Not Found in FASTA'
             # Optionally clear organism name too if desired:
             # df_main.loc[idx, euk_hit_organism_col] = 'Not Found in FASTA'


        if (update_count + map_miss_count) % 25000 == 0 and (update_count + map_miss_count) > 0:
             logging.info(f"  Processed {update_count + map_miss_count:,} hits ({update_count:,} mapped, {map_miss_count:,} misses)...")

    logging.info(f"Finished processing. Updated annotations for {update_count:,} hits.")
    if overwritten_org_count > 0:
        logging.info(f"  Note: Overwrote existing organism names from DIAMOND parsing with FASTA header info for {overwritten_org_count:,} hits.")
    if map_miss_count > 0:
        logging.warning(f"Could not find header information for {map_miss_count:,} Euk_Hit_SSEQIDs in the provided FASTA file.")

    # 4. Save Updated Database
    logging.info(f"Saving updated database to: {output_db_updated_path}")
    try:
        # Ensure output directory exists
        output_db_updated_path.parent.mkdir(parents=True, exist_ok=True)
        df_main.to_csv(output_db_updated_path, index=False)
        logging.info("Successfully saved updated database.")
    except Exception as e:
        logging.error(f"Failed to save updated database: {e}")

    end_time = time.time()
    logging.info(f"--- Script finished in {end_time - start_time:.2f} seconds ---")

if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3

"""
Updates the Euk_Hit_Organism column (to Genus species) and adds 
Euk_Hit_Protein_Name information to the main protein database by parsing 
headers from a eukaryotic proteome FASTA file.
Version 1.4: Refined organism name parsing to Genus_species only.
"""

import pandas as pd
import numpy as np
from pathlib import Path
import time
import sys
import logging
import re 
# Ensure Biopython is installed: pip install biopython
try:
    from Bio import SeqIO 
except ImportError:
    print("ERROR: Biopython is required for this script. Please install it: pip install biopython")
    sys.exit(1)


# --- Setup Logging ---
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()
# Prevent adding handlers multiple times if script is re-run in interactive session
if not logger.hasHandlers():
    logger.setLevel(logging.INFO) 
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(log_formatter)
    logger.addHandler(console_handler)
else:
    logger.setLevel(logging.INFO) # Ensure level is INFO


# --- Configuration ---
# INPUT: Main database CSV (should contain Euk_Hit_SSEQID and Euk_Hit_Organism columns)
# Make sure this is the correct starting point (e.g., v1.8 before adding FASTA annotations)
db_with_hits_path = Path('proteome_database_combined_v1.9.csv') 

# INPUT: Eukaryotic proteome FASTA file with informative headers
euk_fasta_path = Path('euk63_proteomes_final.fasta') 

# OUTPUT: Updated database CSV file
# Adjusted filename to reflect the specific update
output_db_updated_path = Path('proteome_database_combined_v2.0.csv') 

# --- Define Key Column Names ---
protein_id_col = 'ProteinID'
group_col = 'Group'
hit_flag_col = 'Has_Euk_DIAMOND_Hit'
euk_hit_sseqid_col = 'Euk_Hit_SSEQID' # The NCBI ID (e.g., XP_...) from DIAMOND
euk_hit_organism_col = 'Euk_Hit_Organism' # Existing column to UPDATE

# New column to add
euk_hit_protein_name_col = 'Euk_Hit_Protein_Name'

# --- Helper Function to Parse FASTA Headers (Refined Organism Parsing) ---
def parse_euk_fasta_headers(fasta_file: Path) -> dict:
    """
    Parses a FASTA file to extract information from headers.
    Extracts ID before the first pipe '|' or space ' '.
    Extracts Genus species (first two words) for the organism name.
    Assumes header format like: >ID|Genus species strain|Protein Name
    
    Returns:
        A dictionary mapping the extracted sequence ID (e.g., 'XP_020145405.1') to 
        {'Genus_Species': 'Genus species', 'Protein_Name': 'Protein Name'}
    """
    logging.info(f"Parsing FASTA headers from: {fasta_file}...")
    header_info = {}
    count = 0
    skipped_count = 0
    
    if not fasta_file.is_file():
        logging.error(f"FASTA file not found: {fasta_file}")
        return header_info

    try:
        for record in SeqIO.parse(fasta_file, "fasta"):
            count += 1
            description = record.description 
            
            # --- ID Extraction (same as v1.2) ---
            first_pipe = description.find('|')
            first_space = description.find(' ')
            end_of_id = -1
            if first_pipe != -1 and first_space != -1: end_of_id = min(first_pipe, first_space)
            elif first_pipe != -1: end_of_id = first_pipe
            elif first_space != -1: end_of_id = first_space
            fasta_id_key = description[:end_of_id].strip() if end_of_id != -1 else description.strip()
            
            if not fasta_id_key:
                 skipped_count += 1
                 logging.warning(f"Could not extract valid ID Key from header: {description[:100]}...")
                 continue

            # --- Organism and Protein Name Parsing ---
            genus_species_raw = "Unknown Species"
            protein_name = "Unknown Protein"
            parts = description.split('|')

            if len(parts) >= 3: 
                # Assumes ID|Genus species strain etc|Protein Name ...
                genus_species_raw = parts[1].strip()
                protein_name = " | ".join(p.strip() for p in parts[2:]) 
            elif len(parts) == 2:
                # Might be ID|Protein Name or ID|Genus species strain etc
                second_part = parts[1].strip()
                if ' ' in second_part and not second_part.isupper(): 
                    genus_species_raw = second_part
                    protein_name = "Unknown Protein (Genus/Species only in header)"
                else:
                    genus_species_raw = "Unknown Species (Name only in header)"
                    protein_name = second_part
            elif len(parts) == 1:
                 id_len = len(fasta_id_key)
                 first_space_after_id = description.find(' ', id_len)
                 if first_space_after_id != -1:
                     # Assume the rest is protein name, cannot reliably get species
                     protein_name = description[first_space_after_id:].strip()
                     genus_species_raw = "Unknown Species (Space delimited header)"
                 else:
                     protein_name = "Unknown Protein (ID only header)"
                     genus_species_raw = "Unknown Species (ID only header)"

            # --- Refine Genus_Species ---
            genus_species_cleaned = "Unknown Species"
            if genus_species_raw != "Unknown Species":
                # Replace underscores, split by space, take first two words
                temp_name = genus_species_raw.replace('_', ' ').split()
                if len(temp_name) >= 2:
                    genus_species_cleaned = f"{temp_name[0]} {temp_name[1]}"
                elif len(temp_name) == 1:
                    genus_species_cleaned = temp_name[0] # Keep genus if only one word
                else:
                     genus_species_cleaned = genus_species_raw # Fallback if split fails unexpectedly
            # --- End Refine Genus_Species ---
                 
            header_info[fasta_id_key] = {
                'Genus_Species': genus_species_cleaned, # Store the cleaned version
                'Protein_Name': protein_name
            }

            if count % 50000 == 0:
                logging.info(f"  Processed {count:,} FASTA records...")

    except Exception as e:
        logging.error(f"Error parsing FASTA file {fasta_file}: {e}", exc_info=True) 
        
    logging.info(f"Finished parsing. Extracted info for {len(header_info):,} sequences.")
    if skipped_count > 0:
        logging.warning(f"Skipped {skipped_count} records due to missing/unparseable IDs.")
        
    # Diagnostic: Print first 5 keys and their parsed Genus_Species
    if header_info:
        logging.info("First 5 keys and parsed Genus_Species from FASTA headers (for verification):")
        keys_sample = list(header_info.keys())[:5]
        for key in keys_sample:
             logging.info(f"  Key: '{key}', Parsed Genus_Species: '{header_info[key]['Genus_Species']}'")
        
    return header_info

# --- Main Script Logic ---
def main():
    logging.info("--- Starting Eukaryotic Annotation Addition Script (v1.4 - Genus/Species Update) ---")
    start_time = time.time()

    # 1. Load Main Database
    logging.info(f"Loading main database: {db_with_hits_path}")
    if not db_with_hits_path.is_file():
        logging.error(f"Database file not found: {db_with_hits_path}")
        sys.exit(1)
    try:
        df_main = pd.read_csv(db_with_hits_path, low_memory=False)
        logging.info(f"Loaded database shape: {df_main.shape}")
    except Exception as e:
        logging.error(f"Failed to load database: {e}")
        sys.exit(1)

    # Ensure key columns exist
    required_cols = [protein_id_col, group_col, hit_flag_col, euk_hit_sseqid_col, euk_hit_organism_col]
    if not all(col in df_main.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df_main.columns]
        logging.error(f"Database is missing required columns: {missing}")
        sys.exit(1)

    # 2. Parse Eukaryotic FASTA Headers
    euk_header_map = parse_euk_fasta_headers(euk_fasta_path)
    if not euk_header_map:
        logging.warning("FASTA header map is empty. Cannot add annotations.")
        # Save the original data if no map is generated
        output_db_updated_path.parent.mkdir(parents=True, exist_ok=True)
        df_main.to_csv(output_db_updated_path, index=False)
        logging.info(f"Saved original database to {output_db_updated_path} as FASTA map was empty.")
        sys.exit(0) 

    # 3. Add New Columns and Populate
    logging.info(f"Preparing to update '{euk_hit_organism_col}' (with Genus species) and add '{euk_hit_protein_name_col}'")
    
    # Add the new protein name column if it doesn't exist
    if euk_hit_protein_name_col not in df_main.columns:
        df_main[euk_hit_protein_name_col] = np.nan
    else:
        logging.info(f"Column '{euk_hit_protein_name_col}' already exists. Values will be overwritten.")
        df_main[euk_hit_protein_name_col] = np.nan # Clear existing values before populating

    # Get indices of Asgard proteins with hits
    asgard_hit_indices = df_main[
        (df_main[group_col] == 'Asgard') & (df_main[hit_flag_col] == True)
    ].index

    logging.info(f"Processing {len(asgard_hit_indices):,} Asgard proteins with eukaryotic hits...")
    
    update_count = 0
    map_miss_count = 0
    overwritten_org_count = 0
    
    # Diagnostic: Print first 5 Euk_Hit_SSEQID values to be looked up
    if not asgard_hit_indices.empty:
         logging.info("First 5 Euk_Hit_SSEQIDs from database to be looked up (for verification):")
         logging.info(df_main.loc[asgard_hit_indices[:5], euk_hit_sseqid_col].tolist())
    
    # Iterate only through relevant rows
    for idx in asgard_hit_indices:
        # Get the ID from the database; ensure it's a string and clean it
        euk_sseqid_db = df_main.loc[idx, euk_hit_sseqid_col]
        if pd.isna(euk_sseqid_db) or not isinstance(euk_sseqid_db, str):
            continue 
        euk_sseqid_db = euk_sseqid_db.strip() 
        if not euk_sseqid_db:
             continue
             
        original_org_name = df_main.loc[idx, euk_hit_organism_col] 
        
        # Look up directly using the cleaned ID from the database
        hit_info = euk_header_map.get(euk_sseqid_db) 
        
        if hit_info:
            # Use the CLEANED Genus_Species from the map
            new_genus_species = hit_info.get('Genus_Species', 'Not Found in FASTA') 
            new_protein_name = hit_info.get('Protein_Name', 'Not Found in FASTA')

            # Update columns
            if not pd.isna(original_org_name) and original_org_name not in ['Unknown', 'Unknown Organism'] and original_org_name != new_genus_species:
                 if original_org_name != 'Not Found in FASTA':
                    logging.debug(f"Overwriting existing organism '{original_org_name}' with '{new_genus_species}' for hit {euk_sseqid_db}")
                    overwritten_org_count += 1
                 
            # Update the Euk_Hit_Organism column with the cleaned Genus_species
            df_main.loc[idx, euk_hit_organism_col] = new_genus_species 
            df_main.loc[idx, euk_hit_protein_name_col] = new_protein_name
            update_count += 1
        else:
             map_miss_count += 1
             # Keep existing Euk_Hit_Organism if lookup fails, but mark protein name as not found
             df_main.loc[idx, euk_hit_protein_name_col] = 'Not Found in FASTA'


        if (update_count + map_miss_count) % 25000 == 0 and (update_count + map_miss_count) > 0:
             logging.info(f"  Processed {update_count + map_miss_count:,} hits ({update_count:,} mapped, {map_miss_count:,} misses)...")

    logging.info(f"Finished processing. Updated annotations for {update_count:,} hits.")
    if overwritten_org_count > 0:
        logging.info(f"  Note: Overwrote existing organism names from DIAMOND parsing with FASTA header info for {overwritten_org_count:,} hits.")
    if map_miss_count > 0:
        logging.warning(f"Could not find header information for {map_miss_count:,} Euk_Hit_SSEQIDs in the provided FASTA file.")

    # 4. Save Updated Database
    logging.info(f"Saving updated database to: {output_db_updated_path}")
    try:
        # Ensure output directory exists
        output_db_updated_path.parent.mkdir(parents=True, exist_ok=True)
        df_main.to_csv(output_db_updated_path, index=False)
        logging.info("Successfully saved updated database.")
    except Exception as e:
        logging.error(f"Failed to save updated database: {e}")

    end_time = time.time()
    logging.info(f"--- Script finished in {end_time - start_time:.2f} seconds ---")

if __name__ == "__main__":
    main()


In [ ]:
# Cell: Integrate Giant Virus (GV) Eukaryotic DIAMOND Hit Information

import pandas as pd
from Bio import SeqIO
import sys
import os
import numpy as np # For NaN
import re # For parsing
import logging # For detailed logging
from pathlib import Path

# --- Setup Logging ---
# Ensures messages are printed to the notebook output
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger("GV_Euk_Hit_Integration") # Use a specific logger name
logger.handlers = [] # Clear existing handlers for this logger if re-running cell
logger.setLevel(logging.INFO) 
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)

# --- Configuration ---
MAIN_DB_INPUT_PATH = 'proteome_database_v2.3.csv'
GV_DIAMOND_RESULTS_PATH = "giant_virus_diamond_results/gv_vs_euk_diamond_hits.tsv" 
EUK_FASTA_PATH = 'euk63_proteomes_final.fasta' # Make sure this path is correct
MAIN_DB_OUTPUT_PATH = 'proteome_database_v2.4_gv_euk_hits.csv'
E_VALUE_THRESHOLD = 1e-10

# Define column names for DIAMOND output (12 standard + qlen, slen)
DIAMOND_COLS = ['qseqid_full', 'sseqid_full', 'pident', 'length', 'mismatch', 'gapopen',
                'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'qlen', 'slen']

# --- Helper Function to Parse Eukaryotic FASTA Headers ---
def parse_euk_fasta_headers_for_hits(fasta_file: Path) -> dict:
    """
    Parses a FASTA file to extract information from headers.
    Extracts ID before the first pipe '|' or space ' '.
    Attempts to extract Genus species and Protein Name.
    """
    logger.info(f"Parsing Eukaryotic FASTA headers from: {fasta_file} for hit annotation...")
    header_info = {}
    count = 0
    skipped_count = 0
    
    if not fasta_file.is_file():
        logger.error(f"Eukaryotic FASTA file not found: {fasta_file}")
        return header_info

    try:
        for record in SeqIO.parse(fasta_file, "fasta"):
            count += 1
            description = record.description 
            
            first_pipe_idx = description.find('|')
            first_space_idx = description.find(' ')
            end_of_id = -1
            if first_pipe_idx != -1 and (first_space_idx == -1 or first_pipe_idx < first_space_idx):
                end_of_id = first_pipe_idx
            elif first_space_idx != -1:
                end_of_id = first_space_idx
            fasta_id_key = description[:end_of_id].strip() if end_of_id != -1 else description.strip()
            
            if not fasta_id_key:
                 skipped_count += 1
                 logger.warning(f"Could not extract valid ID Key from header: {description[:100]}...")
                 continue

            genus_species = "Unknown Species"
            protein_name = "Unknown Protein"
            name_part = description[len(fasta_id_key):].strip().lstrip('|').strip() # Part after ID

            organism_match = re.search(r"\[(.*?)\]$", name_part) # Check for [Genus species] at the end
            if organism_match:
                genus_species = organism_match.group(1).strip()
                protein_name = name_part[:organism_match.start()].strip()
                if not protein_name: protein_name = "Unknown Protein (Organism in brackets)"
            else:
                pipe_parts = name_part.split('|')
                if len(pipe_parts) >= 1: # Take first part as protein name if no brackets
                    protein_name = pipe_parts[0].strip()
                    if len(pipe_parts) > 1: # If more parts, could be organism or more name
                        # This part is heuristic; NCBI headers vary.
                        # If a part after the first clearly looks like a species name, use it.
                        # For now, we prioritize [Genus species] format.
                        # If you have specific common formats for organism in pipe-delimited headers, add logic here.
                        pass 


            if genus_species != "Unknown Species":
                gs_parts = genus_species.replace('_', ' ').split()
                genus_species = " ".join(gs_parts[:2]) if len(gs_parts) >= 2 else gs_parts[0] if gs_parts else "Unknown Species"
            
            header_info[fasta_id_key] = {
                'Genus_Species': genus_species,
                'Protein_Name': protein_name if protein_name else "Unknown Protein"
            }

            if count % 200000 == 0: 
                logger.info(f"  Processed {count:,} Eukaryotic FASTA records...")
    except Exception as e:
        logger.error(f"Error parsing Eukaryotic FASTA file {fasta_file}: {e}", exc_info=True)
        
    logger.info(f"Finished Eukaryotic FASTA parsing. Extracted info for {len(header_info):,} sequences.")
    if skipped_count > 0: logger.warning(f"Skipped {skipped_count} Eukaryotic FASTA records due to ID parsing issues.")
    return header_info

# --- Main Processing Logic ---
logger.info(f"--- Starting GV Eukaryotic Hit Integration ---")
logger.info(f"Reading main database: {MAIN_DB_INPUT_PATH}")
try:
    df_main = pd.read_csv(MAIN_DB_INPUT_PATH, low_memory=False)
    df_main['ProteinID'] = df_main['ProteinID'].astype(str)
    if 'Original_Seq_Length' not in df_main.columns:
        logger.warning("'Original_Seq_Length' not found, trying 'Length' for query coverage.")
        if 'Length' in df_main.columns:
             df_main['Original_Seq_Length'] = df_main['Length']
        else:
             logger.error("Neither 'Original_Seq_Length' nor 'Length' found. Cannot calculate query coverage accurately for GVs.")
             df_main['Original_Seq_Length'] = np.nan
    logger.info(f"Main database loaded. Shape: {df_main.shape}")
except FileNotFoundError:
    logger.error(f"Main database file not found: {MAIN_DB_INPUT_PATH}")
    sys.exit(1)
except Exception as e:
    logger.error(f"Error loading main database: {e}")
    sys.exit(1)

logger.info(f"Reading GV DIAMOND results: {GV_DIAMOND_RESULTS_PATH}")
if not Path(GV_DIAMOND_RESULTS_PATH).is_file():
    logger.error(f"GV DIAMOND results file not found: {GV_DIAMOND_RESULTS_PATH}. Please provide the correct path.")
    sys.exit(1)
try:
    df_gv_hits_raw = pd.read_csv(GV_DIAMOND_RESULTS_PATH, sep='\t', header=None, names=DIAMOND_COLS)
    # Parse GV ProteinID from qseqid_full (e.g., take part before first '|')
    df_gv_hits_raw['qseqid'] = df_gv_hits_raw['qseqid_full'].astype(str).str.split('|').str[0]
    df_gv_hits_raw['sseqid_full'] = df_gv_hits_raw['sseqid_full'].astype(str)
except Exception as e:
    logger.error(f"Error loading GV DIAMOND results: {e}")
    sys.exit(1)

logger.info(f"  Read {len(df_gv_hits_raw)} raw GV DIAMOND alignments.")
df_gv_hits_filtered = df_gv_hits_raw[df_gv_hits_raw['evalue'] <= E_VALUE_THRESHOLD].copy()
logger.info(f"  Found {len(df_gv_hits_filtered)} alignments passing e-value <= {E_VALUE_THRESHOLD}.")

if df_gv_hits_filtered.empty:
    logger.warning("No GV DIAMOND hits passed the e-value threshold. No GV eukaryotic hits will be added.")
    df_main.to_csv(MAIN_DB_OUTPUT_PATH, index=False, na_rep='NA')
    logger.info(f"Saved original database (no GV hits added) to: {MAIN_DB_OUTPUT_PATH}")
else:
    df_gv_hits_filtered.sort_values(by=['qseqid', 'evalue', 'bitscore'], ascending=[True, True, False], inplace=True)
    df_gv_best_hits = df_gv_hits_filtered.drop_duplicates(subset=['qseqid'], keep='first').copy()
    logger.info(f"  Identified {len(df_gv_best_hits)} unique GV proteins with significant eukaryotic hits.")

    euk_header_map = parse_euk_fasta_headers_for_hits(Path(EUK_FASTA_PATH))
    if not euk_header_map:
        logger.warning("Eukaryotic FASTA header map is empty. Organism and protein names for GV hits will be 'Unknown'.")

    gv_hit_data_list = []
    for _, row in df_gv_best_hits.iterrows():
        gv_protein_id = row['qseqid'] # Already parsed clean GV ID
        euk_sseqid_full = row['sseqid_full']
        euk_sseqid_clean = euk_sseqid_full.split('|')[0].split(' ')[0]

        header_data = euk_header_map.get(euk_sseqid_clean, {})
        euk_organism = header_data.get('Genus_Species', 'Unknown Organism')
        euk_prot_name = header_data.get('Protein_Name', 'Unknown Protein Name')

        gv_protein_entry = df_main[df_main['ProteinID'] == gv_protein_id]
        q_len_from_db = np.nan
        if not gv_protein_entry.empty and 'Original_Seq_Length' in gv_protein_entry.columns and pd.notna(gv_protein_entry['Original_Seq_Length'].iloc[0]):
            q_len_from_db = gv_protein_entry['Original_Seq_Length'].iloc[0]
        if pd.isna(q_len_from_db) or q_len_from_db == 0: # Fallback to DIAMOND qlen if not found or zero
             q_len_from_db = row['qlen']


        query_cov = (row['qend'] - row['qstart'] + 1) / q_len_from_db if q_len_from_db > 0 else 0
        subject_cov = (row['send'] - row['sstart'] + 1) / row['slen'] if row['slen'] > 0 else 0
        
        gv_hit_data_list.append({
            'ProteinID': gv_protein_id,
            'Has_Euk_DIAMOND_Hit': True, # This will be the final column name
            'Euk_Hit_SSEQID': euk_sseqid_clean,
            'Euk_Hit_Organism': euk_organism,
            'Euk_Hit_PIDENT': row['pident'],
            'Euk_Hit_EVALUE': row['evalue'],
            'Euk_Hit_Protein_Name': euk_prot_name,
            'Euk_Hit_Qstart': row['qstart'],
            'Euk_Hit_Qend': row['qend'],
            'Euk_Hit_Sstart': row['sstart'],
            'Euk_Hit_Send': row['send'],
            'Euk_Hit_Slen_Diamond': row['slen'],
            'Query_Coverage': query_cov,
            'Subject_Coverage': subject_cov
        })
    
    df_gv_hits_to_update = pd.DataFrame(gv_hit_data_list)

    if not df_gv_hits_to_update.empty:
        logger.info(f"Updating {len(df_gv_hits_to_update)} GV proteins with their best eukaryotic hit details.")
        
        # Ensure target Euk_Hit columns exist in df_main, creating them if necessary
        euk_hit_cols = [
            'Has_Euk_DIAMOND_Hit', 'Euk_Hit_SSEQID', 'Euk_Hit_Organism', 
            'Euk_Hit_PIDENT', 'Euk_Hit_EVALUE', 'Euk_Hit_Protein_Name',
            'Euk_Hit_Qstart', 'Euk_Hit_Qend', 'Euk_Hit_Sstart', 'Euk_Hit_Send',
            'Euk_Hit_Slen_Diamond', 'Query_Coverage', 'Subject_Coverage'
        ]
        for col in euk_hit_cols:
            if col not in df_main.columns:
                df_main[col] = np.nan if col != 'Has_Euk_DIAMOND_Hit' else False
                logger.info(f"Added missing column to main DataFrame: {col}")
        
        # Set ProteinID as index for efficient update
        df_main = df_main.set_index('ProteinID')
        df_gv_hits_to_update = df_gv_hits_to_update.set_index('ProteinID')

        # Update only GV rows
        gv_indices = df_main[df_main['Group'] == 'GV'].index
        common_indices = gv_indices.intersection(df_gv_hits_to_update.index)
        
        logger.info(f"Found {len(common_indices)} GV proteins with hits to update in df_main.")

        for col in df_gv_hits_to_update.columns: # Iterate through columns in the hits dataframe
            df_main.loc[common_indices, col] = df_gv_hits_to_update.loc[common_indices, col]

        # For GV proteins that did NOT have a hit, ensure 'Has_Euk_DIAMOND_Hit' is False
        # And other Euk_Hit fields are NaN
        gv_without_hits_indices = gv_indices.difference(df_gv_hits_to_update.index)
        if not gv_without_hits_indices.empty:
            logger.info(f"Setting 'Has_Euk_DIAMOND_Hit' to False for {len(gv_without_hits_indices)} GV proteins without hits.")
            df_main.loc[gv_without_hits_indices, 'Has_Euk_DIAMOND_Hit'] = False
            for col in euk_hit_cols: # Clear other Euk_Hit fields for GVs without hits
                if col != 'Has_Euk_DIAMOND_Hit':
                    df_main.loc[gv_without_hits_indices, col] = np.nan
        
        df_main.reset_index(inplace=True) # Restore ProteinID as a column
        
        num_gv_hits_in_main = df_main[(df_main['Group'] == 'GV') & (df_main['Has_Euk_DIAMOND_Hit'] == True)].shape[0]
        logger.info(f"Total GV proteins now flagged with eukaryotic hits in main DB: {num_gv_hits_in_main}")
    else:
        logger.info("No GV eukaryotic hits to merge after processing.")
        # Ensure 'Has_Euk_DIAMOND_Hit' is False for all GVs if no hits were processed
        df_main.loc[df_main['Group'] == 'GV', 'Has_Euk_DIAMOND_Hit'] = df_main.loc[df_main['Group'] == 'GV', 'Has_Euk_DIAMOND_Hit'].fillna(False)

    try:
        df_main.to_csv(MAIN_DB_OUTPUT_PATH, index=False, na_rep='NA')
        logger.info(f"Successfully saved updated database to: {MAIN_DB_OUTPUT_PATH}")
    except Exception as e:
        logger.error(f"Error writing updated database CSV: {e}")

logger.info("--- GV Eukaryotic Hit Integration Cell Finished ---")

# --- Example of how to run this cell ---
# Make sure the paths are correctly set above.
# Then, you would typically run this cell in your Jupyter notebook.
# For standalone script execution, you might wrap the main logic in a function
# and call it from an if __name__ == "__main__": block, passing paths as arguments.
# Example (if this were a script, not a notebook cell):
# if __name__ == "__main__":
#     add_gv_eukaryotic_hits(
#         main_db_path='proteome_database_v2.3.csv',
#         gv_diamond_results_path='YOUR_GV_vs_EUK_DIAMOND_RESULTS.tsv', # !!! REPLACE !!!
#         euk_fasta_path_str='euk63_proteomes_final.fasta',
#         output_db_path='proteome_database_v2.4_gv_euk_hits.csv'
#     )


In [ ]:
# Cell: Integrate Giant Virus (GV) Eukaryotic DIAMOND Hit Information (v2 - Standardized Organism Names)

import pandas as pd
from Bio import SeqIO
import sys
import os
import numpy as np # For NaN
import re # For parsing
import logging # For detailed logging
from pathlib import Path

# --- Setup Logging ---
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger("GV_Euk_Hit_Integration_V2") 
logger.handlers = [] 
logger.setLevel(logging.INFO) 
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)

# --- Configuration ---
MAIN_DB_INPUT_PATH = 'proteome_database_v2.3.csv' # Input is v2.3
GV_DIAMOND_RESULTS_PATH = "giant_virus_diamond_results/gv_vs_euk_diamond_hits.tsv" 
EUK_FASTA_PATH = 'euk63_proteomes_final.fasta' 
MAIN_DB_OUTPUT_PATH = 'proteome_database_v2.5_gv_euk_hits_std_org.csv' # Output will be v2.5
E_VALUE_THRESHOLD = 1e-10

DIAMOND_COLS = ['qseqid_full', 'sseqid_full', 'pident', 'length', 'mismatch', 'gapopen',
                'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'qlen', 'slen']

# --- Helper Function to Parse Eukaryotic FASTA Headers (Refined for Genus species) ---
def parse_euk_fasta_headers_for_hits_std_org(fasta_file: Path) -> dict:
    logger.info(f"Parsing Eukaryotic FASTA headers from: {fasta_file} for hit annotation (std org)...")
    header_info = {}
    count = 0; skipped_count = 0
    if not fasta_file.is_file():
        logger.error(f"Eukaryotic FASTA file not found: {fasta_file}")
        return header_info
    try:
        for record in SeqIO.parse(fasta_file, "fasta"):
            count += 1
            description = record.description
            
            first_pipe_idx = description.find('|')
            first_space_idx = description.find(' ')
            end_of_id = -1
            if first_pipe_idx != -1 and (first_space_idx == -1 or first_pipe_idx < first_space_idx):
                end_of_id = first_pipe_idx
            elif first_space_idx != -1:
                end_of_id = first_space_idx
            fasta_id_key = description[:end_of_id].strip() if end_of_id != -1 else description.strip()
            
            if not fasta_id_key:
                 skipped_count += 1; continue

            genus_species_raw = "Unknown Species"
            protein_name = "Unknown Protein"
            remaining_description = description[len(fasta_id_key):].lstrip().lstrip('|').strip()

            organism_match_in_brackets = re.search(r"\[(.*?)\]$", remaining_description)
            if organism_match_in_brackets:
                genus_species_raw = organism_match_in_brackets.group(1).strip()
                protein_name = remaining_description[:organism_match_in_brackets.start()].strip()
                if not protein_name: protein_name = "Unknown Protein (Organism in brackets)"
            else:
                parts_after_id = remaining_description.split('|')
                if len(parts_after_id) >= 2: # Format: ID|Organism Info|Protein Name
                    genus_species_raw = parts_after_id[0].strip()
                    protein_name = " | ".join(p.strip() for p in parts_after_id[1:])
                elif len(parts_after_id) == 1 and parts_after_id[0]: # Format: ID|Protein Name (or ID|Organism if only one word)
                    # Heuristic: if it contains spaces, assume it's an organism name attempt
                    if ' ' in parts_after_id[0] or '_' in parts_after_id[0]:
                        genus_species_raw = parts_after_id[0].strip()
                        protein_name = "Unknown Protein (Organism only after ID)"
                    else: # Assume it's protein name
                        protein_name = parts_after_id[0].strip()
                elif not parts_after_id and remaining_description: 
                     protein_name = remaining_description
            
            genus_species_cleaned = "Unknown Species"
            if genus_species_raw and genus_species_raw != "Unknown Species":
                temp_name_parts = genus_species_raw.replace('_', ' ').split()
                if len(temp_name_parts) >= 2:
                    genus_species_cleaned = f"{temp_name_parts[0]} {temp_name_parts[1]}"
                elif len(temp_name_parts) == 1:
                    genus_species_cleaned = temp_name_parts[0]
            
            header_info[fasta_id_key] = {
                'Genus_Species': genus_species_cleaned,
                'Protein_Name': protein_name if protein_name else "Unknown Protein"
            }
            if count % 200000 == 0: logger.info(f"  Processed {count:,} Eukaryotic FASTA records...")
    except Exception as e:
        logger.error(f"Error parsing Eukaryotic FASTA file {fasta_file}: {e}", exc_info=True)
    logger.info(f"Finished Eukaryotic FASTA parsing. Extracted info for {len(header_info):,} sequences (Skipped: {skipped_count}).")
    return header_info

# --- Main Processing Logic (largely same as before, uses new helper) ---
logger.info(f"--- Starting GV Eukaryotic Hit Integration (v2) ---")
# ... (rest of the GV hit integration logic from the previous cell, ensuring it calls 
#      parse_euk_fasta_headers_for_hits_std_org and saves to MAIN_DB_OUTPUT_PATH)
# The following is a condensed version of that logic:

logger.info(f"Reading main database: {MAIN_DB_INPUT_PATH}")
try:
    df_main = pd.read_csv(MAIN_DB_INPUT_PATH, low_memory=False)
    df_main['ProteinID'] = df_main['ProteinID'].astype(str)
    if 'Original_Seq_Length' not in df_main.columns:
        df_main['Original_Seq_Length'] = df_main.get('Length', pd.Series(np.nan, index=df_main.index))
    logger.info(f"Main database loaded. Shape: {df_main.shape}")
except Exception as e:
    logger.error(f"Error loading main database: {e}"); sys.exit(1)

logger.info(f"Reading GV DIAMOND results: {GV_DIAMOND_RESULTS_PATH}")
if not Path(GV_DIAMOND_RESULTS_PATH).is_file():
    logger.error(f"GV DIAMOND results file not found: {GV_DIAMOND_RESULTS_PATH}."); sys.exit(1)
try:
    df_gv_hits_raw = pd.read_csv(GV_DIAMOND_RESULTS_PATH, sep='\t', header=None, names=DIAMOND_COLS)
    df_gv_hits_raw['qseqid'] = df_gv_hits_raw['qseqid_full'].astype(str).str.split('|').str[0]
    df_gv_hits_raw['sseqid_full'] = df_gv_hits_raw['sseqid_full'].astype(str)
except Exception as e:
    logger.error(f"Error loading GV DIAMOND results: {e}"); sys.exit(1)

df_gv_hits_filtered = df_gv_hits_raw[df_gv_hits_raw['evalue'] <= E_VALUE_THRESHOLD].copy()
logger.info(f"  Found {len(df_gv_hits_filtered)} GV DIAMOND alignments passing e-value.")

if df_gv_hits_filtered.empty:
    logger.warning("No GV DIAMOND hits passed e-value. Saving original DB.")
    df_main.to_csv(MAIN_DB_OUTPUT_PATH, index=False, na_rep='NA')
else:
    df_gv_hits_filtered.sort_values(by=['qseqid', 'evalue', 'bitscore'], ascending=[True, True, False], inplace=True)
    df_gv_best_hits = df_gv_hits_filtered.drop_duplicates(subset=['qseqid'], keep='first').copy()
    logger.info(f"  Identified {len(df_gv_best_hits)} unique GV proteins with Euk hits.")

    euk_header_map = parse_euk_fasta_headers_for_hits_std_org(Path(EUK_FASTA_PATH)) # Call refined parser

    gv_hit_data_list = []
    for _, row in df_gv_best_hits.iterrows():
        gv_protein_id = row['qseqid']
        euk_sseqid_full = row['sseqid_full']
        euk_sseqid_clean = euk_sseqid_full.split('|')[0].split(' ')[0]
        header_data = euk_header_map.get(euk_sseqid_clean, {})
        euk_organism = header_data.get('Genus_Species', 'Unknown Organism') # This will now be "Genus species"
        euk_prot_name = header_data.get('Protein_Name', 'Unknown Protein Name')
        
        q_len_from_db_series = df_main.loc[df_main['ProteinID'] == gv_protein_id, 'Original_Seq_Length']
        q_len_from_db = q_len_from_db_series.iloc[0] if not q_len_from_db_series.empty and pd.notna(q_len_from_db_series.iloc[0]) else row['qlen']
        if pd.isna(q_len_from_db) or q_len_from_db == 0: q_len_from_db = row['qlen']

        query_cov = (row['qend'] - row['qstart'] + 1) / q_len_from_db if q_len_from_db > 0 else 0
        subject_cov = (row['send'] - row['sstart'] + 1) / row['slen'] if row['slen'] > 0 else 0
        
        gv_hit_data_list.append({
            'ProteinID': gv_protein_id, 'Has_Euk_DIAMOND_Hit': True,
            'Euk_Hit_SSEQID': euk_sseqid_clean, 'Euk_Hit_Organism': euk_organism,
            'Euk_Hit_PIDENT': row['pident'], 'Euk_Hit_EVALUE': row['evalue'],
            'Euk_Hit_Protein_Name': euk_prot_name, 'Euk_Hit_Qstart': row['qstart'],
            'Euk_Hit_Qend': row['qend'], 'Euk_Hit_Sstart': row['sstart'],
            'Euk_Hit_Send': row['send'], 'Euk_Hit_Slen_Diamond': row['slen'],
            'Query_Coverage': query_cov, 'Subject_Coverage': subject_cov
        })
    
    df_gv_hits_to_update = pd.DataFrame(gv_hit_data_list)

    if not df_gv_hits_to_update.empty:
        logger.info(f"Updating {len(df_gv_hits_to_update)} GV proteins with Euk hit details.")
        euk_hit_cols = ['Has_Euk_DIAMOND_Hit', 'Euk_Hit_SSEQID', 'Euk_Hit_Organism', 
                        'Euk_Hit_PIDENT', 'Euk_Hit_EVALUE', 'Euk_Hit_Protein_Name',
                        'Euk_Hit_Qstart', 'Euk_Hit_Qend', 'Euk_Hit_Sstart', 'Euk_Hit_Send',
                        'Euk_Hit_Slen_Diamond', 'Query_Coverage', 'Subject_Coverage']
        for col in euk_hit_cols:
            if col not in df_main.columns:
                df_main[col] = np.nan if col != 'Has_Euk_DIAMOND_Hit' else False
        
        df_main = df_main.set_index('ProteinID')
        df_gv_hits_to_update = df_gv_hits_to_update.set_index('ProteinID')
        gv_indices = df_main[df_main['Group'] == 'GV'].index
        common_indices = gv_indices.intersection(df_gv_hits_to_update.index)
        
        for col in df_gv_hits_to_update.columns:
            df_main.loc[common_indices, col] = df_gv_hits_to_update.loc[common_indices, col]
        
        gv_without_hits_indices = gv_indices.difference(df_gv_hits_to_update.index)
        if not gv_without_hits_indices.empty:
            df_main.loc[gv_without_hits_indices, 'Has_Euk_DIAMOND_Hit'] = False
            for col in euk_hit_cols:
                if col != 'Has_Euk_DIAMOND_Hit':
                    df_main.loc[gv_without_hits_indices, col] = np.nan
        df_main.reset_index(inplace=True)
        logger.info(f"Total GV proteins now flagged with Euk hits: {df_main[(df_main['Group'] == 'GV') & (df_main['Has_Euk_DIAMOND_Hit'] == True)].shape[0]}")
    else:
        logger.info("No GV Euk hits to merge after processing.")
        df_main.loc[df_main['Group'] == 'GV', 'Has_Euk_DIAMOND_Hit'] = df_main.loc[df_main['Group'] == 'GV', 'Has_Euk_DIAMOND_Hit'].fillna(False)

    try:
        df_main.to_csv(MAIN_DB_OUTPUT_PATH, index=False, na_rep='NA')
        logger.info(f"Successfully saved updated database to: {MAIN_DB_OUTPUT_PATH}")
    except Exception as e:
        logger.error(f"Error writing updated database CSV: {e}")

logger.info("--- GV Eukaryotic Hit Integration Cell (v2) Finished ---")



In [ ]:
# Cell: Merge Intra-OG APSI Data

import pandas as pd
import numpy as np
import os
import logging

# --- Configuration ---

# Input: Path to the main database CSV created by a previous cell
# Adjust this path to match the output of your main assembly cell
MAIN_DB_INPUT_PATH = "proteome_database_v2.5.csv" 

# Input: Path to the APSI values CSV file
APSI_CSV_PATH = "output_summary_data_hit_validation_phase1/intra_og_apsi_values.csv" # Make sure this path is correct

# Output: Path to save the database CSV after merging APSI data
# You can overwrite the input file or save to a new file
MAIN_DB_OUTPUT_PATH = "proteome_database_v2.6.csv" 

# Column name in the main database DataFrame that contains the Orthogroup ID
MAIN_DB_OG_COLUMN = "Orthogroup" # Verify this matches your main DataFrame

# Column name in the APSI CSV file that contains the Orthogroup ID
APSI_DB_OG_COLUMN = "Orthogroup" # Verify this matches the APSI CSV file

# --- Setup Logging (Optional, but good practice) ---
logger = logging.getLogger(__name__) 
# Basic configuration if not already set up in the notebook
if not logger.hasHandlers():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

# --- Main Logic ---
logger.info("--- Starting APSI Merge Cell ---")
logger.info(f"Reading main database from: {MAIN_DB_INPUT_PATH}")

try:
    # Load the main database CSV
    df_main = pd.read_csv(MAIN_DB_INPUT_PATH, low_memory=False)
    logger.info(f"Loaded {len(df_main):,} records from main database.")
    logger.info(f"Main DB columns: {df_main.columns.tolist()}")

    # Check if the main OG column exists
    if MAIN_DB_OG_COLUMN not in df_main.columns:
        logger.error(f"Required column '{MAIN_DB_OG_COLUMN}' not found in {MAIN_DB_INPUT_PATH}. Cannot merge APSI.")
        raise KeyError(f"Column '{MAIN_DB_OG_COLUMN}' not found.")

    logger.info(f"Reading APSI data from: {APSI_CSV_PATH}")
    # Load the APSI data CSV
    df_apsi = pd.read_csv(APSI_CSV_PATH)
    logger.info(f"Loaded {len(df_apsi):,} records from APSI file.")
    logger.info(f"APSI DB columns: {df_apsi.columns.tolist()}")

    # Check if the APSI OG column exists
    if APSI_DB_OG_COLUMN not in df_apsi.columns:
        logger.error(f"Required column '{APSI_DB_OG_COLUMN}' not found in {APSI_CSV_PATH}. Cannot merge.")
        raise KeyError(f"Column '{APSI_DB_OG_COLUMN}' not found.")

    # Prepare APSI DataFrame for merging
    # Rename columns for clarity and to avoid potential conflicts
    df_apsi_renamed = df_apsi.rename(columns={
        'APSI': 'Intra_OG_APSI',
        'Num_Sequences': 'Num_OG_Sequences',
        APSI_DB_OG_COLUMN: 'APSI_Orthogroup_Key' # Use a temporary unique key name
    })
    
    # Select only necessary columns from APSI data
    apsi_cols_to_merge = ['APSI_Orthogroup_Key', 'Intra_OG_APSI', 'Num_OG_Sequences']
    df_apsi_to_merge = df_apsi_renamed[apsi_cols_to_merge]

    # Perform the merge
    logger.info(f"Merging APSI data into main database on '{MAIN_DB_OG_COLUMN}' <-> '{APSI_DB_OG_COLUMN}'")
    # Keep track of original columns to see if merge adds columns unexpectedly
    original_cols = set(df_main.columns)
    
    df_merged = df_main.merge(
        df_apsi_to_merge,
        left_on=MAIN_DB_OG_COLUMN,
        right_on='APSI_Orthogroup_Key',
        how='left' # Keep all rows from the main database
    )

    # Check if merge was successful and drop the temporary key
    if 'Intra_OG_APSI' in df_merged.columns:
        logger.info("Merge successful. Added 'Intra_OG_APSI' and 'Num_OG_Sequences' columns.")
        # Drop the temporary key column used for merging
        if 'APSI_Orthogroup_Key' in df_merged.columns:
            df_merged.drop(columns=['APSI_Orthogroup_Key'], inplace=True)
        
        # Report merge statistics
        apsi_merged_count = df_merged['Intra_OG_APSI'].notna().sum()
        total_rows = len(df_merged)
        logger.info(f"Number of proteins with merged APSI values: {apsi_merged_count} / {total_rows} ({apsi_merged_count/total_rows:.2%})")
        
        # Check if the number of rows changed unexpectedly
        if len(df_merged) != len(df_main):
             logger.warning(f"Row count changed after merge! Original: {len(df_main)}, Merged: {len(df_merged)}")
             
    else:
        logger.warning("Merge completed, but 'Intra_OG_APSI' column was not added. Check column names and merge keys.")
        # Ensure placeholder columns exist if merge failed to add them
        if 'Intra_OG_APSI' not in df_merged.columns: df_merged['Intra_OG_APSI'] = pd.NA
        if 'Num_OG_Sequences' not in df_merged.columns: df_merged['Num_OG_Sequences'] = pd.NA


    # Save the updated DataFrame
    logger.info(f"Saving updated database with APSI info to: {MAIN_DB_OUTPUT_PATH}")
    df_merged.to_csv(MAIN_DB_OUTPUT_PATH, index=False, na_rep='NA')
    logger.info("Successfully saved updated database.")

except FileNotFoundError as e:
    logger.error(f"File not found error: {e}. Please check input paths.")
except KeyError as e:
    logger.error(f"Column name error: {e}. Please check column names in configuration and CSV files.")
except Exception as e:
    logger.error(f"An unexpected error occurred during APSI merge: {e}", exc_info=True) # Log traceback

logger.info("--- APSI Merge Cell Finished ---")

# Display the first few rows and info of the merged dataframe (optional)
# print("\nMerged DataFrame Info:")
# df_merged.info()
# print("\nMerged DataFrame Head:")
# print(df_merged.head())
